In this notebook, we'll use what we learned via GraphQL introspection to query the CDS database for cases and assocaited files that we can export for analysis
A few useful links:

The CDS Data Model: https://dataservice.datacommons.cancer.gov/#/resources
The CDS GraphQL endpoint: https://dataservice.datacommons.cancer.gov/v1/graphql/
The GraphiQL interface in CDS (a good place to build and practice queries): https://dataservice.datacommons.cancer.gov/#/graphql


As before we'll import a few useful libraries and set up a routine to run queries and return JSON results

In [1]:
import pandas as pd
import requests
from IPython.display import display, Markdown, Latex
import json

In [2]:
cds_graphql_url = "https://dataservice.datacommons.cancer.gov/v1/graphql/"

In [3]:
def runGraphQLQuery(url,query,variables):
    headers = {"content-type":"application/json"}
    try:
        if variables is None:
            results = requests.post(url = url, json={"query":query, "headers":headers})
        else:
            results = requests.post(url=url, json={"query":query, "headers":headers, "variables":variables})
        results = results.json()
    except requests.exceptions.HTTPError as exception:
        print(exception)
    return results

One potential starting point would be to find cases associated with a specific disease.  CDS has many different diseases, so a first step would be to query the system and get a listing.  CDS stores this information in the "primary_diagnosis" field in the diagnosis section of the CDS data model

In [4]:
disease_query = """
{
  diagnosis{
    primary_diagnosis
  }
}
"""

In [5]:
diseases = runGraphQLQuery(cds_graphql_url, disease_query, None)

In [6]:
diseases = diseases['data']['diagnosis']

In [7]:
disease_list = []
for disease in diseases:
    disease_list.append(disease['primary_diagnosis'])

#remove any duplicates
disease_list = list(set(disease_list))
print(disease_list)

['Histiocytic Malignancy', 'Low-grade glial-glioneuronal tumor', 'Desmoplastic/Nodular Medulloblastoma', 'Anaplastic Large Cell Lymphoma, ALK Positive', 'Residual CIC-Rearranged Sarcoma', 'Ganglioneuroblastoma', 'Nasopharyngeal Carcinoma Metastatic', 'Pituitary Tumor', 'Left Thoracic Tumor', 'Low-grade Glial Neoplasm', 'Spinal Tumor', 'Pineal Parenchymal Tumor', 'Large Cell Medulloblastoma', 'High-grade Central Nervous System Neoplasm Favor Embryonal', 'Residual Dermatofibrosarcoma Protuberans', 'High-grade serous carcinoma', 'Glioblastoma', 'Acute Megakaryoblastic Leukemia', 'Low-grade circumscribed glial/glioneuronal tumor', 'Well Differentiated Neuroendocrine Tumor', 'Residual High Grade Astrocytoma With Piloid Features', 'Hodgkin Lymphoma, NOS', 'Anaplastic Ependymoma', 'Supratentorial Choroid Plexus Papilloma', 'Malignant Rhabdoid Tumor', 'Neuroepithelial neoplasm, Glioma', 'Glial Tumor', 'Cellular Ependymoma', 'Malignant embryonal tumor, favor medulloblastoma', 'Ependymoma NOS', 

That's a fair number of diseases.  If we're interested in something like Glioma, that could be represented in several different names, so let's see if this can be reduced

In [8]:
glioma_list = [s for s in disease_list if "Glioma" in s]
print(glioma_list)

['Neuroepithelial neoplasm, Glioma', 'Infant Hemispheric Glioma', 'Low-grade Glioma', 'Glioma', 'Diffuse Glioma', 'Low Cellularity Glioma', 'Low Grade Glioma', 'Infiltrating Glioma', 'Angiocentric Glioma', 'Optic Pathway Glioma', 'Astrocytic Glioma']


Finding out what studies these are associated with could help understand what kinds of data are avaiable.  From the CDS data model, the way to find out what studies are associated with a diagnosis is to go through the Participant node.  Hence, the query includes the participant.
We'll also get the phs_accession number as that is a less clumsy way to query the database than the name.

In [9]:
diagnosisStudyQuery = """
query diagnosisStudies($diagnosis: String!){
  diagnosis(primary_diagnosis:$diagnosis){
    participant{
      study{
        study_name
        phs_accession
      }
    }
  }
}
"""

In [ ]:
columns = ["Diagnosis", "Studies","PHS Accession"]
study_df = pd.DataFrame(columns = columns)
for glioma in glioma_list:
    studies= {}
    variables = {"diagnosis": glioma}
    results = runGraphQLQuery(cds_graphql_url, diagnosisStudyQuery, variables)
    results = results['data']['diagnosis']
    #Putting everything in a dictionary removes duplicates
    for result in results:
        studies[result['participant']['study']['study_name']] = result['participant']['study']['phs_accession']
    for studyname, phs in studies.items():
        study_df.loc[len(study_df.index)] = [glioma, studyname, phs]
        
    

In [11]:
display(Markdown(study_df.to_markdown()))

|    | Diagnosis                        | Studies                                                                                                                                       | PHS Accession   |
|---:|:---------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------|:----------------|
|  0 | Low Grade Glioma                 | Clinical Trial Specimen Molecular Characterization (CTSMC)                                                                                    | phs002790       |
|  1 | Neuroepithelial neoplasm, Glioma | Clinical Trial Specimen Molecular Characterization (CTSMC)                                                                                    | phs002790       |
|  2 | Low-grade Glioma                 | Clinical Trial Specimen Molecular Characterization (CTSMC)                                                                                    | phs002790       |
|  3 | Optic Pathway Glioma             | Clinical Trial Specimen Molecular Characterization (CTSMC)                                                                                    | phs002790       |
|  4 | Angiocentric Glioma              | Clinical Trial Specimen Molecular Characterization (CTSMC)                                                                                    | phs002790       |
|  5 | Infiltrating Glioma              | Clinical Trial Specimen Molecular Characterization (CTSMC)                                                                                    | phs002790       |
|  6 | Astrocytic Glioma                | Clinical Trial Specimen Molecular Characterization (CTSMC)                                                                                    | phs002790       |
|  7 | Glioma                           | Human Tumor Atlas Network (HTAN) primary sequencing data                                                                                      | phs002371       |
|  8 | Glioma                           | CIDR: Discovery, Biology, and Risk of Inherited Variants in Glioma sample                                                                     | phs002250       |
|  9 | Glioma                           | Clinical Trial Specimen Molecular Characterization (CTSMC)                                                                                    | phs002790       |
| 10 | Glioma                           | Childhood Cancer Data Initiative (CCDI): Integration of genomic and clinical data from unique rare cancer datasets to facilitate data sharing | phs002517       |
| 11 | Glioma                           | TCGA WGS Variants Across 18 Cancer Types                                                                                                      | phs003155       |
| 12 | Low Cellularity Glioma           | Clinical Trial Specimen Molecular Characterization (CTSMC)                                                                                    | phs002790       |
| 13 | Infant Hemispheric Glioma        | Clinical Trial Specimen Molecular Characterization (CTSMC)                                                                                    | phs002790       |
| 14 | Diffuse Glioma                   | Clinical Trial Specimen Molecular Characterization (CTSMC)                                                                                    | phs002790       |

From this results, it's very clear that the CTSMC study is the one to take a much closer look at.  We know from the introspection queries that there's a StudyDetail query that provides an overview, let's see what that looks like. As before, we'll use a variable so we can more easily reuse the query

In [12]:
study_detail_query = """
query getStudyDetails ($phs:String!){
  studyDetail(phs_accession:$phs){
    study_acronym
    study_description
    study_external_url
    study_name
    phs_accession
    numberOfDiseaseSites
    numberOfFiles
    numberOfSamples
    numberOfSubjects
    data_types
  }
}
"""

In [13]:
variables = {"phs": "phs002790"}
study_detail = runGraphQLQuery(cds_graphql_url, study_detail_query, variables)
study_detail = study_detail['data']['studyDetail']

In [14]:
columns = ['Disease Sites', 'Files', 'Samples', 'Subject', 'Data Types', 'Acronym', 'Description','External URL']
data = [study_detail['numberOfDiseaseSites'],study_detail['numberOfFiles'],study_detail['numberOfSamples'],study_detail['numberOfSubjects'],study_detail['data_types'],study_detail['study_acronym'],study_detail['study_description'],study_detail['study_external_url']]
detail_df = pd.DataFrame([data], columns=columns)


In [15]:
display(Markdown(detail_df.to_markdown()))

|    |   Disease Sites |   Files |   Samples |   Subject | Data Types   | Acronym   | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | External URL                                                                         |
|---:|----------------:|--------:|----------:|----------:|:-------------|:----------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------------------------------------------------------------------------------|
|  0 |             135 |   49747 |      5647 |      2446 | Genomic      | CTSMC     | The National Cancer Institute's (NCI) Childhood Cancer Data Initiative (CCDI) focuses on the critical need to collect, analyze, and share data to address the burden of cancer in children, adolescents, and young adults (AYAs). The Molecular Characterization Initiative (MCI) will further the CCDI's goals by providing access to better diagnostic tests for pediatric and AYA patients. The molecular characterizations of solid tumors, soft tissue sarcomas, and rare diseases are performed in a CLIA-certified setting as results may be used to screen for and/or confirm clinical trial eligibility, direct treatment, or otherwise contribute to the conduct of the trial. The following molecular characterization assays were performed: | https://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/study.cgi?study_id=phs002790.v5.p1 |

So there are plenty of subjects, samples, and files in CTSMC, this moves the question to how many of these subject have a glioma

In [16]:
participant_query = """
query studyDiagnosis($phs:String!, $disease:String!){
  study(phs_accession:$phs){
    study_acronym
    participants{
      participant_id
      diagnoses(primary_diagnosis:$disease){
        primary_diagnosis
        primary_site
        progression_or_recurrence
      }
    }
  }
}
"""

In [17]:
columns = ["Primary Diagnosis", "Participant ID", "Primary Site", "Progression or Recurrance"]
glioma_df = pd.DataFrame(columns = columns)
#glioma_list = ['Diffuse Glioma']
for glioma in glioma_list:
    print(glioma)
    variables = {"phs":"phs002790", "disease": glioma}
    result = runGraphQLQuery(cds_graphql_url, participant_query, variables)
    #print(result)
    plist = result['data']['study'][0]['participants']
    #print(plist)
    for participant in plist:
        #print(participant)
        #Not all participants have diagnosis information
        if len(participant['diagnoses']) >= 1:
            for diagnosis in participant['diagnoses']:
                glioma_df.loc[len(glioma_df.index)] = [diagnosis['primary_diagnosis'], participant['participant_id'], diagnosis['primary_site'], diagnosis['progression_or_recurrence']]
            
    

Low Grade Glioma
Neuroepithelial neoplasm, Glioma
Low-grade Glioma
Optic Pathway Glioma
Angiocentric Glioma
Infiltrating Glioma
Astrocytic Glioma
Glioma
Low Cellularity Glioma
Infant Hemispheric Glioma
Diffuse Glioma


In [18]:
display(Markdown(glioma_df.to_markdown()))

|     | Primary Diagnosis                | Participant ID   | Primary Site   | Progression or Recurrance   |
|----:|:---------------------------------|:-----------------|:---------------|:----------------------------|
|   0 | Low Grade Glioma                 | PBBXHW           |                |                             |
|   1 | Low Grade Glioma                 | PBBUYL           |                |                             |
|   2 | Low Grade Glioma                 | PBBTGI           |                |                             |
|   3 | Low Grade Glioma                 | PBBRUJ           |                |                             |
|   4 | Low Grade Glioma                 | PBBTUH           |                |                             |
|   5 | Low Grade Glioma                 | PBBJPZ           |                |                             |
|   6 | Low Grade Glioma                 | PBBNZF           |                |                             |
|   7 | Neuroepithelial neoplasm, Glioma | PBBSYH           |                |                             |
|   8 | Neuroepithelial neoplasm, Glioma | PBBMIY           |                |                             |
|   9 | Neuroepithelial neoplasm, Glioma | PBBISR           |                |                             |
|  10 | Low-grade Glioma                 | PBCHSW           |                |                             |
|  11 | Low-grade Glioma                 | PBCHMT           |                |                             |
|  12 | Low-grade Glioma                 | PBBWSY           |                |                             |
|  13 | Low-grade Glioma                 | PBBWXH           |                |                             |
|  14 | Low-grade Glioma                 | PBBLXG           |                |                             |
|  15 | Optic Pathway Glioma             | PBBJIL           |                |                             |
|  16 | Angiocentric Glioma              | PBBWHU           |                |                             |
|  17 | Angiocentric Glioma              | PBBKIL           |                |                             |
|  18 | Infiltrating Glioma              | PBCHYK           |                |                             |
|  19 | Infiltrating Glioma              | PBCCCU           |                |                             |
|  20 | Infiltrating Glioma              | PBBKNA           |                |                             |
|  21 | Astrocytic Glioma                | PBBWKJ           |                |                             |
|  22 | Astrocytic Glioma                | PBBKZT           |                |                             |
|  23 | Astrocytic Glioma                | PBBPLT           |                |                             |
|  24 | Glioma                           | PBCHTI           |                |                             |
|  25 | Glioma                           | PBCHEC           |                |                             |
|  26 | Glioma                           | PBCIRT           |                |                             |
|  27 | Glioma                           | PBCILI           |                |                             |
|  28 | Glioma                           | PBCDHR           |                |                             |
|  29 | Glioma                           | PBCESS           |                |                             |
|  30 | Glioma                           | PBCDTJ           |                |                             |
|  31 | Glioma                           | PBCDLM           |                |                             |
|  32 | Glioma                           | PBCECF           |                |                             |
|  33 | Glioma                           | PBCEFF           |                |                             |
|  34 | Glioma                           | PBCEHF           |                |                             |
|  35 | Glioma                           | PBCBYH           |                |                             |
|  36 | Glioma                           | PBCDNI           |                |                             |
|  37 | Glioma                           | PBCAET           |                |                             |
|  38 | Glioma                           | PBCCVE           |                |                             |
|  39 | Glioma                           | PBCCMF           |                |                             |
|  40 | Glioma                           | PBCCEK           |                |                             |
|  41 | Glioma                           | PBCDET           |                |                             |
|  42 | Glioma                           | PBCCJT           |                |                             |
|  43 | Glioma                           | PBCCKE           |                |                             |
|  44 | Glioma                           | PBCANM           |                |                             |
|  45 | Glioma                           | PBCCCJ           |                |                             |
|  46 | Glioma                           | PBCDLD           |                |                             |
|  47 | Glioma                           | PBCDGG           |                |                             |
|  48 | Glioma                           | PBCCYH           |                |                             |
|  49 | Glioma                           | PBCCXW           |                |                             |
|  50 | Glioma                           | PBCDAC           |                |                             |
|  51 | Glioma                           | PBCDFH           |                |                             |
|  52 | Glioma                           | PBCCFA           |                |                             |
|  53 | Glioma                           | PBCCXK           |                |                             |
|  54 | Glioma                           | PBCBHX           |                |                             |
|  55 | Glioma                           | PBCCIP           |                |                             |
|  56 | Glioma                           | PBBYKX           |                |                             |
|  57 | Glioma                           | PBCBFB           |                |                             |
|  58 | Glioma                           | PBCBDL           |                |                             |
|  59 | Glioma                           | PBCCJG           |                |                             |
|  60 | Glioma                           | PBCBYV           |                |                             |
|  61 | Glioma                           | PBCCLV           |                |                             |
|  62 | Glioma                           | PBCBBE           |                |                             |
|  63 | Glioma                           | PBCASF           |                |                             |
|  64 | Glioma                           | PBCAKG           |                |                             |
|  65 | Glioma                           | PBCAWA           |                |                             |
|  66 | Glioma                           | PBCAFX           |                |                             |
|  67 | Glioma                           | PBBZGH           |                |                             |
|  68 | Glioma                           | PBBZKS           |                |                             |
|  69 | Glioma                           | PBBWUR           |                |                             |
|  70 | Glioma                           | PBBXRV           |                |                             |
|  71 | Glioma                           | PBBYAL           |                |                             |
|  72 | Glioma                           | PBBWDB           |                |                             |
|  73 | Glioma                           | PBBVKH           |                |                             |
|  74 | Glioma                           | PBBXAT           |                |                             |
|  75 | Glioma                           | PBBXGC           |                |                             |
|  76 | Glioma                           | PBBVXS           |                |                             |
|  77 | Glioma                           | PBBWAC           |                |                             |
|  78 | Glioma                           | PBBWLY           |                |                             |
|  79 | Glioma                           | PBBWWF           |                |                             |
|  80 | Glioma                           | PBBWVB           |                |                             |
|  81 | Glioma                           | PBBVRV           |                |                             |
|  82 | Glioma                           | PBBVUP           |                |                             |
|  83 | Glioma                           | PBBVKF           |                |                             |
|  84 | Glioma                           | PBBUYK           |                |                             |
|  85 | Glioma                           | PBBSKD           |                |                             |
|  86 | Glioma                           | PBBSRT           |                |                             |
|  87 | Glioma                           | PBBUKM           |                |                             |
|  88 | Glioma                           | PBBTVK           |                |                             |
|  89 | Glioma                           | PBBTLT           |                |                             |
|  90 | Glioma                           | PBBTRI           |                |                             |
|  91 | Glioma                           | PBBTAY           |                |                             |
|  92 | Glioma                           | PBBSRD           |                |                             |
|  93 | Glioma                           | PBBTSW           |                |                             |
|  94 | Glioma                           | PBBTSD           |                |                             |
|  95 | Glioma                           | PBBSZR           |                |                             |
|  96 | Glioma                           | PBBTTG           |                |                             |
|  97 | Glioma                           | PBBSXH           |                |                             |
|  98 | Glioma                           | PBBNSW           |                |                             |
|  99 | Glioma                           | PBBPGJ           |                |                             |
| 100 | Glioma                           | PBBPAP           |                |                             |
| 101 | Glioma                           | PBBPAA           |                |                             |
| 102 | Glioma                           | PBBNTY           |                |                             |
| 103 | Glioma                           | PBBKPA           |                |                             |
| 104 | Glioma                           | PBBJVU           |                |                             |
| 105 | Glioma                           | PBBJJT           |                |                             |
| 106 | Glioma                           | PBBMPA           |                |                             |
| 107 | Glioma                           | PBBJSS           |                |                             |
| 108 | Glioma                           | PBBLMC           |                |                             |
| 109 | Glioma                           | PBBILM           |                |                             |
| 110 | Glioma                           | PBBHPP           |                |                             |
| 111 | Glioma                           | PBBHIL           |                |                             |
| 112 | Glioma                           | PBBJVN           |                |                             |
| 113 | Glioma                           | PBBKSG           |                |                             |
| 114 | Glioma                           | PBBHYH           |                |                             |
| 115 | Glioma                           | PBBPWG           |                |                             |
| 116 | Glioma                           | PBBRKP           |                |                             |
| 117 | Glioma                           | PBBPIX           |                |                             |
| 118 | Glioma                           | PBBPTA           |                |                             |
| 119 | Glioma                           | PBBPNU           |                |                             |
| 120 | Glioma                           | PBBRKJ           |                |                             |
| 121 | Glioma                           | PBBMCC           |                |                             |
| 122 | Glioma                           | PBBPJK           |                |                             |
| 123 | Glioma                           | PBBNZV           |                |                             |
| 124 | Glioma                           | PBBPCE           |                |                             |
| 125 | Glioma                           | PBBSJN           |                |                             |
| 126 | Glioma                           | PBBPUG           |                |                             |
| 127 | Glioma                           | PBBMZW           |                |                             |
| 128 | Glioma                           | PBBLYY           |                |                             |
| 129 | Glioma                           | PBBNXZ           |                |                             |
| 130 | Glioma                           | PBBPWH           |                |                             |
| 131 | Low Cellularity Glioma           | PBCHNH           |                |                             |
| 132 | Low Cellularity Glioma           | PBBSDG           |                |                             |
| 133 | Infant Hemispheric Glioma        | PBBMCU           |                |                             |
| 134 | Diffuse Glioma                   | PBCEET           |                |                             |
| 135 | Diffuse Glioma                   | PBCEGU           |                |                             |
| 136 | Diffuse Glioma                   | PBCDKY           |                |                             |
| 137 | Diffuse Glioma                   | PBCEIH           |                |                             |
| 138 | Diffuse Glioma                   | PBCDDX           |                |                             |
| 139 | Diffuse Glioma                   | PBCCIK           |                |                             |
| 140 | Diffuse Glioma                   | PBCCJV           |                |                             |
| 141 | Diffuse Glioma                   | PBBZRU           |                |                             |
| 142 | Diffuse Glioma                   | PBCBVY           |                |                             |
| 143 | Diffuse Glioma                   | PBCBPM           |                |                             |
| 144 | Diffuse Glioma                   | PBCCBA           |                |                             |
| 145 | Diffuse Glioma                   | PBBZSX           |                |                             |
| 146 | Diffuse Glioma                   | PBCALC           |                |                             |
| 147 | Diffuse Glioma                   | PBBZXG           |                |                             |
| 148 | Diffuse Glioma                   | PBBVVI           |                |                             |
| 149 | Diffuse Glioma                   | PBBYCD           |                |                             |
| 150 | Diffuse Glioma                   | PBBYNW           |                |                             |
| 151 | Diffuse Glioma                   | PBBXSM           |                |                             |
| 152 | Diffuse Glioma                   | PBBWGV           |                |                             |
| 153 | Diffuse Glioma                   | PBBWUS           |                |                             |
| 154 | Diffuse Glioma                   | PBBWMI           |                |                             |
| 155 | Diffuse Glioma                   | PBBWFV           |                |                             |
| 156 | Diffuse Glioma                   | PBBWFG           |                |                             |
| 157 | Diffuse Glioma                   | PBBVDV           |                |                             |
| 158 | Diffuse Glioma                   | PBBWNC           |                |                             |
| 159 | Diffuse Glioma                   | PBBTUX           |                |                             |
| 160 | Diffuse Glioma                   | PBBVPC           |                |                             |
| 161 | Diffuse Glioma                   | PBBTLR           |                |                             |
| 162 | Diffuse Glioma                   | PBBUCX           |                |                             |
| 163 | Diffuse Glioma                   | PBBTXS           |                |                             |
| 164 | Diffuse Glioma                   | PBBRET           |                |                             |
| 165 | Diffuse Glioma                   | PBBTWG           |                |                             |
| 166 | Diffuse Glioma                   | PBBSKE           |                |                             |
| 167 | Diffuse Glioma                   | PBBTTH           |                |                             |
| 168 | Diffuse Glioma                   | PBBSSM           |                |                             |
| 169 | Diffuse Glioma                   | PBBKSS           |                |                             |
| 170 | Diffuse Glioma                   | PBBJLB           |                |                             |
| 171 | Diffuse Glioma                   | PBBLFW           |                |                             |
| 172 | Diffuse Glioma                   | PBBNKF           |                |                             |
| 173 | Diffuse Glioma                   | PBBLWR           |                |                             |
| 174 | Diffuse Glioma                   | PBBKAW           |                |                             |
| 175 | Diffuse Glioma                   | PBBIBT           |                |                             |
| 176 | Diffuse Glioma                   | PBBKBT           |                |                             |
| 177 | Diffuse Glioma                   | PBBMLH           |                |                             |
| 178 | Diffuse Glioma                   | PBBIUN           |                |                             |
| 179 | Diffuse Glioma                   | PBBSGU           |                |                             |
| 180 | Diffuse Glioma                   | PBBNZL           |                |                             |
| 181 | Diffuse Glioma                   | PBBSGI           |                |                             |
| 182 | Diffuse Glioma                   | PBBMZU           |                |                             |

So we've got 182 participants, unfortunately without any primary site or recurrance information.  Still, let's see what samples are associated with these participants and since files are assocaited with samples, we'll also get some basic file info to see what is accessible.  Just to see what's available, let's run a single participant

In [19]:
sample_query = """
query participantFiles($pid:String!){
  participant(participant_id:$pid){
    samples{
      sample_id
      sample_anatomic_site
      sample_tumor_status
      sample_type
      files{
        file_name
        file_type
      }
    }
 
  }
  }
"""

In [20]:
variables = {"pid" : "PBBWKJ"}

In [21]:
singlepatient = runGraphQLQuery(cds_graphql_url, sample_query, variables)

KeyboardInterrupt: 

In [66]:
print(json.dumps(singlepatient, indent=2))

{
  "data": {
    "participant": [
      {
        "samples": [
          {
            "sample_id": "0DJ4CA_DNA",
            "sample_anatomic_site": "Blood",
            "sample_tumor_status": "Normal",
            "sample_type": "DNA",
            "files": [
              {
                "file_name": "PBBWKJ_tumor_normal.clinical_report.json",
                "file_type": "JSON"
              },
              {
                "file_name": "IGM_PBBWKJ-0DJ4CA_20230311_normal.aln.cram.crai",
                "file_type": "CRAI"
              },
              {
                "file_name": "IGM_PBBWKJ-0DJ4CA_20230311_cnv_germline.json",
                "file_type": "JSON"
              },
              {
                "file_name": "IGM_PBBWKJ-0DJ4C0_20230311_cnv_panel_of_normals.json",
                "file_type": "JSON"
              },
              {
                "file_name": "IGM_PBBWKJ-0DJ4CA_20230311_normal.aln.cram",
                "file_type": "CRAM"
              },
   

If this is a representative participant, we've got RNA sequencing and DNA sequencing but we've also got a lot of different kinds of files as well.  Rather than filter the fily type in the notebook, GraphQL has a filter function that can be used to filter on the server side.  We'll also pull some additiona information that will be useful for getting access to the files for downstream analysis.

In [75]:
revised_sample_query = """
query participantFiles($pid:String!){
  participant(participant_id:$pid){
    samples{
      sample_id
      sample_anatomic_site
      sample_tumor_status
      sample_type
      files(filter:{OR:[{file_type:"BAM"},{file_type:"BAI"},{file_type:"CRAM"}, {file_type:"CRAI"}]}){
        file_id
        file_name
        md5sum
        file_type
      }
    }
 
  }
  }
"""

In [76]:
columns = ["Participant ID", "Sample ID", "Sample Anatomic Site", "Tumor Status", "Sample Type", "File ID", "File Name", "MD5SUM", "File Type"]
genomic_df = pd.DataFrame(columns=columns)

In [77]:
participant_list = glioma_df['Participant ID']

In [ ]:
#participant_list = ["PBBMCU"]
for participant in participant_list:
    variables = {"pid": participant}
    participant_results = runGraphQLQuery(cds_graphql_url, revised_sample_query, variables)
    sample_list = participant_results['data']['participant'][0]['samples']
    for sample in sample_list:
        file_list = sample['files']
        for file in file_list:
            genomic_df.loc[len(genomic_df.index)] = [participant, sample['sample_id'], sample['sample_anatomic_site'], sample['sample_tumor_status'], sample['sample_type'], "drs://nci-crdc.datacommons.io/"+file['file_id'], file['file_name'], file['md5sum'], file['file_type']]
    

In [79]:
display(Markdown(genomic_df.to_markdown()))

|      | Participant ID   | Sample ID   | Sample Anatomic Site   | Tumor Status   | Sample Type   | File ID                                        | File Name                                        | MD5SUM                           | File Type   |
|-----:|:-----------------|:------------|:-----------------------|:---------------|:--------------|:-----------------------------------------------|:-------------------------------------------------|:---------------------------------|:------------|
|    0 | PBCEET           | 0DPU2Y_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/5b6fb906-ca01-4d16-80c7-5ee7036a4091   | IGM_PBCEET-0DPU2Y_20230908_tumor.aln.cram.crai   | 24fd7b10529fb17abc6511f7a06aab31 | CRAI        |
|    1 | PBCEET           | 0DPU2Y_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/8f3ff3c5-c982-44dd-a9b7-a2f799315361   | IGM_PBCEET-0DPU2Y_20230908_tumor.aln.cram        | 72a711ba9708769d4accfa1feaa8b235 | CRAM        |
|    2 | PBCEET           | 0DPU2P_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/9290b0d0-95e4-4ee4-9443-377ebd64f09d   | IGM_PBCEET-0DPU2P_20230906_merged_bam.bai        | 89e62149f208e54c34997524c138ef89 | BAI         |
|    3 | PBCEET           | 0DPU2P_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/329df1db-a36d-4e93-a448-b668f0ae5c72   | IGM_PBCEET-0DPU2P_20230906_merged_bam.bam        | fa70d90daee09bee463a50d8eb01dfa8 | BAM         |
|    4 | PBCEET           | 0DPU2P_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/28168dba-4108-40ac-8586-743ee6f74916   | IGM_PBCEET-0DPU2P_20230906_bam.bam               | f4a244aa13e28148feda3bf27c012060 | BAM         |
|    5 | PBCEET           | 0DPU2P_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31a2459a-fd3a-4e2a-94ec-c39a0317a94e   | IGM_PBCEET-0DPU2P_20230906_bam.bai               | 598631bca417bf7c9e3da9806e95bea5 | BAI         |
|    6 | PBCEET           | 0DPU2M_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/54107458-d93f-40c0-bb8e-0be92b28fb7d   | IGM_PBCEET-0DPU2M_20230908_normal.aln.cram.crai  | b8b0dce34141aa45d0de05e1b65ac98d | CRAI        |
|    7 | PBCEET           | 0DPU2M_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/d33a3511-d5d9-41bd-8384-87472250c833   | IGM_PBCEET-0DPU2M_20230908_normal.aln.cram       | af9f63068ed9a3b9bfac910153172520 | CRAM        |
|    8 | PBCEGU           | 0DQ18M_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/d2d0e77d-27df-48c8-9f38-98d21354d2c3   | IGM_PBCEGU-0DQ18M_20230913_normal.aln.cram.crai  | 20b71fc99270e71b82b9420971e5b365 | CRAI        |
|    9 | PBCEGU           | 0DQ18M_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b9864b21-6bee-4b2d-86d5-13cd71ab4c44   | IGM_PBCEGU-0DQ18M_20230913_normal.aln.cram       | 36b0a46c24afd6aca9516751abdc492f | CRAM        |
|   10 | PBCEGU           | 0DQ18G_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/02dbd10f-91f2-4464-84d1-d0ede87526c9   | IGM_PBCEGU-0DQ18G_20230918_merged_bam.bam        | 73fd6cfbc8c1f97bcfa3054b6a8fb631 | BAM         |
|   11 | PBCEGU           | 0DQ18G_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8cff1f41-67fa-4835-b816-8a94266e5ea9   | IGM_PBCEGU-0DQ18G_20230918_merged_bam.bai        | 44109d8c724651ae491aef898777b008 | BAI         |
|   12 | PBCEGU           | 0DQ18G_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b4aaf05b-5a62-43e4-8567-c0617f1f68ca   | IGM_PBCEGU-0DQ18G_20230918_bam.bai               | 57aa25ced43e9885e688e3c721abd561 | BAI         |
|   13 | PBCEGU           | 0DQ18G_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/923668a0-21df-46b3-ae31-9461a0767efa   | IGM_PBCEGU-0DQ18G_20230918_bam.bam               | 352830bc11ccf00844c1089ce543a3d6 | BAM         |
|   14 | PBCEGU           | 0DQ18H_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/f0144121-3057-49f8-9f2b-cf2a2baaf54c   | IGM_PBCEGU-0DQ18H_20230913_tumor.aln.cram.crai   | c4e7f11afa14f4c6f7d6b8e76aa4ee84 | CRAI        |
|   15 | PBCEGU           | 0DQ18H_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/fc0b96af-4a6c-4cbc-a588-81d1a4a8c463   | IGM_PBCEGU-0DQ18H_20230913_tumor.aln.cram        | d960bb5a3c4df06a12ce729ae37e337e | CRAM        |
|   16 | PBCDKY           | 0DPH9Z_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/dad9a54b-146d-4cfb-bdee-4348b8df7620   | IGM_PBCDKY-0DPH9Z_20230828_normal.aln.cram.crai  | 84a241cd23187d71d1dac7d309b419ae | CRAI        |
|   17 | PBCDKY           | 0DPH9Z_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/4a220890-518b-4427-9d81-04491041adf4   | IGM_PBCDKY-0DPH9Z_20230828_normal.aln.cram       | 2f2e7b50300285e7f8515b6f34fe830f | CRAM        |
|   18 | PBCDKY           | 0DPHA7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/318e25f9-46d5-4e7a-84ca-b5abc02508a4   | IGM_PBCDKY-0DPHA7_20230828_bam.bai               | 279000dfa55ae4aec10612c64daf671b | BAI         |
|   19 | PBCDKY           | 0DPHA7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7b33c075-48f7-409a-9159-26b5517be772   | IGM_PBCDKY-0DPHA7_20230828_merged_bam.bam        | 375076bc4219b285b82282e73d9fc367 | BAM         |
|   20 | PBCDKY           | 0DPHA7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a4c9c977-cc74-46c1-9fb6-cd289fffcf4c   | IGM_PBCDKY-0DPHA7_20230828_bam.bam               | 0b2beaf21dc80b7add945b94146b9969 | BAM         |
|   21 | PBCDKY           | 0DPHA7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/43cd31e1-26b4-419d-b838-dd579bc1185d   | IGM_PBCDKY-0DPHA7_20230828_merged_bam.bai        | 5a88e0ee8a1078441c6eaca9b6ddba1f | BAI         |
|   22 | PBCDKY           | 0DPHAN_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/323d1656-c797-4083-9cc6-c0a2c3d8f4c2   | IGM_PBCDKY-0DPHAN_20230828_tumor.aln.cram        | 1f586463f7b7ab9bf6e0c22cf9dad47d | CRAM        |
|   23 | PBCDKY           | 0DPHAN_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/d7f6a289-46fb-4e19-9231-d44dff9dfc2c   | IGM_PBCDKY-0DPHAN_20230828_tumor.aln.cram.crai   | 7d0270b3af99cdcd375cedd23123c276 | CRAI        |
|   24 | PBCEIH           | 0DQ80X_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/9cb69e07-800e-48af-86b7-5c86b7e2859f   | IGM_PBCEIH-0DQ80X_20230922_tumor.aln.cram        | 0374827c654281dc3a7bc8a8701fb511 | CRAM        |
|   25 | PBCEIH           | 0DQ80X_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/db5d7c50-1f91-49fc-94d1-cfcab58a4f13   | IGM_PBCEIH-0DQ80X_20230922_tumor.aln.cram.crai   | 249e72f3ceb956f1a888bfb91a5f25c2 | CRAI        |
|   26 | PBCEIH           | 0DQ7W9_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/89dbb600-a9a6-4ad8-afe1-d4f30b1cf8a5   | IGM_PBCEIH-0DQ7W9_20230922_normal.aln.cram.crai  | e1881ad17058e5d27745711c11593c51 | CRAI        |
|   27 | PBCEIH           | 0DQ7W9_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/3d2e3f75-2fc4-4e7e-9728-0949c0698951   | IGM_PBCEIH-0DQ7W9_20230922_normal.aln.cram       | 7815c118cdac36d1f0499c9e387e8b66 | CRAM        |
|   28 | PBCEIH           | 0DQ80N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ee8815db-9b67-46a5-8f1f-b39f3c96298c   | IGM_PBCEIH-0DQ80N_20230919_bam.bai               | 4724eba146716fd2db13ce2a54c165d4 | BAI         |
|   29 | PBCEIH           | 0DQ80N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/945ca2bd-2a7c-4501-830a-6b4926bea734   | IGM_PBCEIH-0DQ80N_20230919_bam.bam               | 33f56d31cbf8cae15ff2c246ce4b83e6 | BAM         |
|   30 | PBCEIH           | 0DQ80N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8775a0b3-aec3-42e1-931f-658980212755   | IGM_PBCEIH-0DQ80N_20230919_merged_bam.bai        | 28258a585984f0053e08dc462fe4b157 | BAI         |
|   31 | PBCEIH           | 0DQ80N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/04064794-ff58-4b7b-b35e-927aef4a4abc   | IGM_PBCEIH-0DQ80N_20230919_merged_bam.bam        | 1f9cd6a59fe48649cf26bc7a9468b29c | BAM         |
|   32 | PBCDDX           | 0DPFM9_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/526673b4-398a-4757-bc61-360383c62372   | IGM_PBCDDX-0DPFM9_20230826_normal.aln.cram       | 886201e09bc818352ca95f2f2383aff3 | CRAM        |
|   33 | PBCDDX           | 0DPFM9_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/299f2854-dcf6-45c2-b632-981b8a9d8d8a   | IGM_PBCDDX-0DPFM9_20230826_normal.aln.cram.crai  | f0083fc33b3d7b0eb70e0ea441e2d822 | CRAI        |
|   34 | PBCDDX           | 0DPFMJ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/38c130a7-0ecc-4c79-a9e0-5a0ca2784596   | IGM_PBCDDX-0DPFMJ_20230826_tumor.aln.cram.crai   | e2830a014baaeebfc97d186a21712227 | CRAI        |
|   35 | PBCDDX           | 0DPFMJ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/d2a0effa-cde5-4dd9-bd79-24aec82e27d4   | IGM_PBCDDX-0DPFMJ_20230826_tumor.aln.cram        | 33c6d9032a079a465c233ae3dc905bcf | CRAM        |
|   36 | PBCDDX           | 0DPFMF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f95f64aa-4fe0-42e4-975c-ead29794f645   | IGM_PBCDDX-0DPFMF_20230826_bam.bai               | 22c917bf7231c521fd7b0b27a5cc101c | BAI         |
|   37 | PBCDDX           | 0DPFMF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/5bfedaf0-a9f8-4c0a-a54a-5873af516781   | IGM_PBCDDX-0DPFMF_20230826_bam.bam               | 3fdf85311a3107c7a033192fd430cbce | BAM         |
|   38 | PBCDDX           | 0DPFMF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/80f9d480-fd01-4fe9-8e68-a8cc386e6a30   | IGM_PBCDDX-0DPFMF_20230826_merged_bam.bai        | 5bd3e27e4f361e55a75978ab15bfa1f7 | BAI         |
|   39 | PBCDDX           | 0DPFMF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d90cbd98-3de6-4748-9b86-5a5acef925fe   | IGM_PBCDDX-0DPFMF_20230826_merged_bam.bam        | 1a90dc759455d65e355525e4d724fcd3 | BAM         |
|   40 | PBCCIK           | 0DOJW1_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/28fc425f-6863-4fe8-86c6-9e6b452a7a6f   | IGM_PBCCIK-0DOJW1_20230801_tumor.aln.cram.crai   | d6b6221a28e0c1633661399ce01c2932 | CRAI        |
|   41 | PBCCIK           | 0DOJW1_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/94006bc0-e87e-49ad-b217-b007aaf1724b   | IGM_PBCCIK-0DOJW1_20230801_tumor.aln.cram        | 5ce2113ff80a9036e4e9066b6bae9336 | CRAM        |
|   42 | PBCCIK           | 0DOJY5_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/6768102c-09bc-4376-8194-f93afab9c635   | IGM_PBCCIK-0DOJY5_20230801_merged_bam.bai        | 77c05e700f419edf38e901c34cfdca47 | BAI         |
|   43 | PBCCIK           | 0DOJY5_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a07e61aa-f1b6-48a1-a05b-a79ef4df7ac3   | IGM_PBCCIK-0DOJY5_20230801_merged_bam.bam        | aa4f1914fe95ef18b2f4989f75a2ca89 | BAM         |
|   44 | PBCCIK           | 0DOJY5_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0df5ef3a-0a70-4c5b-b6b8-05c15dcae07a   | IGM_PBCCIK-0DOJY5_20230801_bam.bam               | 84f4976aaca2a2f58e9cd0496894c102 | BAM         |
|   45 | PBCCIK           | 0DOJY5_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d654e273-3978-4f71-8876-5e193fc6aff2   | IGM_PBCCIK-0DOJY5_20230801_bam.bai               | 64ae023e8cc7972b109ab007fe56f917 | BAI         |
|   46 | PBCCIK           | 0DOJUU_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/924760bc-91fe-46a6-9d43-114369f7177b   | IGM_PBCCIK-0DOJUU_20230801_normal.aln.cram.crai  | 746645b08568a2f474d0f001315444b7 | CRAI        |
|   47 | PBCCIK           | 0DOJUU_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/73ca0cbf-f63b-420a-b6d3-ce7367b0bb3a   | IGM_PBCCIK-0DOJUU_20230801_normal.aln.cram       | 35ec17a953787f39e4613b456fe1d8f1 | CRAM        |
|   48 | PBCCJV           | 0DOA3V_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/da1f9eee-2b45-44cf-8e78-ade6ea95b36f   | IGM_PBCCJV-0DOA3V_20230726_tumor.aln.cram.crai   | fb7276038b70930cbf4c566b3b9859d8 | CRAI        |
|   49 | PBCCJV           | 0DOA3V_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/9dff101d-0fbd-44df-a732-77599691d40b   | IGM_PBCCJV-0DOA3V_20230726_tumor.aln.cram        | e28412a6400dc9d85cb44300ab33f4e0 | CRAM        |
|   50 | PBCCJV           | 0DOA29_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/8be03d42-f200-4533-92a2-aa1fa7ce13cb   | IGM_PBCCJV-0DOA29_20230726_normal.aln.cram       | 50a89cbcfec0ca29deec68ddd7dfa123 | CRAM        |
|   51 | PBCCJV           | 0DOA29_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/ac9e7264-f59b-43f1-bb89-0920edbf78fa   | IGM_PBCCJV-0DOA29_20230726_normal.aln.cram.crai  | baef7d1803902b8bffa58fbbcc32be4e | CRAI        |
|   52 | PBBZRU           | 0DNJA1_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/e8e80e90-7130-4fab-8294-ee96a4e409c7   | IGM_PBBZRU-0DNJA1_20230710_normal.aln.cram       | 9ea918a460756bfa15c424383a2fe9dd | CRAM        |
|   53 | PBBZRU           | 0DNJA1_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f31e1c98-eb10-4427-9642-d0aa34b028c1   | IGM_PBBZRU-0DNJA1_20230710_normal.aln.cram.crai  | 65c91e764fdef1a002b0bda2c902f664 | CRAI        |
|   54 | PBBZRU           | 0DNJB7_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/72049367-1df8-49a6-89ed-afbd11ca9d70   | IGM_PBBZRU-0DNJB7_20230710_tumor.aln.cram        | eea5c4d978538d2f305371a570983afb | CRAM        |
|   55 | PBBZRU           | 0DNJB7_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/98412bf2-ee99-42ba-a7d1-424077dbfa90   | IGM_PBBZRU-0DNJB7_20230710_tumor.aln.cram.crai   | bab8276a1a9f7481748ec8a083c5a568 | CRAI        |
|   56 | PBCBVY           | 0DNJBO_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/fc5def7e-cc11-4b16-8057-564e9cffe5c7   | IGM_PBCBVY-0DNJBO_20230707_tumor.aln.cram.crai   | b91803ba1e920798fd7fab09eefd1d6b | CRAI        |
|   57 | PBCBVY           | 0DNJBO_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/378adcad-9041-43dd-9f91-c129fd87aa8b   | IGM_PBCBVY-0DNJBO_20230707_tumor.aln.cram        | 6fd5c9006301d903ef839485c5e68e39 | CRAM        |
|   58 | PBCBVY           | 0DNJA9_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/45816e16-6432-40b0-bd6f-f3c42fec61f6   | IGM_PBCBVY-0DNJA9_20230707_normal.aln.cram.crai  | 01818bde4e210decaa647defaa8e43d3 | CRAI        |
|   59 | PBCBVY           | 0DNJA9_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/651d4fdc-030b-4302-96be-82f1c486a765   | IGM_PBCBVY-0DNJA9_20230707_normal.aln.cram       | 08c0d6d9a7b81a6cc7eb0c5cc11940ed | CRAM        |
|   60 | PBCBVY           | 0DNJAF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d52d580d-fb87-4286-9cf4-2bb6a5d2b81b   | IGM_PBCBVY-0DNJAF_20230712_merged_bam.bam        | 4a4e129bce229eb21372f111100573de | BAM         |
|   61 | PBCBVY           | 0DNJAF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/eabb4eef-8942-4ff1-8240-c628457e7704   | IGM_PBCBVY-0DNJAF_20230712_merged_bam.bai        | c91f1cade86239d0119a69742bed5e51 | BAI         |
|   62 | PBCBVY           | 0DNJAF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f3e919d6-2348-466e-809b-e071ae67cf99   | IGM_PBCBVY-0DNJAF_20230712_bam.bai               | 124809add6b44660010d4e0c415e907a | BAI         |
|   63 | PBCBVY           | 0DNJAF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c5245e52-3f24-4802-8750-f8712d440348   | IGM_PBCBVY-0DNJAF_20230712_bam.bam               | 4c5af18e55269a0255be3a6bee4794b3 | BAM         |
|   64 | PBCBPM           | 0DNGGQ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/afc172ab-e717-4603-b405-4973f750ce30   | IGM_PBCBPM-0DNGGQ_20230706_tumor.aln.cram.crai   | 5f34483509891ddb49e338c1fdec514a | CRAI        |
|   65 | PBCBPM           | 0DNGGQ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/d27baeac-796a-4fa0-addd-885ab0d1296e   | IGM_PBCBPM-0DNGGQ_20230706_tumor.aln.cram        | 08e8abd784a64986d2fa8b23c346869f | CRAM        |
|   66 | PBCBPM           | 0DNGG6_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/524ffabb-514e-43aa-adf7-246175c5a00f   | IGM_PBCBPM-0DNGG6_20230706_normal.aln.cram.crai  | 9edb41c37413dc94f40c6f1682b78cd8 | CRAI        |
|   67 | PBCBPM           | 0DNGG6_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/e52366ee-7681-4809-91db-a77a27dcbfc9   | IGM_PBCBPM-0DNGG6_20230706_normal.aln.cram       | c1a1dc850cc2e5cf26fa008b7171321d | CRAM        |
|   68 | PBCBPM           | 0DNGGU_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/efc13c80-40df-4dd3-9427-68b7390f0099   | IGM_PBCBPM-0DNGGU_20230705_bam.bam               | f79778e0b602c8aa4539e8d9e73d2817 | BAM         |
|   69 | PBCBPM           | 0DNGGU_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/049b2299-1ddc-40e3-a37c-3dfd4ade982c   | IGM_PBCBPM-0DNGGU_20230705_bam.bai               | bf77c2923d2c461e1726d87b835a3163 | BAI         |
|   70 | PBCBPM           | 0DNGGU_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4bcadb4c-4f5c-4b2c-8c11-dd64d2c9e75e   | IGM_PBCBPM-0DNGGU_20230705_merged_bam.bam        | 831f8da7e20f126091196a4942cc2da9 | BAM         |
|   71 | PBCBPM           | 0DNGGU_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/26cd218e-5aef-4602-ae3e-d5bcaa30d9d5   | IGM_PBCBPM-0DNGGU_20230705_merged_bam.bai        | 3dcdd7052dd9d0a6fcf60e45b70af0ab | BAI         |
|   72 | PBCCBA           | 0DOG2V_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/65a67819-04ef-405e-ad98-e1a83d0dd528   | IGM_PBCCBA-0DOG2V_20230731_normal.aln.cram.crai  | 0b120bc6f3256ecbbe3d792fee0b3ddc | CRAI        |
|   73 | PBCCBA           | 0DOG2V_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/99c66cc2-5d88-419e-ab52-4084d8eb4fe2   | IGM_PBCCBA-0DOG2V_20230731_normal.aln.cram       | 03370ba797b4410654d04f8c6e1c7b8c | CRAM        |
|   74 | PBCCBA           | 0DOG3F_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/07c948c4-1c4b-46c8-9846-cf34b46f7cd7   | IGM_PBCCBA-0DOG3F_20230731_tumor.aln.cram        | d3d258c8c9fb7dc183205efb0d599add | CRAM        |
|   75 | PBCCBA           | 0DOG3F_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/29879d06-6b72-4e2d-b32e-778233a295da   | IGM_PBCCBA-0DOG3F_20230731_tumor.aln.cram.crai   | 27a6d31b43dd2a8584b33035277578cf | CRAI        |
|   76 | PBCCBA           | 0DOG3O_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0c981e3e-2ffa-4530-8b83-2945184f5d62   | IGM_PBCCBA-0DOG3O_20230731_merged_bam.bai        | f9e88f72042e6be2ccf05de253389458 | BAI         |
|   77 | PBCCBA           | 0DOG3O_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/cb7f5555-7486-444a-a965-dad9ee1527a2   | IGM_PBCCBA-0DOG3O_20230731_merged_bam.bam        | 16d7204888344bd93ffac6e9bbb25d82 | BAM         |
|   78 | PBCCBA           | 0DOG3O_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/eb45e95e-4a65-445e-a240-d780e0b4708b   | IGM_PBCCBA-0DOG3O_20230731_bam.bam               | c2fb869d95aa393d2dcb30b15f8b2b91 | BAM         |
|   79 | PBCCBA           | 0DOG3O_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c7fd4397-1510-4a83-a40f-3c7d3f48a6f2   | IGM_PBCCBA-0DOG3O_20230731_bam.bai               | da620607cc33f0ff92e31c3a376fe244 | BAI         |
|   80 | PBBZSX           | 0DNCAB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/094fc9fc-14e7-40c1-9e66-4124292e1d46   | IGM_PBBZSX-0DNCAB_20230703_bam.bam               | bde12e042f64881c7e5ace045a797993 | BAM         |
|   81 | PBBZSX           | 0DNCAB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/86372f85-1aed-4602-bbcc-b31f58564297   | IGM_PBBZSX-0DNCAB_20230703_merged_bam.bam        | fcaedd775e254b28db89f3929e523b0b | BAM         |
|   82 | PBBZSX           | 0DNCAB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a87d3fea-fc60-4fa1-bc84-3ce098b36bcc   | IGM_PBBZSX-0DNCAB_20230703_bam.bai               | 415e5558e7fcfbf232a3e97d20b8d5ab | BAI         |
|   83 | PBBZSX           | 0DNCAB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/57bdf54e-3fe2-439a-a95d-df20014c56ca   | IGM_PBBZSX-0DNCAB_20230703_merged_bam.bai        | 29d767fe47566d48e823016c4ecd7b9a | BAI         |
|   84 | PBBZSX           | 0DNCOO_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/942ed71c-f3bd-4eeb-87d3-57468671925a   | IGM_PBBZSX-0DNCOO_20230703_tumor.aln.cram        | 2690d3ac3b358d55d974ece4f7172a79 | CRAM        |
|   85 | PBBZSX           | 0DNCOO_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/3ae8ffb2-27f5-4aa0-94b0-c1530ebeb3c1   | IGM_PBBZSX-0DNCOO_20230703_tumor.aln.cram.crai   | cfb36f39cd83d426422703ec2a54c69e | CRAI        |
|   86 | PBBZSX           | 0DNCGW_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f784f847-bc2e-4e42-91e8-9db6544bd85d   | IGM_PBBZSX-0DNCGW_20230703_normal.aln.cram.crai  | 9569646533b03dc8e3b1ba8b83c6043c | CRAI        |
|   87 | PBBZSX           | 0DNCGW_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/48022a3a-a58a-420f-b88b-98b1a5926cb4   | IGM_PBBZSX-0DNCGW_20230703_normal.aln.cram       | 4f4ad93e08968a9a215813ab9dc7c47b | CRAM        |
|   88 | PBCALC           | 0DMKB5_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f7e07840-7641-47fa-9b42-964c5b5a1283   | IGM_PBCALC-0DMKB5_20230614_bam.bam               | 941606aa1ae90d0637c7b25c9adcff70 | BAM         |
|   89 | PBCALC           | 0DMKB5_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/581cbd03-34b2-47e4-953c-5311a12e8b1a   | IGM_PBCALC-0DMKB5_20230614_bam.bai               | 5c8ff63a2a349df8a7d17c095bfbc51f | BAI         |
|   90 | PBCALC           | 0DMKB5_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0ef1748b-6eda-476b-aa78-0c187c806891   | IGM_PBCALC-0DMKB5_20230614_merged_bam.bam        | dbb0f22ed6b9e53cff357ccfcc4e6423 | BAM         |
|   91 | PBCALC           | 0DMKB5_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f46f4ddc-eb1c-4df6-8694-08f7c33d8bad   | IGM_PBCALC-0DMKB5_20230614_merged_bam.bai        | 7fb49ded6b7273419f55045519b0ef3e | BAI         |
|   92 | PBCALC           | 0DMKKK_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/153cbe3e-1a66-4f93-851e-ef7844420538   | IGM_PBCALC-0DMKKK_20230609_normal.aln.cram.crai  | 90d8ef0b1d755bc03116d98d1d6140cc | CRAI        |
|   93 | PBCALC           | 0DMKKK_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/0f64ada9-125b-4f5a-bf34-71afc2646e6a   | IGM_PBCALC-0DMKKK_20230609_normal.aln.cram       | bc5db475cee14f5ceb7e36e8ae4f5ab8 | CRAM        |
|   94 | PBCALC           | 0DMKN9_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/ae0d7b5a-18e3-46d5-845e-7900a8a06b1a   | IGM_PBCALC-0DMKN9_20230609_tumor.aln.cram.crai   | ec15aa394c02fca6da302b79558b51d7 | CRAI        |
|   95 | PBCALC           | 0DMKN9_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/2fc35384-0e6b-4264-89e7-63193b0bd233   | IGM_PBCALC-0DMKN9_20230609_tumor.aln.cram        | b8bcf42b523c835a03a77e2f990853f7 | CRAM        |
|   96 | PBBZXG           | 0DMBLF_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/19792462-653e-49be-b1c4-65b806f0fddf   | IGM_PBBZXG-0DMBLF_20230605_normal.aln.cram       | 0f0412775d54a7a80d31e36c312bc0e0 | CRAM        |
|   97 | PBBZXG           | 0DMBLF_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f2bbb465-1f1a-4931-a2ee-47d34a37fb91   | IGM_PBBZXG-0DMBLF_20230605_normal.aln.cram.crai  | 2fbba172594c08421567691e4ce004ff | CRAI        |
|   98 | PBBZXG           | 0DMBM8_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/b8e65efc-8dd0-4a71-939a-72eb8ac15e6c   | IGM_PBBZXG-0DMBM8_20230605_tumor.aln.cram        | ba1274283bee978e6ecf2440ff6457fd | CRAM        |
|   99 | PBBZXG           | 0DMBM8_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/286d8a11-0dda-4497-b1bb-c6ef2f2ee043   | IGM_PBBZXG-0DMBM8_20230605_tumor.aln.cram.crai   | c4ca06a8943e9d30417e1da6e7d4537d | CRAI        |
|  100 | PBBZXG           | 0DMBLZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4e1331a9-655b-4e88-8d40-43aaafe124f4   | IGM_PBBZXG-0DMBLZ_20230605_merged_bam.bam        | 17cdcced97eb475555c7ab55760b1f79 | BAM         |
|  101 | PBBZXG           | 0DMBLZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2a4bd5a7-f8ce-42bc-8acf-4ccb4a54461f   | IGM_PBBZXG-0DMBLZ_20230605_bam.bai               | ab73fb064f76ddb26f14278f881b7f2f | BAI         |
|  102 | PBBZXG           | 0DMBLZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2bd3c150-1e6f-427f-bd9b-d123cab0494f   | IGM_PBBZXG-0DMBLZ_20230605_merged_bam.bai        | f0a1de0def94f8b88f9cbab2acfcc5c4 | BAI         |
|  103 | PBBZXG           | 0DMBLZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/dc37f17b-8b00-4294-a3b0-48ca7824c937   | IGM_PBBZXG-0DMBLZ_20230605_bam.bam               | 402f65ae99fe01338f038988d2976ce8 | BAM         |
|  104 | PBBVVI           | 0DLRHZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/915ca3e8-9a8a-4cd3-a36e-8a8238ab48e7   | IGM_PBBVVI-0DLRHZ_20230516_bam.bai               | 06bf725938a95882a5ed7921540fbcce | BAI         |
|  105 | PBBVVI           | 0DLRHZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/61e078d6-9977-446b-913f-023e54894d8f   | IGM_PBBVVI-0DLRHZ_20230516_merged_bam.bam        | ccb20f438c1133d05aa1eaedd88a1f93 | BAM         |
|  106 | PBBVVI           | 0DLRHZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/88fa1215-cf6a-48ec-b34e-b5ebb75027af   | IGM_PBBVVI-0DLRHZ_20230516_bam.bam               | f9a3f417476745247abc8461ea3a7fa2 | BAM         |
|  107 | PBBVVI           | 0DLRHZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d466b954-dd9e-4e09-9067-3e0287420176   | IGM_PBBVVI-0DLRHZ_20230516_merged_bam.bai        | 8d843bdf0adf603e373b9891041a26bd | BAI         |
|  108 | PBBVVI           | 0DLRHE_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/4e2adaab-0cf1-4557-b236-135ff02ee21b   | IGM_PBBVVI-0DLRHE_20230522_normal.aln.cram.crai  | c83b75f20bb4677628c9eff0a5f57748 | CRAI        |
|  109 | PBBVVI           | 0DLRHE_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b2f2a9c2-094e-4134-9f80-66d042744d03   | IGM_PBBVVI-0DLRHE_20230522_normal.aln.cram       | fbdbf6a1a8e29899ef125a67c6b5d6c6 | CRAM        |
|  110 | PBBVVI           | 0DLRIB_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/cf007413-37a3-453d-82b8-68ee1aeb6ae5   | IGM_PBBVVI-0DLRIB_20230522_tumor.aln.cram.crai   | 53e4228573b7918acbe52994c7b2ebe0 | CRAI        |
|  111 | PBBVVI           | 0DLRIB_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/17a3fb6e-e46d-4007-9300-cd10133e868a   | IGM_PBBVVI-0DLRIB_20230522_tumor.aln.cram        | 452a0259789ddc7982a49d0267455fae | CRAM        |
|  112 | PBBYCD           | 0DLIFR_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/ed0ab96b-496e-4410-910e-ed365b3f2fd5   | IGM_PBBYCD-0DLIFR_20230515_tumor.aln.cram        | d0f86d01c481a6bd23f9428abf2d1c71 | CRAM        |
|  113 | PBBYCD           | 0DLIFR_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/64083a46-eff3-44ae-9646-bdbe716c38c5   | IGM_PBBYCD-0DLIFR_20230515_tumor.aln.cram.crai   | 448c33d59e039c961f4e05692e6039cf | CRAI        |
|  114 | PBBYCD           | 0DLIF6_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/39ce5d27-2e0f-4c55-be5a-a6401728c503   | IGM_PBBYCD-0DLIF6_20230515_normal.aln.cram       | 1fe9430e11525dfc0c489e9a3ab73b8d | CRAM        |
|  115 | PBBYCD           | 0DLIF6_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/a6aec1f6-d5d9-4bc6-8a8f-80136190ed9e   | IGM_PBBYCD-0DLIF6_20230515_normal.aln.cram.crai  | 682bf0a151d3fb05cd47f78b50e5e339 | CRAI        |
|  116 | PBBYCD           | 0DLIEZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/5d4f4676-501e-47c4-a1d5-f637fdafc55a   | IGM_PBBYCD-0DLIEZ_20230509_merged_bam.bai        | e5fa55daadb9d9b6769b8905dd2429e3 | BAI         |
|  117 | PBBYCD           | 0DLIEZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/1429b2a6-80e0-462c-972e-52ce10658d00   | IGM_PBBYCD-0DLIEZ_20230509_merged_bam.bam        | 536f10e8931b9b309c4f73c69aa4d134 | BAM         |
|  118 | PBBYCD           | 0DLIEZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/02d6666a-cbb8-4860-84c5-2e60c64b756f   | IGM_PBBYCD-0DLIEZ_20230509_bam.bai               | d3ba039fb90a0b2664a482d38fba3899 | BAI         |
|  119 | PBBYCD           | 0DLIEZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b7655ecb-0a2f-4248-82cc-9c6286dcc3ec   | IGM_PBBYCD-0DLIEZ_20230509_bam.bam               | a5cd54a7b7af8e89333ebc228f971280 | BAM         |
|  120 | PBBYNW           | 0DKWNP_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/ac8bfd26-2fa1-4380-8e32-ec1a11d66cb1   | IGM_PBBYNW-0DKWNP_20230428_normal.aln.cram.crai  | b9ea13637b3072b0352b1c164b0e1059 | CRAI        |
|  121 | PBBYNW           | 0DKWNP_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/25cc2d10-afb2-4258-acb0-0e6e79e57646   | IGM_PBBYNW-0DKWNP_20230428_normal.aln.cram       | c9eb6400d8cbee566b6c2542c7c080fa | CRAM        |
|  122 | PBBYNW           | 0DKWND_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/68420fea-70c8-4bfe-a332-88cf4217f0f9   | IGM_PBBYNW-0DKWND_20230428_tumor.aln.cram.crai   | b618ece9e4ca211d15893ed62e258e8c | CRAI        |
|  123 | PBBYNW           | 0DKWND_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/3343eb07-4f7b-463a-b78c-3054e48427d8   | IGM_PBBYNW-0DKWND_20230428_tumor.aln.cram        | 2a72f1de301f344471fc03395c6fd287 | CRAM        |
|  124 | PBBYNW           | 0DKWN2_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/303f9880-183d-4b14-9c77-afcf909fd887   | IGM_PBBYNW-0DKWN2_20230425_merged_bam.bai        | 451834f2d5df7fc4a93aec4f63171569 | BAI         |
|  125 | PBBYNW           | 0DKWN2_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/9c2ae886-91d8-4bad-a12f-80bf918ea9d5   | IGM_PBBYNW-0DKWN2_20230425_bam.bam               | 1f7296740fbf0613f1cf057b83488df1 | BAM         |
|  126 | PBBYNW           | 0DKWN2_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0a17f523-d17d-40e2-8537-decc7a4934fa   | IGM_PBBYNW-0DKWN2_20230425_bam.bai               | 6218454613369a023880d4e184a0acd6 | BAI         |
|  127 | PBBYNW           | 0DKWN2_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/5c6a3270-84a0-444d-a3b1-df215a822f1b   | IGM_PBBYNW-0DKWN2_20230425_merged_bam.bam        | ccd56a30e41df12fa91b2e15b7c368c3 | BAM         |
|  128 | PBBXSM           | 0DKIHQ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/efbfe701-5de7-4716-9324-00d39ba33b1e   | IGM_PBBXSM-0DKIHQ_20230419_normal.aln.cram.crai  | 450bc0f3edb052b4c51c19f25cc36a07 | CRAI        |
|  129 | PBBXSM           | 0DKIHQ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/ee63a36b-79eb-4d2e-a858-8b82282d998d   | IGM_PBBXSM-0DKIHQ_20230419_normal.aln.cram       | e053b4a32976da49373e62c8ecba7ecb | CRAM        |
|  130 | PBBXSM           | 0DKII2_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/353020be-4522-44bf-b02a-4284ad32c34a   | IGM_PBBXSM-0DKII2_20230411_bam.bam               | c6e9455c94536683c288e10d54b09ccf | BAM         |
|  131 | PBBXSM           | 0DKII2_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a44d6023-088a-40eb-abc1-05431d451b10   | IGM_PBBXSM-0DKII2_20230411_merged_bam.bai        | 50d937e08f7bf33a3200b99f853b9724 | BAI         |
|  132 | PBBXSM           | 0DKII2_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a8e9fa3d-4e53-43e4-8157-ed882ce78c27   | IGM_PBBXSM-0DKII2_20230411_bam.bai               | 782ad8c5b692ba01754e123b84329071 | BAI         |
|  133 | PBBXSM           | 0DKII2_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3c9fafe5-3475-45df-b2d9-c064d7724390   | IGM_PBBXSM-0DKII2_20230411_merged_bam.bam        | 3ff87eae0a5e7345dbe06473ad0fe60b | BAM         |
|  134 | PBBXSM           | 0DKIIC_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/a6c44d95-cbdf-4f8c-8c3d-778d8ca02819   | IGM_PBBXSM-0DKIIC_20230419_tumor.aln.cram.crai   | 8f5f87b98f245592a42f81fb7bed8cbb | CRAI        |
|  135 | PBBXSM           | 0DKIIC_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/7c7b3519-38d4-4a4f-a50e-c2284303d68a   | IGM_PBBXSM-0DKIIC_20230419_tumor.aln.cram        | ffc1a097a0972309513dea58285eeac2 | CRAM        |
|  136 | PBBWGV           | 0DJ6K1_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/814603c7-2a38-4eb2-84de-cc9d9e373ce5   | IGM_PBBWGV-0DJ6K1_20230314_normal.aln.cram.crai  | dee48746bdb4616f36d63ca82eaadcc7 | CRAI        |
|  137 | PBBWGV           | 0DJ6K1_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/d2aea6de-af8d-4232-bf86-eefa39af7a8e   | IGM_PBBWGV-0DJ6K1_20230314_normal.aln.cram       | 5e297505ba0212247a74acf18fcbf194 | CRAM        |
|  138 | PBBWGV           | 0DJ6KB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f83b5b01-4bc0-4d26-bfab-fe0f4ba37290   | IGM_PBBWGV-0DJ6KB_20230309_merged_bam.bai        | 49099811aa9d4858da688a36358fab64 | BAI         |
|  139 | PBBWGV           | 0DJ6KB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3465a1a5-ae54-43ee-97fb-033feadd52c2   | IGM_PBBWGV-0DJ6KB_20230309_merged_bam.bam        | 23ca4d1c5fbfd39c8d5904d892a33b00 | BAM         |
|  140 | PBBWGV           | 0DJ6KB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/37360b91-ad4d-4626-95f9-e361bcafecb5   | IGM_PBBWGV-0DJ6KB_20230309_bam.bai               | cd59dccb42d2884a51377c3652dd721d | BAI         |
|  141 | PBBWGV           | 0DJ6KB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/eca33780-9eeb-49d5-a703-419e75f71842   | IGM_PBBWGV-0DJ6KB_20230309_bam.bam               | 767166fbda4de4829a22634bea02e31f | BAM         |
|  142 | PBBWGV           | 0DJ6KO_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/d2f2bc5d-e77e-41a2-aac8-32b619afb9f1   | IGM_PBBWGV-0DJ6KO_20230314_tumor.aln.cram.crai   | 3f75cff1ba08676d0c4d044397fa077e | CRAI        |
|  143 | PBBWGV           | 0DJ6KO_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/29e46c15-28bf-4daa-8e7f-e23a681e3a59   | IGM_PBBWGV-0DJ6KO_20230314_tumor.aln.cram        | 7c38bfb4ad088d5823873297afafa8e8 | CRAM        |
|  144 | PBBWUS           | 0DJKKH_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/1fb27c59-fcc7-473f-950e-1b59d42f7e74   | IGM_PBBWUS-0DJKKH_20230321_bam.bam               | 2384639c7cb420a061512edc52c537a0 | BAM         |
|  145 | PBBWUS           | 0DJKKH_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0785f351-b654-417f-aaa8-e1538b0df365   | IGM_PBBWUS-0DJKKH_20230321_bam.bai               | 91f7367bbb778998273e3d84e6565591 | BAI         |
|  146 | PBBWUS           | 0DJKKH_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/aa59b2be-0b71-4852-b253-41582acee002   | IGM_PBBWUS-0DJKKH_20230321_merged_bam.bai        | 6a25bd428a191f214a959162774de4e6 | BAI         |
|  147 | PBBWUS           | 0DJKKH_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a4b4a87f-9a75-469d-b368-d9415b034d0d   | IGM_PBBWUS-0DJKKH_20230321_merged_bam.bam        | 37226412524add438f80a4faf83b3219 | BAM         |
|  148 | PBBWUS           | 0DJKK8_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/adbeac85-ae81-4e67-bf7e-ee5797a10cdd   | IGM_PBBWUS-0DJKK8_20230324_normal.aln.cram.crai  | fbab16576d19e8e44753681fb8979689 | CRAI        |
|  149 | PBBWUS           | 0DJKK8_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/864256a6-6ab8-4a5e-bea0-1c8db208c55f   | IGM_PBBWUS-0DJKK8_20230324_normal.aln.cram       | 1cf95d028af1978389f6b3359b72deb3 | CRAM        |
|  150 | PBBWUS           | 0DJKL0_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/688c9f62-5586-465d-9f56-a5519512be4f   | IGM_PBBWUS-0DJKL0_20230324_tumor.aln.cram        | 68630b35012d289b63d34c846ed1a4a4 | CRAM        |
|  151 | PBBWUS           | 0DJKL0_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/53b0fa29-dc46-4b94-856f-ae69b05d6354   | IGM_PBBWUS-0DJKL0_20230324_tumor.aln.cram.crai   | a4745944f36653ee185c62f83839617c | CRAI        |
|  152 | PBBWMI           | 0DJ8KO_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e5985dd8-2b15-422c-b968-8720557a3332   | IGM_PBBWMI-0DJ8KO_20230309_bam.bam               | a3cb7ff535dff0c77a998eee19f2264c | BAM         |
|  153 | PBBWMI           | 0DJ8KO_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/9e120868-c79b-44ce-8552-49a607f175bb   | IGM_PBBWMI-0DJ8KO_20230309_bam.bai               | 7b19fe110241c38ee8ac9ec2b92bf424 | BAI         |
|  154 | PBBWMI           | 0DJ8KO_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/71804f05-c5cf-4c7f-92ce-43308a6645d8   | IGM_PBBWMI-0DJ8KO_20230309_merged_bam.bai        | d0fc777eeecae02c0cc5a728de8ff706 | BAI         |
|  155 | PBBWMI           | 0DJ8KO_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/39436e83-454b-42cb-b90a-1d555c9e0546   | IGM_PBBWMI-0DJ8KO_20230309_merged_bam.bam        | 9d44010a41dcedd8cb7b7ba2907c3b56 | BAM         |
|  156 | PBBWMI           | 0DJKKO_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/f76afc08-1a6a-466c-833a-615c7732cd03   | IGM_PBBWMI-0DJKKO_20230323_tumor.aln.cram        | 34648e67e1262f60d19353fe3b15a855 | CRAM        |
|  157 | PBBWMI           | 0DJKKO_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/a7be919a-1bff-48be-b2ad-6544f40b9939   | IGM_PBBWMI-0DJKKO_20230323_tumor.aln.cram.crai   | 5e38d32e894343a474dd9ad51b6a403b | CRAI        |
|  158 | PBBWMI           | 0DJKK7_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/49f41e09-057d-4242-b096-2653fdbc406c   | IGM_PBBWMI-0DJKK7_20230323_normal.aln.cram.crai  | f1eba48cf69e60d3dcc0f9827e5533db | CRAI        |
|  159 | PBBWMI           | 0DJKK7_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b427bd05-beab-4b40-bfdf-2be4e6e6828b   | IGM_PBBWMI-0DJKK7_20230323_normal.aln.cram       | 574f7bc76a320ef089b838be4920e604 | CRAM        |
|  160 | PBBWFV           | 0DIWMG_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/ce844855-0607-4a42-a43f-2f910c848715   | IGM_PBBWFV-0DIWMG_20230311_normal.aln.cram       | 47b1f3fcf53fad2b8e44fb2f59fa1639 | CRAM        |
|  161 | PBBWFV           | 0DIWMG_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/7bc5b4e4-1807-492b-b4d3-233ca62ca037   | IGM_PBBWFV-0DIWMG_20230311_normal.aln.cram.crai  | 048b1f85d376f6ad5848a7dd7b0816a6 | CRAI        |
|  162 | PBBWFV           | 0DIWMH_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/918bb352-3566-4010-8cd9-3e7171893ee9   | IGM_PBBWFV-0DIWMH_20230303_bam.bam               | 85f7b2693dfa62fb68fa66fdab02bda7 | BAM         |
|  163 | PBBWFV           | 0DIWMH_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/1f504e5a-7ed2-4474-83cb-5e83779324d4   | IGM_PBBWFV-0DIWMH_20230303_merged_bam.bai        | a91f967bb483b8e0518d6ae42dde10a2 | BAI         |
|  164 | PBBWFV           | 0DIWMH_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4e8c86af-69c4-4967-a12d-f5f8c3190bf9   | IGM_PBBWFV-0DIWMH_20230303_bam.bai               | 58ebed5f598c330f9d236af230f0194b | BAI         |
|  165 | PBBWFV           | 0DIWMH_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/199df787-4f9b-46ab-ae4c-2d1d745bf4a7   | IGM_PBBWFV-0DIWMH_20230303_merged_bam.bam        | 42d28902839211487aed204f31c9a594 | BAM         |
|  166 | PBBWFV           | 0DIWMI_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/e5330bb7-9a0c-4c48-aff2-da98dd2cdafd   | IGM_PBBWFV-0DIWMI_20230311_tumor.aln.cram.crai   | df19828e9c951973454dbabe316798c6 | CRAI        |
|  167 | PBBWFV           | 0DIWMI_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/546e9598-0eaa-4213-ab59-562d05e22857   | IGM_PBBWFV-0DIWMI_20230311_tumor.aln.cram        | c263f712193039ec431e6510dccf60b9 | CRAM        |
|  168 | PBBWFG           | 0DIW76_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/5a095f2f-3021-4982-904c-2100e2b15144   | IGM_PBBWFG-0DIW76_20230228_merged_bam.bam        | 29cbd65b4a41e1e4312cbc8c7cd3c883 | BAM         |
|  169 | PBBWFG           | 0DIW76_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/da9d2bbe-4d71-4eab-9771-e842bf14caf9   | IGM_PBBWFG-0DIW76_20230228_bam.bam               | 3dd9ae8fa93979a6c57e737bc443403b | BAM         |
|  170 | PBBWFG           | 0DIW76_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/24d2cd93-b477-45c4-bc00-1a6227b8a5d8   | IGM_PBBWFG-0DIW76_20230228_merged_bam.bai        | 22193d5c9482a6dd5b6083f0b9ea52ab | BAI         |
|  171 | PBBWFG           | 0DIW76_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/930ac440-dc79-4aa6-9440-b990b480a597   | IGM_PBBWFG-0DIW76_20230228_bam.bai               | 3e01011b8026534cd7025dde8de3922c | BAI         |
|  172 | PBBWFG           | 0DIW7U_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/facf72df-b28e-41a3-aa12-a57f4997bff6   | IGM_PBBWFG-0DIW7U_20230306_tumor.aln.cram.crai   | 1677567777d75fdf480b306f3690829f | CRAI        |
|  173 | PBBWFG           | 0DIW7U_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/3d11ff89-fb9e-4365-aa8e-ed301b1fd8f1   | IGM_PBBWFG-0DIW7U_20230306_tumor.aln.cram        | 8c4ed9c2b2760cfec9b25e47796d1bf5 | CRAM        |
|  174 | PBBWFG           | 0DIW6A_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/d1383e7c-b27c-462d-a937-473970ed4b88   | IGM_PBBWFG-0DIW6A_20230306_normal.aln.cram.crai  | d6513121669b9c21492993af23535e14 | CRAI        |
|  175 | PBBWFG           | 0DIW6A_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/edfed3ad-f8c1-40a8-8842-c388e35a6b5f   | IGM_PBBWFG-0DIW6A_20230306_normal.aln.cram       | 1bb9f38e89be1a1a1bef71c35e817f40 | CRAM        |
|  176 | PBBVDV           | 0DIYS3_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/95beb62e-d02a-4652-9dfb-08bb6e2f757e   | IGM_PBBVDV-0DIYS3_20230315_normal.aln.cram       | f6fe6775d03cec4448e772e4c0cb3764 | CRAM        |
|  177 | PBBVDV           | 0DIYS3_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/7f3ab64b-f477-4d76-b224-2aafcac51e35   | IGM_PBBVDV-0DIYS3_20230315_normal.aln.cram.crai  | f5fee8a5f9b505bf7791b884cb0c5507 | CRAI        |
|  178 | PBBVDV           | 0DIYSF_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/9f9dd567-a62e-4fd4-9d80-6b2d20807bc4   | IGM_PBBVDV-0DIYSF_20230315_tumor.aln.cram.crai   | 161abdcdeab64e20cb8b596a501c4549 | CRAI        |
|  179 | PBBVDV           | 0DIYSF_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/1ca7f850-99f3-44f2-bee6-358482735c8a   | IGM_PBBVDV-0DIYSF_20230315_tumor.aln.cram        | 4527805898d12f8b5bbdf4ddc6f82c47 | CRAM        |
|  180 | PBBVDV           | 0DIYS9_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ccf064ce-fde5-4768-9fa2-c95486f76194   | IGM_PBBVDV-0DIYS9_20230307_merged_bam.bam        | 0b1090a0d402a439f44a16b4f7aed16e | BAM         |
|  181 | PBBVDV           | 0DIYS9_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/af166bd9-3948-43db-80a3-33033b21c908   | IGM_PBBVDV-0DIYS9_20230307_bam.bam               | 1d512927206e1558c1b0079d1693a4fc | BAM         |
|  182 | PBBVDV           | 0DIYS9_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/95d9bb02-c9d2-43e2-9d2f-fe47462a75b2   | IGM_PBBVDV-0DIYS9_20230307_bam.bai               | 2021f7f4c534512d813057b35bbe2129 | BAI         |
|  183 | PBBVDV           | 0DIYS9_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7bb2d4e2-c923-46d3-89ea-bf01e2d34fdb   | IGM_PBBVDV-0DIYS9_20230307_merged_bam.bai        | ee401eba7dc38c35bfb63621c88f02e7 | BAI         |
|  184 | PBBWNC           | 0DJGE4_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/6002e7bb-33b3-4633-9c47-df49ff10de73   | IGM_PBBWNC-0DJGE4_20230321_tumor.aln.cram        | 879d6517b76e51e827cb24d13fe6fd65 | CRAM        |
|  185 | PBBWNC           | 0DJGE4_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/172ee9a8-9597-4f29-be9f-18ac79fbc953   | IGM_PBBWNC-0DJGE4_20230321_tumor.aln.cram.crai   | fc4040de87711d93b815535d9bf7f81b | CRAI        |
|  186 | PBBWNC           | 0DJGCR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a9f8b65c-758f-457b-b22f-33252cceb2fc   | IGM_PBBWNC-0DJGCR_20230313_merged_bam.bam        | 1ba16185df63ff01fef8c1e4ad40e065 | BAM         |
|  187 | PBBWNC           | 0DJGCR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4a7eb75a-27ad-4375-825f-55021a3fe5c3   | IGM_PBBWNC-0DJGCR_20230313_merged_bam.bai        | aa228136ea260809df8c2c207853311a | BAI         |
|  188 | PBBWNC           | 0DJGCR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/eb8c6e3a-23e0-416a-b341-dc9c57348e92   | IGM_PBBWNC-0DJGCR_20230313_bam.bai               | eaa1e40840fa21bd03720ebbf2760c3d | BAI         |
|  189 | PBBWNC           | 0DJGCR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/82d0edcf-9417-4dea-86ce-e7a5ecd6e68c   | IGM_PBBWNC-0DJGCR_20230313_bam.bam               | 384eeb90d137c7845d901cca72139a23 | BAM         |
|  190 | PBBWNC           | 0DJG6Z_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/775c552a-04db-4115-8114-5344a60f7015   | IGM_PBBWNC-0DJG6Z_20230321_normal.aln.cram.crai  | 38b7906c7712e1c74e86b387829fb37d | CRAI        |
|  191 | PBBWNC           | 0DJG6Z_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/dfbe1614-a86e-44df-9153-e4f6ed1cac36   | IGM_PBBWNC-0DJG6Z_20230321_normal.aln.cram       | c66f66f85dfa6076c896c37ed2f066f0 | CRAM        |
|  192 | PBBTUX           | 0DITTN_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/60876f2b-9b0a-4308-bdc1-d68262fcaf17   | IGM_PBBTUX-0DITTN_20230304_tumor.aln.cram.crai   | 4be020aa767909194d6956f34a505899 | CRAI        |
|  193 | PBBTUX           | 0DITTN_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/d6d44086-a2c1-4c14-abfa-f755621c8fab   | IGM_PBBTUX-0DITTN_20230304_tumor.aln.cram        | 8ec06a1d4863270f276cbb6ff9c371e8 | CRAM        |
|  194 | PBBTUX           | 0DITS9_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b3db8918-15ca-44b3-9a9a-79012b551d1c   | IGM_PBBTUX-0DITS9_20230304_normal.aln.cram.crai  | 8c9ac52115d5dd9898a9d8b4799555f8 | CRAI        |
|  195 | PBBTUX           | 0DITS9_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/ff86e4d0-e6f1-4695-8d70-9ae7bfd39747   | IGM_PBBTUX-0DITS9_20230304_normal.aln.cram       | cf5ab55869f7dc8fabd3ac40b588bf52 | CRAM        |
|  196 | PBBTUX           | 0DITSU_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/6a12afbc-1e13-413a-ae7c-2f87bf7f4e40   | IGM_PBBTUX-0DITSU_20230225_merged_bam.bai        | 1d7b85fabe51513466430b73d0b32267 | BAI         |
|  197 | PBBTUX           | 0DITSU_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2d4e90e9-ec3f-48e5-8ea0-62e519c643c5   | IGM_PBBTUX-0DITSU_20230225_bam.bai               | 904e50f973e5d25900e4e7874f194262 | BAI         |
|  198 | PBBTUX           | 0DITSU_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/34770f3e-3e64-45e0-9abc-903f706437c5   | IGM_PBBTUX-0DITSU_20230225_merged_bam.bam        | 9de3ebe31d8ae5243911794112eb03d5 | BAM         |
|  199 | PBBTUX           | 0DITSU_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f730be35-50f5-4e8a-af18-f184c1e157e7   | IGM_PBBTUX-0DITSU_20230225_bam.bam               | d260e25c7859bbb3d75f5fc4153bf312 | BAM         |
|  200 | PBBVPC           | 0DIKJO_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c0b8a460-713f-4599-9c94-c43a32d32ed5   | IGM_PBBVPC-0DIKJO_20230218_bam.bam               | 3ce28d239a8573a1167c6ee9a3857984 | BAM         |
|  201 | PBBVPC           | 0DIKJO_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/fc56987f-739d-4adf-ad7d-b435399b1164   | IGM_PBBVPC-0DIKJO_20230218_bam.bai               | 97860d3fa23abcb5ba4950c2f0dae361 | BAI         |
|  202 | PBBVPC           | 0DIKJO_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/6b40b5b4-e5eb-4f2b-a8f1-7d5b6262a98f   | IGM_PBBVPC-0DIKJO_20230218_merged_bam.bam        | d4d1eceec02ebf236b9e48baf18cecbe | BAM         |
|  203 | PBBVPC           | 0DIKJO_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f5d7a23a-f65c-4eca-808e-7695b7c00cdc   | IGM_PBBVPC-0DIKJO_20230218_merged_bam.bai        | 5faeecd35b770ebbda0a51e7fe95c027 | BAI         |
|  204 | PBBTLR           | 0DGJ64_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/a8cf8423-cb5f-40fc-8d4e-7ea42bb9342b   | IGM_PBBTLR-0DGJ64_20221226_normal.aln.cram.crai  | afb24a1d9226a1d4aff8415ecaa69499 | CRAI        |
|  205 | PBBTLR           | 0DGJ64_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/8842e357-7fcb-4724-8011-7bdc4ed6a9c1   | IGM_PBBTLR-0DGJ64_20221226_normal.aln.cram       | 08b77bf54e59c0e33b133cc98b351a90 | CRAM        |
|  206 | PBBTLR           | 0DGJ4Q_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8380ae8f-cfef-431e-bc49-5dbd5ef55c3d   | IGM_PBBTLR-0DGJ4Q_20221223_merged_bam.bai        | 0efc572c28e9a2f75603c92e318432f4 | BAI         |
|  207 | PBBTLR           | 0DGJ4Q_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/cec886a2-b3bc-47bc-9519-68227fd25b04   | IGM_PBBTLR-0DGJ4Q_20221223_merged_bam.bam        | 4cc1371d5e9f96e7fa113049a98aa8fe | BAM         |
|  208 | PBBTLR           | 0DGJ4Q_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/fa631e6f-3173-4d9a-95ab-45c1d354196b   | IGM_PBBTLR-0DGJ4Q_20221223_bam.bam               | e0ae45fd89c8ae415c058d04589a991c | BAM         |
|  209 | PBBTLR           | 0DGJ4Q_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c6e932c0-a673-47e4-80b3-1bc31aa60b03   | IGM_PBBTLR-0DGJ4Q_20221223_bam.bai               | c51e2780b41189ac2b6fb5279b5b11f3 | BAI         |
|  210 | PBBTLR           | 0DGJ83_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/d4263f1a-e223-477c-ba6b-a9e018bfc1ef   | IGM_PBBTLR-0DGJ83_20221226_tumor.aln.cram.crai   | 2061bf2b544c2d1f9521f53e1b5a9d54 | CRAI        |
|  211 | PBBTLR           | 0DGJ83_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/19a389fc-eed1-400b-82e5-25d21b93b4f5   | IGM_PBBTLR-0DGJ83_20221226_tumor.aln.cram        | 735b8b284662f5e30f3e9b6b735a6304 | CRAM        |
|  212 | PBBUCX           | 0DH732_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/c3c818a8-d161-4423-9eb4-2b9fd4ced333   | IGM_PBBUCX-0DH732_20230118_tumor.aln.cram        | 97b3b7a936be020822b448c7d2be45c2 | CRAM        |
|  213 | PBBUCX           | 0DH732_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/ee25961a-9d52-4bfd-8522-a08dcc42cb65   | IGM_PBBUCX-0DH732_20230118_tumor.aln.cram.crai   | d336dd3be56301e44c72c02758c105b3 | CRAI        |
|  214 | PBBUCX           | 0DH6ZR_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/d7d7c3cb-f877-42ec-9e6f-4d1f572404dc   | IGM_PBBUCX-0DH6ZR_20230118_normal.aln.cram       | e0fadf3a70b720ae3678199de8bf73bc | CRAM        |
|  215 | PBBUCX           | 0DH6ZR_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/51a92e74-6bb8-416d-8264-103500972b90   | IGM_PBBUCX-0DH6ZR_20230118_normal.aln.cram.crai  | 74e07481b810b433dc803dd2fad053f1 | CRAI        |
|  216 | PBBUCX           | 0DH72N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/943272f4-5a36-4d56-a780-ca78b8f4d12a   | IGM_PBBUCX-0DH72N_20230113_bam.bam               | 1f386796834e85de058713729d21e065 | BAM         |
|  217 | PBBUCX           | 0DH72N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3093e5b8-f6d2-415d-b745-1eeea1b50bf7   | IGM_PBBUCX-0DH72N_20230113_merged_bam.bai        | 72f1f5216b54022bfd8af785b4a7900f | BAI         |
|  218 | PBBUCX           | 0DH72N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/06620f98-e274-4b34-b840-ffe3e98c15cd   | IGM_PBBUCX-0DH72N_20230113_merged_bam.bam        | efd056b024bbb0f33f175727135b063c | BAM         |
|  219 | PBBUCX           | 0DH72N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d32f5db1-5c21-4f6e-95f3-467f0121f03f   | IGM_PBBUCX-0DH72N_20230113_bam.bai               | 5278185478cc72f62b7d4a2a9f7bec61 | BAI         |
|  220 | PBBTXS           | 0DI7VE_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/23b81257-1299-4675-aea0-414bc190115e   | IGM_PBBTXS-0DI7VE_20230217_normal.aln.cram.crai  | 70586fd5ebcb89c6265dded40f6cd559 | CRAI        |
|  221 | PBBTXS           | 0DI7VE_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/592b9030-da6c-44bc-959f-86e29a9a2b65   | IGM_PBBTXS-0DI7VE_20230217_normal.aln.cram       | 988f4a260290600111645d3ee366e4ff | CRAM        |
|  222 | PBBTXS           | 0DI7V2_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/ea3028e5-ca45-4aa8-b4f3-02da6a5115b6   | IGM_PBBTXS-0DI7V2_20230217_tumor.aln.cram        | fe46017fb31a52b580164117eff0a9ff | CRAM        |
|  223 | PBBTXS           | 0DI7V2_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/48de61a3-ce0f-4294-b3cb-5b8a20b375fb   | IGM_PBBTXS-0DI7V2_20230217_tumor.aln.cram.crai   | 5a85220da55898aad25bac3013411313 | CRAI        |
|  224 | PBBRET           | 0DFXFB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/6c30a6b1-4530-49ab-bad9-0330e94df070   | IGM_PBBRET-0DFXFB_20221213_bam.bai               | 1c366a68e66efec314b7610a73fa55e0 | BAI         |
|  225 | PBBRET           | 0DFXFB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/bf5e3167-d24c-4c65-8e50-e0728dfb4be7   | IGM_PBBRET-0DFXFB_20221213_merged_bam.bam        | f01c1d24e49c673c66732480e0bbfaec | BAM         |
|  226 | PBBRET           | 0DFXFB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/76689a6b-0b77-4230-87d7-594836385bc0   | IGM_PBBRET-0DFXFB_20221213_merged_bam.bai        | 89b383daba85cc253a58b3709d773750 | BAI         |
|  227 | PBBRET           | 0DFXFB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/13175b3c-f4fb-454f-abec-cc6646d9c73d   | IGM_PBBRET-0DFXFB_20221213_bam.bam               | f864849b79a02141f4cd5ae8bfaeffbf | BAM         |
|  228 | PBBRET           | 0DFXF3_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/87360ded-4ee7-4b52-bd9e-fd3307c7392e   | IGM_PBBRET-0DFXF3_20221221_normal.aln.cram.crai  | 1b561bfe22a90f322c29de94b9d5af8f | CRAI        |
|  229 | PBBRET           | 0DFXF3_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/ca7c1e97-cb08-4d4a-8d46-a1cbd84225c9   | IGM_PBBRET-0DFXF3_20221221_normal.aln.cram       | c22939b4327c80db7f59704eff5a5ce9 | CRAM        |
|  230 | PBBRET           | 0DFXH5_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/3d858a19-79dc-4ea7-8f14-150839d4acfb   | IGM_PBBRET-0DFXH5_20221221_tumor.aln.cram        | b49943db3714ed0705708075f2366e76 | CRAM        |
|  231 | PBBRET           | 0DFXH5_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/930ab329-78bf-4860-93cd-fbd80533da40   | IGM_PBBRET-0DFXH5_20221221_tumor.aln.cram.crai   | 10e1468ab7545540b2409c054f4dc519 | CRAI        |
|  232 | PBBTWG           | 0DGSR6_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/dbbfe839-548a-413a-ad10-01558cded67c   | IGM_PBBTWG-0DGSR6_20230111_normal.aln.cram.crai  | 5fc240dd6369cb2a2cf3cd1270dc3e8f | CRAI        |
|  233 | PBBTWG           | 0DGSR6_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/94607cab-95e9-4ede-9223-d05ea553cb6b   | IGM_PBBTWG-0DGSR6_20230111_normal.aln.cram       | f3fe558c558de7987590cbf415c594c7 | CRAM        |
|  234 | PBBTWG           | 0DGSRA_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/3a0271e9-886e-4b33-863b-370b26c7824d   | IGM_PBBTWG-0DGSRA_20230111_tumor.aln.cram.crai   | 515462acb392f7657ead09570e12cca8 | CRAI        |
|  235 | PBBTWG           | 0DGSRA_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/41d9b9f0-0682-4139-8099-2ab74f3260fc   | IGM_PBBTWG-0DGSRA_20230111_tumor.aln.cram        | 1f820b7256edf62e336362f7b48969f7 | CRAM        |
|  236 | PBBSKE           | 0DFXEW_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/7dd21118-4eb6-4162-a605-906b35e4be7c   | IGM_PBBSKE-0DFXEW_20221213_normal.aln.cram       | 8f21b0fa3ff124486496e532dfb2ca6e | CRAM        |
|  237 | PBBSKE           | 0DFXEW_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/7d5da84f-1439-4f9c-a825-8d9c18f7ac5c   | IGM_PBBSKE-0DFXEW_20221213_normal.aln.cram.crai  | 5b27c309d9f2b49b382f6aa308698f67 | CRAI        |
|  238 | PBBSKE           | 0DFXFE_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0c1ea3ca-29b5-4300-a669-1459a32d9eec   | IGM_PBBSKE-0DFXFE_20221215_bam.bai               | 294a9ddb96c8b1a576c1512e72f7c144 | BAI         |
|  239 | PBBSKE           | 0DFXFE_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/00f0be46-6d76-44b4-81ca-e7abb35d0c5a   | IGM_PBBSKE-0DFXFE_20221215_merged_bam.bam        | a00265cf3b0f9d160baa63af3b875c44 | BAM         |
|  240 | PBBSKE           | 0DFXFE_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/abc6122a-0b28-4f74-bb87-07c73006618c   | IGM_PBBSKE-0DFXFE_20221215_merged_bam.bai        | 219734ffef08ac56914c62ecfbbf3651 | BAI         |
|  241 | PBBSKE           | 0DFXFE_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/6643e4fb-029a-4535-97d3-d64300794528   | IGM_PBBSKE-0DFXFE_20221215_bam.bam               | 55921e8781446b22cd8a8b333e1bcf72 | BAM         |
|  242 | PBBSKE           | 0DFXH6_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/d1d7fdd9-0290-4205-9a85-bfdd2da791c6   | IGM_PBBSKE-0DFXH6_20221213_tumor.aln.cram.crai   | d50e10a2307f43d14e1aa638479899c7 | CRAI        |
|  243 | PBBSKE           | 0DFXH6_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/6b2efbd4-16c6-4f98-b34e-5f49a5d32077   | IGM_PBBSKE-0DFXH6_20221213_tumor.aln.cram        | 98db857af0fb6fbe65c01205899dddb5 | CRAM        |
|  244 | PBBTTH           | 0DGO9X_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ea5f840e-2cdb-4ead-a88f-97e332ac194f   | IGM_PBBTTH-0DGO9X_20221230_bam.bam               | 175d09aabb85089c83b573f56d39506e | BAM         |
|  245 | PBBTTH           | 0DGO9X_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/db0cbbff-ee51-422c-be79-ace7e9a49487   | IGM_PBBTTH-0DGO9X_20221230_merged_bam.bam        | 1018e2367da85fb32418d0d304aab6b6 | BAM         |
|  246 | PBBTTH           | 0DGO9X_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4be32d8e-3b12-40a4-a35e-8a606f575638   | IGM_PBBTTH-0DGO9X_20221230_merged_bam.bai        | 4ecb88006f19af5ad04f6170303d12c4 | BAI         |
|  247 | PBBTTH           | 0DGO9X_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/184a7fae-7183-4aaa-bd2c-91114e53da53   | IGM_PBBTTH-0DGO9X_20221230_bam.bai               | 3d6ee968345e8d47705570507f9b078d | BAI         |
|  248 | PBBTTH           | 0DGOB0_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f7834229-b709-4b96-8aab-31b8dd8acb1f   | IGM_PBBTTH-0DGOB0_20221230_normal.aln.cram       | d095b9dbd9c1739bfe376d6440e98765 | CRAM        |
|  249 | PBBTTH           | 0DGOB0_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/0a92d7af-0e47-4b4e-ad63-82a24461c492   | IGM_PBBTTH-0DGOB0_20221230_normal.aln.cram.crai  | 3bac1c412117c04f0dab4619748c7830 | CRAI        |
|  250 | PBBTTH           | 0DGOAW_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/d6d4e330-463b-479b-b273-7a6b3f40b756   | IGM_PBBTTH-0DGOAW_20221230_tumor.aln.cram        | 91173f102dc6048e718997035f40a666 | CRAM        |
|  251 | PBBTTH           | 0DGOAW_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/e28d8271-7ae9-4855-9662-f65a48088c78   | IGM_PBBTTH-0DGOAW_20221230_tumor.aln.cram.crai   | 3fb446aacc682cd57a64cc27b22ba55d | CRAI        |
|  252 | PBBSSM           | 0DFSSJ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/bd8ce997-8fa0-42cd-ac8d-601601dd236f   | IGM_PBBSSM-0DFSSJ_20221220_tumor.aln.cram        | dbda9ec39b50721f93cd8e2d0354a227 | CRAM        |
|  253 | PBBSSM           | 0DFSSJ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/55100cd8-374e-4a91-a779-98d49f746cc7   | IGM_PBBSSM-0DFSSJ_20221220_tumor.aln.cram.crai   | 3f9f0306ead965c10092fcb9b3f05410 | CRAI        |
|  254 | PBBSSM           | 0DFSS4_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/47d9bdb0-fae7-4ce7-ba25-643535c30adf   | IGM_PBBSSM-0DFSS4_20221220_normal.aln.cram       | 79f7d2581dce0637bf4c082e810b36d9 | CRAM        |
|  255 | PBBSSM           | 0DFSS4_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/aebb501a-8418-4dcd-8f43-2829cea77d66   | IGM_PBBSSM-0DFSS4_20221220_normal.aln.cram.crai  | 653a53bdd742f2cb7becd3e00c13738d | CRAI        |
|  256 | PBBSSM           | 0DFSQN_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ad802c72-f446-4411-8704-7d1ce5c98471   | IGM_PBBSSM-0DFSQN_20221202_merged_bam.bai        | d97d6815fc67e8caf5cf2942fbb3376a | BAI         |
|  257 | PBBSSM           | 0DFSQN_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7fe7e672-caef-4e42-810c-fadc71a39a2b   | IGM_PBBSSM-0DFSQN_20221202_merged_bam.bam        | 680204f1cc34b64fdde7a2081dacadac | BAM         |
|  258 | PBBSSM           | 0DFSQN_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4fc0129f-1157-4440-bc2d-e6a692723169   | IGM_PBBSSM-0DFSQN_20221202_bam.bai               | ab8067e841b99f15e293202dd1d6f1dc | BAI         |
|  259 | PBBSSM           | 0DFSQN_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b74d1011-1b20-4365-a883-b9472c061950   | IGM_PBBSSM-0DFSQN_20221202_bam.bam               | a54a97bcab2ff6d683001f69990dcadd | BAM         |
|  260 | PBBKSS           | 0DE0R6_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/92edcfdf-63c1-4aa7-9f31-0e08b39bf0e4   | IGM_PBBKSS-0DE0R6_20221020_merged_bam.bai        | 13744ce0e650422067e496166b07bb1e | BAI         |
|  261 | PBBKSS           | 0DE0R6_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e03e1470-956d-4235-a0d4-11723695d4ee   | IGM_PBBKSS-0DE0R6_20221020_bam.bam               | 42d0e84ca0f59d1150914dc97c86eeb2 | BAM         |
|  262 | PBBKSS           | 0DE0R6_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/76a51d38-89cf-490e-854e-581eb4f50bb0   | IGM_PBBKSS-0DE0R6_20221020_merged_bam.bam        | d8bc26a69392151715e7d5c867c666a3 | BAM         |
|  263 | PBBKSS           | 0DE0R6_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b00ed985-e988-4f32-a46c-8dd556d2a4f8   | IGM_PBBKSS-0DE0R6_20221020_bam.bai               | 2d7c695ac4b1901e9887f4a6d74d3002 | BAI         |
|  264 | PBBKSS           | 0DE0T4_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/9bf65bf7-8dfe-494f-8aaa-f6545442b8f7   | IGM_PBBKSS-0DE0T4_20221018_tumor.aln.cram.crai   | ceec5f398b1d3ecc273b8c430f2ccb0f | CRAI        |
|  265 | PBBKSS           | 0DE0T4_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/8facf95f-aa1a-4ae7-b189-4e9eeade4f82   | IGM_PBBKSS-0DE0T4_20221018_tumor.aln.cram        | 285bb4fc2f8a74516aaa9494d4a1a8b6 | CRAM        |
|  266 | PBBKSS           | 0DE0QR_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b6a35d61-ce1c-4818-9a85-7726f5a2d455   | IGM_PBBKSS-0DE0QR_20221018_normal.aln.cram       | 44a9e744568009a12a083fc56ce23286 | CRAM        |
|  267 | PBBKSS           | 0DE0QR_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/310995e4-8e6e-4a77-aa78-32c03c3e16b4   | IGM_PBBKSS-0DE0QR_20221018_normal.aln.cram.crai  | 3e6d95e2ce0452df8eee230ce6c1e081 | CRAI        |
|  268 | PBBJLB           | 0DA2XG_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/70160d46-6aee-45da-a778-e6c0f9ad7426   | IGM_PBBJLB-0DA2XG_20220902a_tumor.aln.cram.crai  | 4a4d9c20059c7f25130e1120f0e2120f | CRAI        |
|  269 | PBBJLB           | 0DA2XG_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/330c13dc-3e38-4614-8a5b-f727f2583bc5   | IGM_PBBJLB-0DA2XG_20220902a_tumor.aln.cram       | 60a70a8e9a56bec7587c7d20683b3fcd | CRAM        |
|  270 | PBBJLB           | 0DA2XH_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/601acf22-d0df-4089-b178-f44a22f1260b   | IGM_PBBJLB-0DA2XH_20220902a_normal.aln.cram.crai | 1bbf7bb61020898036835ebcf35003a4 | CRAI        |
|  271 | PBBJLB           | 0DA2XH_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/266129f6-020b-4795-bb11-e9a9412c10b0   | IGM_PBBJLB-0DA2XH_20220902a_normal.aln.cram      | c6eea66181900d58b576626690d032bf | CRAM        |
|  272 | PBBJLB           | 0DA2WW_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/399b5d70-b783-41c6-b5c7-a54d89572dc3   | IGM_PBBJLB-0DA2WW_20220919_merged_bam.bai        | 6b0d5a69d656e51de2f27e2a14537a68 | BAI         |
|  273 | PBBJLB           | 0DA2WW_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4ba24d64-4fbe-4c91-aacc-7e60e2e6dda0   | IGM_PBBJLB-0DA2WW_20220919_bam.bam               | 0673ec495b1a30aaf6788ca896c4826a | BAM         |
|  274 | PBBJLB           | 0DA2WW_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ca1ee785-d14e-4181-9f11-0d3cbaa492b2   | IGM_PBBJLB-0DA2WW_20220919_bam.bai               | ffc091308464f59335ec4c209dda760f | BAI         |
|  275 | PBBJLB           | 0DA2WW_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/6a9b5a06-ea6c-4baf-9e0b-2d388c14f367   | IGM_PBBJLB-0DA2WW_20220919_merged_bam.bam        | 51ca3de0b9cf15aa32eed1a0eeed1869 | BAM         |
|  276 | PBBLFW           | 0DCBIS_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/895b82de-7fd5-4ca8-b38d-64c23f81fb20   | IGM_PBBLFW-0DCBIS_20220823_merged_bam.bai        | a11f4012fea31af0050d8d2c128345cc | BAI         |
|  277 | PBBLFW           | 0DCBIS_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d5980d40-c17a-4f7c-bca9-acf20426d032   | IGM_PBBLFW-0DCBIS_20220823_merged_bam.bam        | 75d86540ea5b61a02aac33ae0bf0f651 | BAM         |
|  278 | PBBLFW           | 0DCBIS_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/5098280d-5274-46e6-9244-60d56fed97dd   | IGM_PBBLFW-0DCBIS_20220823_bam.bai               | 37061d7a7607415a8b50b02c98642482 | BAI         |
|  279 | PBBLFW           | 0DCBIS_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/800d4e2c-ca79-4c4c-8b67-9a46fe14534d   | IGM_PBBLFW-0DCBIS_20220823_bam.bam               | 9edb4f6539a0797258e37b73066fd7f9 | BAM         |
|  280 | PBBLFW           | 0DCBIU_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/c024231f-6cef-45f9-a276-d34d58ad8d27   | IGM_PBBLFW-0DCBIU_20220902a_normal.aln.cram.crai | 8e08fc8a66d3ef2408c5866067a20538 | CRAI        |
|  281 | PBBLFW           | 0DCBIU_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b465a37e-3041-4a0b-b5d3-d491c49f2168   | IGM_PBBLFW-0DCBIU_20220902a_normal.aln.cram      | 2c64f0deeacbe0e3ad33d2fbf6515cc9 | CRAM        |
|  282 | PBBLFW           | 0DCBJ8_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/68f9d4e3-92b4-40e9-9bda-c3e3380cb2c1   | IGM_PBBLFW-0DCBJ8_20220902a_tumor.aln.cram       | e8254b28e8778ef3820f49981144acf4 | CRAM        |
|  283 | PBBLFW           | 0DCBJ8_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/e3c1d9d3-82aa-465a-8517-84956996cee4   | IGM_PBBLFW-0DCBJ8_20220902a_tumor.aln.cram.crai  | f2cd26b17c932c7c2db6fb7e39db37bf | CRAI        |
|  284 | PBBNKF           | 0DD86W_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f22710b4-8fbd-41bc-a567-53baa78959b8   | IGM_PBBNKF-0DD86W_20220926_normal.aln.cram.crai  | b4f0785888656afdf4359413f98d4b8f | CRAI        |
|  285 | PBBNKF           | 0DD86W_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/89be9570-22cf-4af2-b5d5-8e6406ba9f09   | IGM_PBBNKF-0DD86W_20220926_normal.aln.cram       | 0496f891a49233b1dc86bb6f6e451ee4 | CRAM        |
|  286 | PBBNKF           | 0DD872_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/06694580-395d-414d-a6a9-100e53ae5d20   | IGM_PBBNKF-0DD872_20220926_tumor.aln.cram        | 03434329c9ca5424d1ca8ec88feec5d3 | CRAM        |
|  287 | PBBNKF           | 0DD872_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/e7b3607c-dab9-4c87-917a-68587c1510d8   | IGM_PBBNKF-0DD872_20220926_tumor.aln.cram.crai   | 7a0727c8ea06b308ad887167d7c027d8 | CRAI        |
|  288 | PBBNKF           | 0DD870_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/be6abbc5-63dd-4de5-861c-82623e79d278   | IGM_PBBNKF-0DD870_20220920_bam.bai               | fd296ff53fd4aeed19014072116e79c4 | BAI         |
|  289 | PBBNKF           | 0DD870_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/5dd53bac-35f9-4ca2-93d2-5f806953241b   | IGM_PBBNKF-0DD870_20220920_merged_bam.bam        | e657aab9c93600be361686418c11cb46 | BAM         |
|  290 | PBBNKF           | 0DD870_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3d6de3db-ef51-4a87-bbbc-1e6c1acd5b7d   | IGM_PBBNKF-0DD870_20220920_merged_bam.bai        | b355ed3a012bb23826efa09741b5c3e2 | BAI         |
|  291 | PBBNKF           | 0DD870_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f27c30e4-23a1-4d89-9670-ff38e2df01ff   | IGM_PBBNKF-0DD870_20220920_bam.bam               | 478a75f6fa324dc9aea54da1820762ba | BAM         |
|  292 | PBBLWR           | 0DD53K_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/8b3b390c-cd1e-473e-aa1e-a4f9b231d87d   | IGM_PBBLWR-0DD53K_20220921_tumor.aln.cram.crai   | 0ead790d7862b57ca18ef8de2d19d312 | CRAI        |
|  293 | PBBLWR           | 0DD53K_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/13fc273e-e1e2-48f7-b024-f16953b5218c   | IGM_PBBLWR-0DD53K_20220921_tumor.aln.cram        | b1dbc0a18e4f5a6ccf179a72e5e8808c | CRAM        |
|  294 | PBBLWR           | 0DD5SA_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/055e7bf2-ca69-446f-b6bd-7bb17a36fc5f   | IGM_PBBLWR-0DD5SA_20220920_bam.bam               | c5205e514e2c84b20987a5bdf2fd4290 | BAM         |
|  295 | PBBLWR           | 0DD5SA_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/84e349ef-f040-4071-a4b7-423f86f89921   | IGM_PBBLWR-0DD5SA_20220920_bam.bai               | 21bd417ca8e67d884a20de111babe900 | BAI         |
|  296 | PBBLWR           | 0DD5SA_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a4ef51b5-9497-462e-a186-7808047ca3fb   | IGM_PBBLWR-0DD5SA_20220920_merged_bam.bai        | c7f9bf60d619e895067b2dbe9df669ae | BAI         |
|  297 | PBBLWR           | 0DD5SA_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/62f3b0fa-efd7-403e-8ba2-7170773b61a0   | IGM_PBBLWR-0DD5SA_20220920_merged_bam.bam        | 33fec454ca570662229d7093552df034 | BAM         |
|  298 | PBBLWR           | 0DD53G_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/97b7f42b-17c7-4e08-9e0e-b217abaa0cd1   | IGM_PBBLWR-0DD53G_20220921_normal.aln.cram       | dc7785a380ced0353e56f022008354aa | CRAM        |
|  299 | PBBLWR           | 0DD53G_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/a4573c6a-f87d-477b-9c89-336b9cf4eacf   | IGM_PBBLWR-0DD53G_20220921_normal.aln.cram.crai  | 5207a8414e1cb7bd6a8284f083b8c6dc | CRAI        |
|  300 | PBBKAW           | 0DAL9M_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/18b29c10-dfe8-4183-bc47-962c1af50352   | IGM_PBBKAW-0DAL9M_20220628_merged_bam.bai        | 6d4ae272970f2b4dee1e82171ad1b1ad | BAI         |
|  301 | PBBKAW           | 0DAL9M_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a389d0a5-6965-4e98-b05e-1e2b0924c650   | IGM_PBBKAW-0DAL9M_20220628_bam.bam               | b9360a3f671007712cd51aae4fe6a3f1 | BAM         |
|  302 | PBBKAW           | 0DAL9M_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0a925f6b-4af5-4d60-b70f-fc2210ac9dfe   | IGM_PBBKAW-0DAL9M_20220628_bam.bai               | ed4cd4f57e2194020e845ef1344aa168 | BAI         |
|  303 | PBBKAW           | 0DAL9M_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e0b48801-2e5d-4ac9-9227-8a44770c2230   | IGM_PBBKAW-0DAL9M_20220628_merged_bam.bam        | 9a2f84cb1601b58ccaf776ccfe921e8a | BAM         |
|  304 | PBBKAW           | 0DAL9O_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/3f9ebe1e-6be0-4313-a2ea-de0baa36d6e2   | IGM_PBBKAW-0DAL9O_20220912_normal.aln.cram.crai  | 443ccbb1a99627036eae9a80cf7cafdd | CRAI        |
|  305 | PBBKAW           | 0DAL9O_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/9fb1f989-d747-48b2-a745-4408e677111c   | IGM_PBBKAW-0DAL9O_20220912_normal.aln.cram       | fda2af39fb093278124ef5de65df6e22 | CRAM        |
|  306 | PBBKAW           | 0DAL9N_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/533cd587-b0af-4fa5-93b0-fe4dbeb81b60   | IGM_PBBKAW-0DAL9N_20220912_tumor.aln.cram        | 204a7bd649b7f69d6c020fb9318df9cd | CRAM        |
|  307 | PBBKAW           | 0DAL9N_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/1a494d2b-9789-42b6-ae61-0b25182a7542   | IGM_PBBKAW-0DAL9N_20220912_tumor.aln.cram.crai   | 3e5ccbb73cc8449f828701168c3f7c05 | CRAI        |
|  308 | PBBIBT           | 0DA4I3_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/47518302-6fd3-49ec-bbfb-040598533bbe   | IGM_PBBIBT-0DA4I3_20220902a_normal.aln.cram      | 34c3578d306c7b2f68dfca0b787fcea2 | CRAM        |
|  309 | PBBIBT           | 0DA4I3_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/a0970475-2ccb-44cc-9909-020698e655e7   | IGM_PBBIBT-0DA4I3_20220902a_normal.aln.cram.crai | 15f4126c954077dc93a2c10852090840 | CRAI        |
|  310 | PBBIBT           | 0DA4HY_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/28c77201-159f-47e4-befb-5ab83482ec64   | IGM_PBBIBT-0DA4HY_20220919_bam.bam               | b1780285ad2b3b785d82cc861b584d31 | BAM         |
|  311 | PBBIBT           | 0DA4HY_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/768ea1d6-15c1-4efd-8185-8da058db9dff   | IGM_PBBIBT-0DA4HY_20220919_merged_bam.bai        | ad64a63ab7857f2934022f39cac8aedd | BAI         |
|  312 | PBBIBT           | 0DA4HY_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2d5866ec-62bd-4a4e-907c-906f1eb8cd24   | IGM_PBBIBT-0DA4HY_20220919_merged_bam.bam        | e20c3c2e7de3587461a0e32710e10d4c | BAM         |
|  313 | PBBIBT           | 0DA4HY_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/fce2eca3-46b3-4f26-842d-3c03bfd5f177   | IGM_PBBIBT-0DA4HY_20220919_bam.bai               | 4d80fc12a816e49940230c4db5b1771b | BAI         |
|  314 | PBBIBT           | 0DA4I2_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/91adc344-d4fd-4eb4-a8c3-44f42e3ff065   | IGM_PBBIBT-0DA4I2_20220902a_tumor.aln.cram.crai  | bc90a6368d69b4b256ef55e8337b5af0 | CRAI        |
|  315 | PBBIBT           | 0DA4I2_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/794d3330-c761-457e-9bcc-5bb5d02cf111   | IGM_PBBIBT-0DA4I2_20220902a_tumor.aln.cram       | 63109996c3cc19d35ce309919c422296 | CRAM        |
|  316 | PBBKBT           | 0DBNBL_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/a613d954-c4c0-4bc9-a86b-f91ca1c9bd36   | IGM_PBBKBT-0DBNBL_20220902a_normal.aln.cram      | 7f2f4f2c74cc3df41dc9d9b2c4d822cb | CRAM        |
|  317 | PBBKBT           | 0DBNBL_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f1153a9d-bb24-4827-b570-04fbdbf7c6cc   | IGM_PBBKBT-0DBNBL_20220902a_normal.aln.cram.crai | 1b5be8361d74d02ecea46e3eede9b42e | CRAI        |
|  318 | PBBKBT           | 0DBNCT_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/38b18a85-a627-4743-8019-93a6fd24f3df   | IGM_PBBKBT-0DBNCT_20220902a_tumor.aln.cram       | 4d859dfe3e152b1d2c2d82ea158b09ce | CRAM        |
|  319 | PBBKBT           | 0DBNCT_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/f48bb9ca-3684-4c09-b7aa-d6398866eecf   | IGM_PBBKBT-0DBNCT_20220902a_tumor.aln.cram.crai  | b082c7099af8df3f0c8c0ba00afaeb64 | CRAI        |
|  320 | PBBKBT           | 0DBN9M_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d5f94680-5ee0-4c52-98d3-401d452fc932   | IGM_PBBKBT-0DBN9M_20220801_merged_bam.bai        | 222a46a953575c5c2679080fa3547fde | BAI         |
|  321 | PBBKBT           | 0DBN9M_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f046da2d-c37d-4768-a2b3-4419eac16335   | IGM_PBBKBT-0DBN9M_20220801_bam.bai               | fc644e1b0d8b2439b84bc7599275cdaf | BAI         |
|  322 | PBBKBT           | 0DBN9M_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/42767a4b-6fd1-4963-afcb-f9fa4bb0598f   | IGM_PBBKBT-0DBN9M_20220801_bam.bam               | a697381e122f1cbe52a7154461b2236f | BAM         |
|  323 | PBBKBT           | 0DBN9M_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f943c655-56c2-4aca-b85d-e0f369e0ba12   | IGM_PBBKBT-0DBN9M_20220801_merged_bam.bam        | 3905386aa6a23787875d619b38620ca9 | BAM         |
|  324 | PBBMLH           | 0DCFVM_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/78c00535-9212-4bf3-9fea-f2f4fe02c94c   | IGM_PBBMLH-0DCFVM_20220830_bam.bai               | 1f3f7a258d5895457c4457440d26c7c6 | BAI         |
|  325 | PBBMLH           | 0DCFVM_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/5d0684f4-8d38-4467-8160-787dbac13110   | IGM_PBBMLH-0DCFVM_20220830_merged_bam.bai        | eee9755d41f6fdec4dc3f50f58827312 | BAI         |
|  326 | PBBMLH           | 0DCFVM_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c3f52334-479b-40c3-9e17-4fc28485abe6   | IGM_PBBMLH-0DCFVM_20220830_bam.bam               | 1e936d341e5d3888e68fdca907d4f0a3 | BAM         |
|  327 | PBBMLH           | 0DCFVM_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ae860346-ff7f-4168-b138-182a45a39d09   | IGM_PBBMLH-0DCFVM_20220830_merged_bam.bam        | e74333587bb811af43110d98da0067bf | BAM         |
|  328 | PBBMLH           | 0DCFVX_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/4ae17d47-5867-4791-a27f-ea186db4f27b   | IGM_PBBMLH-0DCFVX_20220902a_tumor.aln.cram       | 22e55000e074324e8588b7f433c5a16e | CRAM        |
|  329 | PBBMLH           | 0DCFVX_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/7fcd864a-1383-4506-8a37-610450289f4a   | IGM_PBBMLH-0DCFVX_20220902a_tumor.aln.cram.crai  | 392920016ff05254e136c43a1404511a | CRAI        |
|  330 | PBBMLH           | 0DCFVR_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/242dd776-9378-4cbb-8574-b4c277e5932e   | IGM_PBBMLH-0DCFVR_20220902a_normal.aln.cram.crai | 962e9f66cdc868bf3226287b7bbc4186 | CRAI        |
|  331 | PBBMLH           | 0DCFVR_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/de8dc6fd-597d-4820-a129-a9e9b39724ed   | IGM_PBBMLH-0DCFVR_20220902a_normal.aln.cram      | f77c3d7207154cc39435cad522ac78ba | CRAM        |
|  332 | PBBIUN           | 0D9HZA_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/1622a4c9-edfb-4d8a-9c6f-831c9e354fc7   | IGM_PBBIUN-0D9HZA_20220902a_tumor.aln.cram       | 7071db5d954df480c4663d0d64ca7aef | CRAM        |
|  333 | PBBIUN           | 0D9HZA_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/14a265bd-ec22-4130-888f-589d20c85fd9   | IGM_PBBIUN-0D9HZA_20220902a_tumor.aln.cram.crai  | e5d7ae9e486a21dcc60ba1ba1581e76d | CRAI        |
|  334 | PBBIUN           | 0D9HYH_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/08104287-112c-4e56-a27e-fe4f8b363f78   | IGM_PBBIUN-0D9HYH_20220919_merged_bam.bam        | 5ae1f988be555db051edcb362ff23cd0 | BAM         |
|  335 | PBBIUN           | 0D9HYH_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/269b4537-ff15-4e9c-9960-679d40787130   | IGM_PBBIUN-0D9HYH_20220919_bam.bam               | 7336013f3c2e2498cdda4db9b7c8af9e | BAM         |
|  336 | PBBIUN           | 0D9HYH_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/eab842de-027f-444f-ad76-97299b4b2879   | IGM_PBBIUN-0D9HYH_20220919_bam.bai               | 01c8d41f2d858de6cdd368e43a26d06c | BAI         |
|  337 | PBBIUN           | 0D9HYH_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/88485f6e-d677-407b-8eb5-a92bf8c3b3d5   | IGM_PBBIUN-0D9HYH_20220919_merged_bam.bai        | 059027ee2bccd8ae971edc53948c4126 | BAI         |
|  338 | PBBIUN           | 0D9HZ2_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/65e28969-07b5-4793-a1e3-98cb6ceda1c1   | IGM_PBBIUN-0D9HZ2_20220902a_normal.aln.cram.crai | 5882d978fe36165c4ffc92e403e12216 | CRAI        |
|  339 | PBBIUN           | 0D9HZ2_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/e480a16c-6093-48ce-8fba-5d41e4d29a42   | IGM_PBBIUN-0D9HZ2_20220902a_normal.aln.cram      | 451d69209d421bb2dc5f620287774278 | CRAM        |
|  340 | PBBSGU           | 0DFJJ6_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/45042033-73bd-4c6b-9f33-4cce3bb14598   | IGM_PBBSGU-0DFJJ6_20221128_normal.aln.cram.crai  | 5f44398dec5b40e7d088532c4a4fa172 | CRAI        |
|  341 | PBBSGU           | 0DFJJ6_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/e57bee2f-19e2-46b4-b6a4-966e0d8fabc3   | IGM_PBBSGU-0DFJJ6_20221128_normal.aln.cram       | 2bfb547da052918fc5d60013c193a2be | CRAM        |
|  342 | PBBSGU           | 0DFJK9_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/44c219eb-03d0-4d11-95dc-aa8cca215889   | IGM_PBBSGU-0DFJK9_20221129_merged_bam.bam        | e191da226877cbc4ea15cfc5bf344c07 | BAM         |
|  343 | PBBSGU           | 0DFJK9_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/047d5577-4b41-4949-b9f9-33cdd78816e7   | IGM_PBBSGU-0DFJK9_20221129_bam.bam               | eb1a5f8470db6cb4caa12e888398e267 | BAM         |
|  344 | PBBSGU           | 0DFJK9_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/da114239-2730-48e7-bdef-95aef1a6d119   | IGM_PBBSGU-0DFJK9_20221129_bam.bai               | 3a474f1d1d9af9f2a274b9e096782202 | BAI         |
|  345 | PBBSGU           | 0DFJK9_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d08992dd-d2ea-4d9e-b45c-fbc4e5268908   | IGM_PBBSGU-0DFJK9_20221129_merged_bam.bai        | 780e1da7028a5ba44861c9539511fdce | BAI         |
|  346 | PBBSGU           | 0DFJKU_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/75575a1f-b341-4f66-aa3f-7dce84137959   | IGM_PBBSGU-0DFJKU_20221128_tumor.aln.cram.crai   | 86f112a8d5aaf0c1f95dc7d32349b2ba | CRAI        |
|  347 | PBBSGU           | 0DFJKU_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/bfc76787-b217-41a1-9c88-3bf11f3fe2a6   | IGM_PBBSGU-0DFJKU_20221128_tumor.aln.cram        | bb7f424e146d645007e1b061838271b3 | CRAM        |
|  348 | PBBNZL           | 0DEFHQ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/1dc96d3e-558c-4335-8e0c-df2d473b5c6c   | IGM_PBBNZL-0DEFHQ_20221111_normal.aln.cram.crai  | a2df6d4668edbae9b52232357747cafd | CRAI        |
|  349 | PBBNZL           | 0DEFHQ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/3c8eb0fc-d23e-4419-8730-49d027ddb5a2   | IGM_PBBNZL-0DEFHQ_20221111_normal.aln.cram       | 8ae298b1c598d3bf09ee9b4501758cb8 | CRAM        |
|  350 | PBBNZL           | 0DEFJY_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/5fb6de9a-f127-44de-8a6e-a1f6e745a9a8   | IGM_PBBNZL-0DEFJY_20221111_tumor.aln.cram.crai   | bc377c54240cbef683db9034f308b6c9 | CRAI        |
|  351 | PBBNZL           | 0DEFJY_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/82101127-9684-4f50-ae45-3b6367372f5c   | IGM_PBBNZL-0DEFJY_20221111_tumor.aln.cram        | aee8b012ec39c41dcbc9c37ebc7cccaa | CRAM        |
|  352 | PBBNZL           | 0DEFJD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a0cb0977-952a-4984-83c2-b822cd9bd34a   | IGM_PBBNZL-0DEFJD_20221103_bam.bam               | d4ae3ef9d0792c00849a10b01cec4f8c | BAM         |
|  353 | PBBNZL           | 0DEFJD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/1cd8d83d-4a5e-4849-818c-58581829e8ac   | IGM_PBBNZL-0DEFJD_20221103_merged_bam.bam        | 51f609e053499efd4e376f0322f18ee0 | BAM         |
|  354 | PBBNZL           | 0DEFJD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ab313e12-d43d-4588-97af-e48822db12f3   | IGM_PBBNZL-0DEFJD_20221103_merged_bam.bai        | 1e1e7fceabce2bfe761bdf675b8925f1 | BAI         |
|  355 | PBBNZL           | 0DEFJD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b6f10cd7-de44-47c5-b0d4-7adc03a6f57f   | IGM_PBBNZL-0DEFJD_20221103_bam.bai               | ecb838ee6ca6f0f972ab452f7d774443 | BAI         |
|  356 | PBBSGI           | 0DFJJ3_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/2534bb00-7dc7-4e2c-a203-9563fc8b58f5   | IGM_PBBSGI-0DFJJ3_20221209_normal.aln.cram.crai  | 4f1771a747eaf70d05dfe87429e156c4 | CRAI        |
|  357 | PBBSGI           | 0DFJJ3_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/ee2c09d6-cf8b-44f8-95d2-c4e6f3778393   | IGM_PBBSGI-0DFJJ3_20221209_normal.aln.cram       | c3e7a7390e93cd84686fd352d55e1cb0 | CRAM        |
|  358 | PBBSGI           | 0DFJKF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e0555ff9-7f4e-4b11-98a6-c8dc7da3a269   | IGM_PBBSGI-0DFJKF_20221129_bam.bai               | 54e472eeea080aeb0bb0befb56ffb4bd | BAI         |
|  359 | PBBSGI           | 0DFJKF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7546b64c-da51-4366-968e-7950df89977f   | IGM_PBBSGI-0DFJKF_20221129_merged_bam.bam        | aa69d5eb99b94f61be339699650ad878 | BAM         |
|  360 | PBBSGI           | 0DFJKF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f61bd606-bb79-4c4e-b407-a3484a40a3f4   | IGM_PBBSGI-0DFJKF_20221129_merged_bam.bai        | 07749b34e4f2a7aad33d1eaf114ed72a | BAI         |
|  361 | PBBSGI           | 0DFJKF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e5d023e8-6d8a-4d22-a4b2-cf7f86d9af1f   | IGM_PBBSGI-0DFJKF_20221129_bam.bam               | c394ba921b541d0d016df370bae4beaf | BAM         |
|  362 | PBBSGI           | 0DFJMX_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/cc758a87-838e-4478-b4c7-af01d5c3bf12   | IGM_PBBSGI-0DFJMX_20221209_tumor.aln.cram        | c74d51f660665d939d27a83cc0dccc81 | CRAM        |
|  363 | PBBSGI           | 0DFJMX_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/d31700a3-6f2f-4136-8945-69f2b63877cb   | IGM_PBBSGI-0DFJMX_20221209_tumor.aln.cram.crai   | ffe2594806ad174cce8a9d4a24df06e5 | CRAI        |
|  364 | PBBMZU           | 0DE7CR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b4c4a729-4a37-43ab-b947-10ff7c8645b9   | IGM_PBBMZU-0DE7CR_20221024_bam.bam               | b273a0c9b4266508b329e63a6994b438 | BAM         |
|  365 | PBBMZU           | 0DE7CR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0ef90856-7522-4159-8816-fcfb48c7aa44   | IGM_PBBMZU-0DE7CR_20221024_merged_bam.bai        | 721774ba231091eb1cc2945b88109024 | BAI         |
|  366 | PBBMZU           | 0DE7CR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2a673b10-2806-4bb8-8132-c0cd868b198d   | IGM_PBBMZU-0DE7CR_20221024_merged_bam.bam        | 3c0709c12f850365f7dbc1d7130878a1 | BAM         |
|  367 | PBBMZU           | 0DE7CR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d957b26c-37f7-4a21-9209-0ab509cab406   | IGM_PBBMZU-0DE7CR_20221024_bam.bai               | 9de9904ff2767572e15d0d21b0221255 | BAI         |
|  368 | PBBWKJ           | 0DJ4CA_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/99467dc5-f132-40ea-8fa7-80b6aef9f0e6   | IGM_PBBWKJ-0DJ4CA_20230311_normal.aln.cram.crai  | e0136d350ed780e45c58e1a31b5de08b | CRAI        |
|  369 | PBBWKJ           | 0DJ4CA_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/9ec78e78-92ff-4455-883a-7ba60157673e   | IGM_PBBWKJ-0DJ4CA_20230311_normal.aln.cram       | 077385acf5b104960d27419ee59f964b | CRAM        |
|  370 | PBBWKJ           | 0DJ4BX_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/80dc2f26-dac1-4a7a-b6bf-dbc4627a62a0   | IGM_PBBWKJ-0DJ4BX_20230303_bam.bai               | 527eab8ebed7f701ef2c8da4cddbdf06 | BAI         |
|  371 | PBBWKJ           | 0DJ4BX_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4c8efc94-952b-4797-ba84-afc81bf58e97   | IGM_PBBWKJ-0DJ4BX_20230303_bam.bam               | ea76b6898e552840198a697ac361c445 | BAM         |
|  372 | PBBWKJ           | 0DJ4BX_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8f43c41f-6e39-493b-bdb0-f5279e01b4a6   | IGM_PBBWKJ-0DJ4BX_20230303_merged_bam.bai        | 653acd558b693427a996f65627dc0810 | BAI         |
|  373 | PBBWKJ           | 0DJ4BX_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/58c9f588-5206-47b9-821d-eb6090f742c8   | IGM_PBBWKJ-0DJ4BX_20230303_merged_bam.bam        | ed37972a3a97d5b294781bef4f640b04 | BAM         |
|  374 | PBBWKJ           | 0DJ4C0_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/415bff6f-9a7c-4339-a410-ad4c59d7c3fb   | IGM_PBBWKJ-0DJ4C0_20230311_tumor.aln.cram        | 52a8e1413e64c19dd6f16c59c9007d59 | CRAM        |
|  375 | PBBWKJ           | 0DJ4C0_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/20a7b46a-d910-45b6-83ac-dd132de721b7   | IGM_PBBWKJ-0DJ4C0_20230311_tumor.aln.cram.crai   | 333a7872cdb302e9fe5470cba6db17a5 | CRAI        |
|  376 | PBBKZT           | 0DBN9N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/aef8ed5e-dbaa-4652-91af-0b3202b4291b   | IGM_PBBKZT-0DBN9N_20220806_merged_bam.bam        | 4ff35c93dc194e05fbbe3a6b2887f8b6 | BAM         |
|  377 | PBBKZT           | 0DBN9N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ab36b71f-b2f7-4fe8-8a04-f09df6914ed7   | IGM_PBBKZT-0DBN9N_20220806_bam.bai               | 9943a3c893ac44856b271ada2b0a0e45 | BAI         |
|  378 | PBBKZT           | 0DBN9N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a18516b0-86f4-446a-980c-7325cbd8f5bb   | IGM_PBBKZT-0DBN9N_20220806_merged_bam.bai        | 0cd8f67896dce3323e99d1888b189418 | BAI         |
|  379 | PBBKZT           | 0DBN9N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0f40a64c-8e6f-4f6e-90e6-9584c19d20d2   | IGM_PBBKZT-0DBN9N_20220806_bam.bam               | 647d8cf5103adebbc39bb8e2139bb39c | BAM         |
|  380 | PBBKZT           | 0DBNBE_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/ec6536ed-4e20-4554-868e-ad46e7b9e7b8   | IGM_PBBKZT-0DBNBE_20220902a_tumor.aln.cram       | bc75280b80c978426ef6206e7b2282f9 | CRAM        |
|  381 | PBBKZT           | 0DBNBE_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/e653efcb-896f-4fa5-8aea-ee991ece9b0f   | IGM_PBBKZT-0DBNBE_20220902a_tumor.aln.cram.crai  | 17b455d62306a26d3a8e2e293251a266 | CRAI        |
|  382 | PBBKZT           | 0DBNBM_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/88b5a7a3-7977-4ee2-98e6-5b10b4e23dac   | IGM_PBBKZT-0DBNBM_20220902a_normal.aln.cram.crai | 3f602606eb978f67352193653d24b0c7 | CRAI        |
|  383 | PBBKZT           | 0DBNBM_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/ee07be77-7c71-488d-a64c-c51e921e7163   | IGM_PBBKZT-0DBNBM_20220902a_normal.aln.cram      | 9b662d44d31267e157bb6ffe13c5b88d | CRAM        |
|  384 | PBBPLT           | 0DE7FM_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/8d0bd84a-32af-4a9c-9c6b-4d5accbb9bc1   | IGM_PBBPLT-0DE7FM_20221109_tumor.aln.cram.crai   | a50d835dd4be8f602d2579fcc3f2ed59 | CRAI        |
|  385 | PBBPLT           | 0DE7FM_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/d5222730-8f4c-4909-acbb-1f54af8614c9   | IGM_PBBPLT-0DE7FM_20221109_tumor.aln.cram        | cb4f8842d8925795bc93f0fd1f93d0f8 | CRAM        |
|  386 | PBBPLT           | 0DE7CV_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/23bd5c6b-3e3c-47e2-9649-06e1a8108bf0   | IGM_PBBPLT-0DE7CV_20221103_merged_bam.bam        | 557dec6036c8bea818bd92e7030b896a | BAM         |
|  387 | PBBPLT           | 0DE7CV_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e4bb9bf8-a198-4c0b-b982-584384aa73ea   | IGM_PBBPLT-0DE7CV_20221103_merged_bam.bai        | e46fbc465d9ebf6eee6c3aa65d6fb04b | BAI         |
|  388 | PBBPLT           | 0DE7CV_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e0afebfc-3f70-489c-b779-0aa9e5327de6   | IGM_PBBPLT-0DE7CV_20221103_bam.bam               | a30c140e9f9c834ee0a814ee6ed9180e | BAM         |
|  389 | PBBPLT           | 0DE7CV_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/87c93397-cbec-4652-bf73-6db9e8764dc1   | IGM_PBBPLT-0DE7CV_20221103_bam.bai               | b160729264fd1e7298d0663865c29437 | BAI         |
|  390 | PBBPLT           | 0DE7E4_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/93ff2566-f487-4809-8dfd-53c5ee52cd6b   | IGM_PBBPLT-0DE7E4_20221109_normal.aln.cram       | ed8a83996eddebc506a2dea0ec59a408 | CRAM        |
|  391 | PBBPLT           | 0DE7E4_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/35c2f58a-b76c-45fd-ba05-47eebb2ee5a5   | IGM_PBBPLT-0DE7E4_20221109_normal.aln.cram.crai  | 8dfa9fae46770fc8997c955a6b48bb66 | CRAI        |
|  392 | PBBMCU           | 0DDIKG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/94283ca6-5993-4250-9249-5cd0ce5100ee   | IGM_PBBMCU-0DDIKG_20221003_bam.bam               | af734e03d4e4191e6796dc2a98552485 | BAM         |
|  393 | PBBMCU           | 0DDIKG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/19455c2a-7f83-4d52-8466-52e7d103fec4   | IGM_PBBMCU-0DDIKG_20221003_merged_bam.bai        | add62ea6c2b98d80b126de672fc54813 | BAI         |
|  394 | PBBMCU           | 0DDIKG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0c1200b8-8ec4-407c-82f7-b60cb759e4f6   | IGM_PBBMCU-0DDIKG_20221003_bam.bai               | c06146e95973e92a9874a05db97178f1 | BAI         |
|  395 | PBBMCU           | 0DDIKG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/401e3219-636e-4296-819c-877ff354230c   | IGM_PBBMCU-0DDIKG_20221003_merged_bam.bam        | 33da738907727badd8ed151a4ccf99cd | BAM         |
|  396 | PBBMCU           | 0DDIKA_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/6846d7c4-a4db-4d9b-bcca-8cae6aee9f05   | IGM_PBBMCU-0DDIKA_20221004_normal.aln.cram       | 904530c2283c0aaa4054d892b336b646 | CRAM        |
|  397 | PBBMCU           | 0DDIKA_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/e69c0052-56e2-4548-9207-ac6f823f4ef1   | IGM_PBBMCU-0DDIKA_20221004_normal.aln.cram.crai  | d1d44c1b49dc59ab674544f90c6f1814 | CRAI        |
|  398 | PBBMCU           | 0DDIKL_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/5bd9c1a9-8af4-4713-87b6-4978ded8b3e3   | IGM_PBBMCU-0DDIKL_20221004_tumor.aln.cram.crai   | 37215745032703b7201abcf14b321af5 | CRAI        |
|  399 | PBBMCU           | 0DDIKL_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/d779bc95-1339-45b2-be39-5bb0be663580   | IGM_PBBMCU-0DDIKL_20221004_tumor.aln.cram        | 991b88ab7ffec045a8683bc656c74e35 | CRAM        |
|  400 | PBCHYK           | 0DSS54_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1992 | IGM_PBCHYK-0DSS54_20231219_merged_bam.bam        | 3f96a3f10198affc6fa0d6cd8ea47c0a | BAM         |
|  401 | PBCHYK           | 0DSS54_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1991 | IGM_PBCHYK-0DSS54_20231219_bam.bai               | 12664054b76cb826c05c677ed4db9c3a | BAI         |
|  402 | PBCHYK           | 0DSS54_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1990 | IGM_PBCHYK-0DSS54_20231219_bam.bam               | 4b465e9ae903f3f265390bf35160d094 | BAM         |
|  403 | PBCHYK           | 0DSS54_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1993 | IGM_PBCHYK-0DSS54_20231219_merged_bam.bai        | 3d80521e635cf1a3a4473adbc7f07871 | BAI         |
|  404 | PBCHYK           | 0DSS5P_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb2011 | IGM_PBCHYK-0DSS5P_20231221_tumor.aln.cram.crai   | e7fef545dd74bff2b773a58e47bdfeda | CRAI        |
|  405 | PBCHYK           | 0DSS5P_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb2010 | IGM_PBCHYK-0DSS5P_20231221_tumor.aln.cram        | b9786f8efcb3056da2aba882c57bf8dd | CRAM        |
|  406 | PBCHYK           | 0DSS69_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb2008 | IGM_PBCHYK-0DSS69_20231221_normal.aln.cram       | 1e8429dbf48be296dfb41e0b57c29774 | CRAM        |
|  407 | PBCHYK           | 0DSS69_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb2009 | IGM_PBCHYK-0DSS69_20231221_normal.aln.cram.crai  | ba888337d5c855516a8c9db19a0012b4 | CRAI        |
|  408 | PBCCCU           | 0DOBSQ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/008f9391-a153-49b8-ab1c-199973084ab0   | IGM_PBCCCU-0DOBSQ_20230804_normal.aln.cram.crai  | a59de9a97127deba4a6292e04b8250c1 | CRAI        |
|  409 | PBCCCU           | 0DOBSQ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/378b85f0-ccc3-4c1d-a04f-faee12f9289c   | IGM_PBCCCU-0DOBSQ_20230804_normal.aln.cram       | a1e96158a7e85a6fe00ba3985122d8d6 | CRAM        |
|  410 | PBCCCU           | 0DOBTG_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/0365c740-3c15-4988-82cd-b80fe0efb0c2   | IGM_PBCCCU-0DOBTG_20230804_tumor.aln.cram.crai   | 2349a7b2b3d43dac9a7200067bc7725b | CRAI        |
|  411 | PBCCCU           | 0DOBTG_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/4eed3e22-36b5-4852-bd8d-9c37ad8d3676   | IGM_PBCCCU-0DOBTG_20230804_tumor.aln.cram        | 2ddd6678238c66b6199493d1be48400c | CRAM        |
|  412 | PBCCCU           | 0DOBT4_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2249d5df-e09c-416a-95c4-206478ccc121   | IGM_PBCCCU-0DOBT4_20230727_bam.bai               | 7ba4e1da2e13027273cbc805b3eb20a1 | BAI         |
|  413 | PBCCCU           | 0DOBT4_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/631ab754-7f5e-4861-9798-bf93d7ff6e14   | IGM_PBCCCU-0DOBT4_20230727_bam.bam               | ccb883919c231a8947583a6099cbb3e9 | BAM         |
|  414 | PBCCCU           | 0DOBT4_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/382f4248-82a3-4a41-923e-94c6ca96bb6f   | IGM_PBCCCU-0DOBT4_20230727_merged_bam.bam        | a2439dc07d471d415cbeb0c44f479d3c | BAM         |
|  415 | PBCCCU           | 0DOBT4_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4ff49bdb-682d-48fc-a9e9-9df9bba3ebe7   | IGM_PBCCCU-0DOBT4_20230727_merged_bam.bai        | dc86fce0e57455254a9f024bcddd7d97 | BAI         |
|  416 | PBBKNA           | 0DAWAK_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/8dfe04f1-3d87-439d-8bb6-0918049b3c4d   | IGM_PBBKNA-0DAWAK_20220902a_tumor.aln.cram       | 3278481d1aa5bd57fbade15fd3a94f29 | CRAM        |
|  417 | PBBKNA           | 0DAWAK_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/fe9d055c-54de-4f53-b243-bac68d2a2c8c   | IGM_PBBKNA-0DAWAK_20220902a_tumor.aln.cram.crai  | f4c2dcb901cc6fa33cb67cc0dc75fc37 | CRAI        |
|  418 | PBBKNA           | 0DAWAN_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/044fe6a2-80a5-4ff9-8e08-3ae797e930bc   | IGM_PBBKNA-0DAWAN_20220707_bam.bam               | ace91e3bb35a5b0b7ff5152ca568cb1d | BAM         |
|  419 | PBBKNA           | 0DAWAN_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a5ebde79-f16a-4428-889e-d596098717ea   | IGM_PBBKNA-0DAWAN_20220707_bam.bai               | 7a92265e5e51c6661c5750f6028c7580 | BAI         |
|  420 | PBBKNA           | 0DAWAN_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/9526424a-3937-4d5b-a0c0-82d985dc11f7   | IGM_PBBKNA-0DAWAN_20220707_merged_bam.bam        | b587150b5790027fc63e33227fc2fe08 | BAM         |
|  421 | PBBKNA           | 0DAWAN_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/248986d8-28f6-43ed-85f0-b7d9e0d25af6   | IGM_PBBKNA-0DAWAN_20220707_merged_bam.bai        | 3dbceb04804cc64380d783e647603ec5 | BAI         |
|  422 | PBBKNA           | 0DAWAC_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/28fb50fb-fb6a-4e5a-9fc3-44abf8452c8f   | IGM_PBBKNA-0DAWAC_20220902a_normal.aln.cram.crai | 07d8d4c4d818ca4bdd33abf5acd201ea | CRAI        |
|  423 | PBBKNA           | 0DAWAC_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/33443ec0-e7cb-481b-b206-6619f45519d9   | IGM_PBBKNA-0DAWAC_20220902a_normal.aln.cram      | 543ef29b5e10be0f4bfb46a7197a0c41 | CRAM        |
|  424 | PBCHNH           | 0DSG1W_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1195 | IGM_PBCHNH-0DSG1W_20231206_merged_bam.bai        | 39db9b3b2b5462de9b993b0daddf3e18 | BAI         |
|  425 | PBCHNH           | 0DSG1W_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1194 | IGM_PBCHNH-0DSG1W_20231206_merged_bam.bam        | b4b490372625a674417e3024ce16fa96 | BAM         |
|  426 | PBCHNH           | 0DSG1W_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1193 | IGM_PBCHNH-0DSG1W_20231206_bam.bai               | eaa183f5b7bc26c3da15e3ccc4308bab | BAI         |
|  427 | PBCHNH           | 0DSG1W_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1192 | IGM_PBCHNH-0DSG1W_20231206_bam.bam               | 7c6df3184254c123fc371237e6796fdf | BAM         |
|  428 | PBCHNH           | 0DSG22_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1210 | IGM_PBCHNH-0DSG22_20231208_tumor.aln.cram        | 7315ae6deb99ef61a4bea52f7bdd29a5 | CRAM        |
|  429 | PBCHNH           | 0DSG22_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1211 | IGM_PBCHNH-0DSG22_20231208_tumor.aln.cram.crai   | a3d023b08a8fbe7d2bddf5310f5fdbaa | CRAI        |
|  430 | PBCHNH           | 0DSG2J_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1208 | IGM_PBCHNH-0DSG2J_20231208_normal.aln.cram       | 14c32d2e1f3ea30dfb8992ac23ae8dc8 | CRAM        |
|  431 | PBCHNH           | 0DSG2J_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1209 | IGM_PBCHNH-0DSG2J_20231208_normal.aln.cram.crai  | 26e3c815e568c409668d47f47b60351e | CRAI        |
|  432 | PBBSDG           | 0DFF7A_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a9213f81-eb6b-492c-bcb7-0734bb46fd26   | IGM_PBBSDG-0DFF7A_20221130_bam.bam               | 0abed3063c48f5d8e5da7a65a107d48d | BAM         |
|  433 | PBBSDG           | 0DFF7A_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b038d798-180d-4f57-a0fa-fd537c3edea1   | IGM_PBBSDG-0DFF7A_20221130_merged_bam.bai        | 0d0c6b457579165482b2a3342fee3035 | BAI         |
|  434 | PBBSDG           | 0DFF7A_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a85af858-1b7e-4a42-9f8c-24f586a3a471   | IGM_PBBSDG-0DFF7A_20221130_merged_bam.bam        | bfb26def509cc29d8f8fdcae330535ce | BAM         |
|  435 | PBBSDG           | 0DFF7A_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ae9a0da0-f603-4dd4-b6e1-3f081784e431   | IGM_PBBSDG-0DFF7A_20221130_bam.bai               | 7df6084e4071ef9d32edb4d7a498b6ec | BAI         |
|  436 | PBBSDG           | 0DFF74_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/e26b51c0-8de6-43d0-a83e-20a9e26a0638   | IGM_PBBSDG-0DFF74_20221124_tumor.aln.cram        | 5a5548d2612b0c6af541319a74d77270 | CRAM        |
|  437 | PBBSDG           | 0DFF74_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/418e3ce0-1e6f-4a85-9885-eeb2aa5082ec   | IGM_PBBSDG-0DFF74_20221124_tumor.aln.cram.crai   | 94b91d3c2ddf1d59ad6431043dcad266 | CRAI        |
|  438 | PBBSDG           | 0DFF71_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/8661bf77-1bc6-4cbf-8cec-dfba3e4ac0fe   | IGM_PBBSDG-0DFF71_20221124_normal.aln.cram.crai  | 8910066cb14fb7cf4bb8c44e837537e9 | CRAI        |
|  439 | PBBSDG           | 0DFF71_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/d9a3e703-3741-4a3b-a655-8195bf675c14   | IGM_PBBSDG-0DFF71_20221124_normal.aln.cram       | 12086457905ddcb294db4199e56586fc | CRAM        |
|  440 | PBCHSW           | 0DS544_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1436 | IGM_PBCHSW-0DS544_20231127_normal.aln.cram       | 6d348ae839033d66d083d538678acb97 | CRAM        |
|  441 | PBCHSW           | 0DS544_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1437 | IGM_PBCHSW-0DS544_20231127_normal.aln.cram.crai  | bb78db7eec03907bd750f725116bf043 | CRAI        |
|  442 | PBCHSW           | 0DS532_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1438 | IGM_PBCHSW-0DS532_20231127_tumor.aln.cram        | 06631b8c2e645e277c3e6130cb0428a5 | CRAM        |
|  443 | PBCHSW           | 0DS532_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1439 | IGM_PBCHSW-0DS532_20231127_tumor.aln.cram.crai   | 77b481f641c324c4b43058f58c3152a1 | CRAI        |
|  444 | PBCHMT           | 0DSEZO_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1068 | IGM_PBCHMT-0DSEZO_20231206_tumor.aln.cram        | 9e08c694c70910cbae5a21c520440544 | CRAM        |
|  445 | PBCHMT           | 0DSEZO_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1069 | IGM_PBCHMT-0DSEZO_20231206_tumor.aln.cram.crai   | 3cda3531e84191a92e3a89d4f6f94b5b | CRAI        |
|  446 | PBCHMT           | 0DSEYU_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1067 | IGM_PBCHMT-0DSEYU_20231206_normal.aln.cram.crai  | 453da72bd9f030bd21b0738d4feedcb6 | CRAI        |
|  447 | PBCHMT           | 0DSEYU_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1066 | IGM_PBCHMT-0DSEYU_20231206_normal.aln.cram       | cbebf7fcd98348b132de716be4c9730e | CRAM        |
|  448 | PBCHMT           | 0DSEZ7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1051 | IGM_PBCHMT-0DSEZ7_20231206_bam.bai               | 787044888d0db0293c08071e773a2729 | BAI         |
|  449 | PBCHMT           | 0DSEZ7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1053 | IGM_PBCHMT-0DSEZ7_20231206_merged_bam.bai        | 8cd9ca13c3cd70c8a05ce3f39bbdbb10 | BAI         |
|  450 | PBCHMT           | 0DSEZ7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1050 | IGM_PBCHMT-0DSEZ7_20231206_bam.bam               | cc387e3673c253dfeb9e2d2db84d6da7 | BAM         |
|  451 | PBCHMT           | 0DSEZ7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1052 | IGM_PBCHMT-0DSEZ7_20231206_merged_bam.bam        | 19007381ac576174bee9feedbf9b487d | BAM         |
|  452 | PBBWSY           | 0DJKKG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2cba0c4f-f61f-4a28-9d71-2fcc95f07690   | IGM_PBBWSY-0DJKKG_20230323_merged_bam.bai        | 6d11b50af3c30f4ab3a440e0068c092f | BAI         |
|  453 | PBBWSY           | 0DJKKG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a3622cee-b7c6-4883-a020-6741fe18a72f   | IGM_PBBWSY-0DJKKG_20230323_merged_bam.bam        | cea491538eb0f8b0c6ebd62515f49b39 | BAM         |
|  454 | PBBWSY           | 0DJKKG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/1d636a32-2f34-43ff-96a4-2954d1702ca5   | IGM_PBBWSY-0DJKKG_20230323_bam.bai               | d66a3eccd2743a10a45e51f94fc2fe52 | BAI         |
|  455 | PBBWSY           | 0DJKKG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/03feba99-8fc4-4a2f-b071-30b6adb24522   | IGM_PBBWSY-0DJKKG_20230323_bam.bam               | 8270edaa70456536f31feed5976264bc | BAM         |
|  456 | PBBWXH           | 0DJG72_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/67733198-5316-42f2-bf11-0e32bb575941   | IGM_PBBWXH-0DJG72_20230320_normal.aln.cram.crai  | 9aee0f5cae630b6d1ed6b90ee74b7cac | CRAI        |
|  457 | PBBWXH           | 0DJG72_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/5bdf7eee-8de4-4aab-bf51-b63630d56527   | IGM_PBBWXH-0DJG72_20230320_normal.aln.cram       | c6dfec64c2ccbd2d684f738c80bc6ccd | CRAM        |
|  458 | PBBWXH           | 0DJGDM_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a94c8775-d502-4727-b693-024c0e968756   | IGM_PBBWXH-0DJGDM_20230313_merged_bam.bai        | f21489c758648e1c90ca881d8692fa23 | BAI         |
|  459 | PBBWXH           | 0DJGDM_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/96e7cefa-3e8f-44de-b9d8-eb830c4b1499   | IGM_PBBWXH-0DJGDM_20230313_bam.bai               | ec3dea81228d4c259b21d35d21a3b15a | BAI         |
|  460 | PBBWXH           | 0DJGDM_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b844f6a9-cdfb-447c-b06b-0d6bccbb77cf   | IGM_PBBWXH-0DJGDM_20230313_merged_bam.bam        | 7b5817623a0f66bc9a3c4f358d56c96d | BAM         |
|  461 | PBBWXH           | 0DJGDM_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/33711777-d59a-491a-9ee8-5d457b0acd79   | IGM_PBBWXH-0DJGDM_20230313_bam.bam               | 15e92c1dae05ef19405becedf9d0f257 | BAM         |
|  462 | PBBWXH           | 0DJGE5_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/90fca4e5-4d15-414c-82b3-f5c061936da6   | IGM_PBBWXH-0DJGE5_20230320_tumor.aln.cram        | 3ab4d3e8f70aa7b030d5b97ae0e4f0be | CRAM        |
|  463 | PBBWXH           | 0DJGE5_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/cbeaf647-a3a0-476d-81aa-1c082938c279   | IGM_PBBWXH-0DJGE5_20230320_tumor.aln.cram.crai   | 8df5698a007ba04ca00cd31646e173a4 | CRAI        |
|  464 | PBBLXG           | 0DCS55_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/bc723d8d-6be0-4b90-a9a1-fbbe18841e67   | IGM_PBBLXG-0DCS55_20220906_merged_bam.bai        | 88e037c0857d3006b6878ddab8e3f06a | BAI         |
|  465 | PBBLXG           | 0DCS55_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d2d51c22-3469-401d-bfb8-ebc30966ec11   | IGM_PBBLXG-0DCS55_20220906_bam.bai               | e6fc56c496a838471f552f20a3ffefe9 | BAI         |
|  466 | PBBLXG           | 0DCS55_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3eddf6f2-0419-40bd-9ec8-cb40267a7ddd   | IGM_PBBLXG-0DCS55_20220906_merged_bam.bam        | ee1129f937143cfda27e1ae02577515d | BAM         |
|  467 | PBBLXG           | 0DCS55_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e1f28964-fcfb-4deb-92ed-f90fc4196dbe   | IGM_PBBLXG-0DCS55_20220906_bam.bam               | df98a14a0a0c314a6cde4acbaa3ef52c | BAM         |
|  468 | PBCHTI           | 0DSA08_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1558 | IGM_PBCHTI-0DSA08_20231201_tumor.aln.cram.crai   | 68f7736369e216a0a66b9cebc77e97a8 | CRAI        |
|  469 | PBCHTI           | 0DSA08_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1557 | IGM_PBCHTI-0DSA08_20231201_tumor.aln.cram        | e687dd3af594f7716384726e7bc2279c | CRAM        |
|  470 | PBCHTI           | 0DSA1F_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1556 | IGM_PBCHTI-0DSA1F_20231201_normal.aln.cram.crai  | a7718786efe7c9737feab4dc352bdf2f | CRAI        |
|  471 | PBCHTI           | 0DSA1F_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1555 | IGM_PBCHTI-0DSA1F_20231201_normal.aln.cram       | 392151e881d5cd46a594758c7165df17 | CRAM        |
|  472 | PBCHTI           | 0DS9ZY_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1539 | IGM_PBCHTI-0DS9ZY_20231128_merged_bam.bam        | 812549e768cb4cce352cfd90e0f4624d | BAM         |
|  473 | PBCHTI           | 0DS9ZY_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1537 | IGM_PBCHTI-0DS9ZY_20231128_bam.bam               | e29923cf14632bc1a2f40a2abd403434 | BAM         |
|  474 | PBCHTI           | 0DS9ZY_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1540 | IGM_PBCHTI-0DS9ZY_20231128_merged_bam.bai        | aff02fa0cfbcd6fd5ae2e2182806cb1d | BAI         |
|  475 | PBCHTI           | 0DS9ZY_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb1538 | IGM_PBCHTI-0DS9ZY_20231128_bam.bai               | f57ec160066c8aad0f1785836953abbf | BAI         |
|  476 | PBCHEC           | 0DSHTD_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb660  | IGM_PBCHEC-0DSHTD_20231209_normal.aln.cram.crai  | fca84818a15e60834b8f56481757e902 | CRAI        |
|  477 | PBCHEC           | 0DSHTD_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb659  | IGM_PBCHEC-0DSHTD_20231209_normal.aln.cram       | 424086fae7f9bb318737e9d646c053b3 | CRAM        |
|  478 | PBCHEC           | 0DSHT3_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb662  | IGM_PBCHEC-0DSHT3_20231209_tumor.aln.cram.crai   | f5f8ef6389ae32536a96661421b67781 | CRAI        |
|  479 | PBCHEC           | 0DSHT3_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb661  | IGM_PBCHEC-0DSHT3_20231209_tumor.aln.cram        | 97269572656c72f8677ef7a15ffe816a | CRAM        |
|  480 | PBCHEC           | 0DSHSV_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb643  | IGM_PBCHEC-0DSHSV_20231208_merged_bam.bam        | d9875cc7cb748ad590669e6b476b53d5 | BAM         |
|  481 | PBCHEC           | 0DSHSV_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb641  | IGM_PBCHEC-0DSHSV_20231208_bam.bam               | 4d9e9f04cd735e505718da56e6b691ef | BAM         |
|  482 | PBCHEC           | 0DSHSV_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb642  | IGM_PBCHEC-0DSHSV_20231208_bam.bai               | 96ec1ebc8d7c76df23e08d2306e9936d | BAI         |
|  483 | PBCHEC           | 0DSHSV_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb644  | IGM_PBCHEC-0DSHSV_20231208_merged_bam.bai        | 860fd636b436af2dc7e76407769ff9c6 | BAI         |
|  484 | PBCIRT           | 0DSR4W_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb3245 | IGM_PBCIRT-0DSR4W_20231220_merged_bam.bam        | 9390e576ea05ab3b8c42530cdbb2f59e | BAM         |
|  485 | PBCIRT           | 0DSR4W_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb3246 | IGM_PBCIRT-0DSR4W_20231220_merged_bam.bai        | bdeb16eed77e479e374b642424df8d43 | BAI         |
|  486 | PBCIRT           | 0DSR4W_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb3243 | IGM_PBCIRT-0DSR4W_20231220_bam.bam               | dda9c719edf2733f14bb660de431950a | BAM         |
|  487 | PBCIRT           | 0DSR4W_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb3244 | IGM_PBCIRT-0DSR4W_20231220_bam.bai               | f36b41dc896bba85e33ad9a1f37093c5 | BAI         |
|  488 | PBCIRT           | 0DSR5D_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb3261 | IGM_PBCIRT-0DSR5D_20231219_normal.aln.cram       | ad97554a5fefde877cf5eae273656d22 | CRAM        |
|  489 | PBCIRT           | 0DSR5D_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb3262 | IGM_PBCIRT-0DSR5D_20231219_normal.aln.cram.crai  | 25edf5d45f973f40eb9dc4217c7ea0a8 | CRAI        |
|  490 | PBCIRT           | 0DSR51_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb3263 | IGM_PBCIRT-0DSR51_20231219_tumor.aln.cram        | 7a95c0ffe1f01f5fa440202e0b04d716 | CRAM        |
|  491 | PBCIRT           | 0DSR51_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb3264 | IGM_PBCIRT-0DSR51_20231219_tumor.aln.cram.crai   | 0b746b69beb9c7341eca050b40cde529 | CRAI        |
|  492 | PBCILI           | 0DSMYG_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb3049 | IGM_PBCILI-0DSMYG_20231215_normal.aln.cram.crai  | 3472c2e4764f2509ce4fc5a6e17c7529 | CRAI        |
|  493 | PBCILI           | 0DSMYG_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb3048 | IGM_PBCILI-0DSMYG_20231215_normal.aln.cram       | ba8a9abebed106ce553933fb6041212c | CRAM        |
|  494 | PBCILI           | 0DSMWS_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb3033 | IGM_PBCILI-0DSMWS_20231212_merged_bam.bai        | 1cf7e77715200e22fad7dc1bcf0f36a2 | BAI         |
|  495 | PBCILI           | 0DSMWS_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb3031 | IGM_PBCILI-0DSMWS_20231212_bam.bai               | f5d7313ba79c237ba0f516cdd5417205 | BAI         |
|  496 | PBCILI           | 0DSMWS_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb3030 | IGM_PBCILI-0DSMWS_20231212_bam.bam               | 8329335aa494bb63dd04e147a3c110cb | BAM         |
|  497 | PBCILI           | 0DSMWS_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb3032 | IGM_PBCILI-0DSMWS_20231212_merged_bam.bam        | a07e2a3a13523e33895aecc6a3f9d565 | BAM         |
|  498 | PBCILI           | 0DSMY3_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb3051 | IGM_PBCILI-0DSMY3_20231215_tumor.aln.cram.crai   | 5c0a798f136b97e9e819ed0c8a2dac5c | CRAI        |
|  499 | PBCILI           | 0DSMY3_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/31f0f8c7-8416-4660-a205-daa7a6cbfb3050 | IGM_PBCILI-0DSMY3_20231215_tumor.aln.cram        | 0a141379cf5e41fe97668e1a0022cd2e | CRAM        |
|  500 | PBCDHR           | 0DPE9D_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0b7c2ab8-396a-45e7-8b4f-e0f31b8bd374   | IGM_PBCDHR-0DPE9D_20230824_bam.bam               | 2ee4ed224c512ece8723c0b404e73e0d | BAM         |
|  501 | PBCDHR           | 0DPE9D_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/44eae1d2-ec81-40a1-b26f-8cba804f72d9   | IGM_PBCDHR-0DPE9D_20230824_bam.bai               | 4df96cdbedd1573417966aa82351d6ab | BAI         |
|  502 | PBCDHR           | 0DPE9D_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/82c472f2-3490-4001-8432-4dbcdb1cfeb3   | IGM_PBCDHR-0DPE9D_20230824_merged_bam.bai        | 79501e9f677238ff9c1e4e41e3a5d065 | BAI         |
|  503 | PBCDHR           | 0DPE9D_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/629892ba-0c57-4edd-a17d-508eba4be6d9   | IGM_PBCDHR-0DPE9D_20230824_merged_bam.bam        | 64afa55330daf4e675bf0d1fda2bba0e | BAM         |
|  504 | PBCDHR           | 0DPE9X_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/02146ff0-d2c2-4ec3-b803-6599602306a3   | IGM_PBCDHR-0DPE9X_20230824_tumor.aln.cram        | ce778247556fadaebac28d7ad93ab83d | CRAM        |
|  505 | PBCDHR           | 0DPE9X_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/46a34d68-f71b-4e9e-afee-1b660a9e33e7   | IGM_PBCDHR-0DPE9X_20230824_tumor.aln.cram.crai   | eea6e0e1d1da664f6bc79a715302f618 | CRAI        |
|  506 | PBCDHR           | 0DPE94_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/503926c7-28f3-4a0b-9b98-33c1f66e4119   | IGM_PBCDHR-0DPE94_20230824_normal.aln.cram       | 0a4419e00c60a42d64910debd0832302 | CRAM        |
|  507 | PBCDHR           | 0DPE94_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/9a48b996-63bf-43c9-826b-5f9c5e2e2882   | IGM_PBCDHR-0DPE94_20230824_normal.aln.cram.crai  | a2da824eb268f1ea74e7a0cd644af417 | CRAI        |
|  508 | PBCESS           | 0DQFJ0_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2652b017-6209-417d-b0e5-be8a163e8ddb   | IGM_PBCESS-0DQFJ0_20230922_bam.bai               | 69bb195c0180c93f10c8c3e4f0237c2d | BAI         |
|  509 | PBCESS           | 0DQFJ0_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b2f92507-0c7b-4cc5-9cd9-18ebadf9cb68   | IGM_PBCESS-0DQFJ0_20230922_bam.bam               | 91d8372eee119ec67aa64bf727f59f8c | BAM         |
|  510 | PBCESS           | 0DQFJ0_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ef702d66-99dc-4359-ab06-85dfd4f9a0a4   | IGM_PBCESS-0DQFJ0_20230922_merged_bam.bam        | 7e068355c28c61fabc6353ff91255444 | BAM         |
|  511 | PBCESS           | 0DQFJ0_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f735f468-b589-42ed-9b2e-77e8c15a76cd   | IGM_PBCESS-0DQFJ0_20230922_merged_bam.bai        | bf57a434848d6932eb491d2b20ddd079 | BAI         |
|  512 | PBCDTJ           | 0DPMRK_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/514c1934-e4fb-47d3-8eb9-0b071c054094   | IGM_PBCDTJ-0DPMRK_20230901_bam.bai               | d16c587e54fc0646657a5e400af18048 | BAI         |
|  513 | PBCDTJ           | 0DPMRK_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/bca27a62-bc4e-4797-8bfb-69bdda2895dd   | IGM_PBCDTJ-0DPMRK_20230901_merged_bam.bai        | 1c455ad84e38eff1b6ff917be3079d74 | BAI         |
|  514 | PBCDTJ           | 0DPMRK_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/98f7417a-58fe-4554-a9fb-daed0725bfd1   | IGM_PBCDTJ-0DPMRK_20230901_bam.bam               | 9c420ea12e7860fa192e45f434b7c9d9 | BAM         |
|  515 | PBCDTJ           | 0DPMRK_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ad258562-152c-45b8-af96-34bcd0e2a38d   | IGM_PBCDTJ-0DPMRK_20230901_merged_bam.bam        | 7c692318564fe8fca06ac0780ea2162b | BAM         |
|  516 | PBCDTJ           | 0DPMRR_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/c50c5f5e-f6b2-414a-a231-ac4e751132b5   | IGM_PBCDTJ-0DPMRR_20230830_tumor.aln.cram.crai   | 3e3856c0dd08b555709ae158ddd8fc49 | CRAI        |
|  517 | PBCDTJ           | 0DPMRR_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/8f2ecc3a-0901-453e-b72d-5f55b1304ba3   | IGM_PBCDTJ-0DPMRR_20230830_tumor.aln.cram        | b673df46d956601caa3eaa3b391ab7fb | CRAM        |
|  518 | PBCDTJ           | 0DPMQR_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/0538f154-6bfb-419c-b944-61150824b16d   | IGM_PBCDTJ-0DPMQR_20230830_normal.aln.cram.crai  | e7b43eae412f21b09f89fd50a965a618 | CRAI        |
|  519 | PBCDTJ           | 0DPMQR_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/79c968b9-b3cf-4cdc-9054-ded7e498f686   | IGM_PBCDTJ-0DPMQR_20230830_normal.aln.cram       | 5b14b678f5aaefa0864bd91a54489864 | CRAM        |
|  520 | PBCDLM           | 0DPDIA_DNA  | Soft Tissue Sarcoma    | Tumor          | DNA           | dg.4DFC/d957c1c2-654c-4d83-b055-dd7e4ccaaba1   | IGM_PBCDLM-0DPDIA_20230824_tumor.aln.cram        | 126f23693f96da1c22504e7c99a46cac | CRAM        |
|  521 | PBCDLM           | 0DPDIA_DNA  | Soft Tissue Sarcoma    | Tumor          | DNA           | dg.4DFC/d75bc60e-025b-4164-9512-c1295638a5b0   | IGM_PBCDLM-0DPDIA_20230824_tumor.aln.cram.crai   | 20a645b9b799913c8933dff1a8ca98f0 | CRAI        |
|  522 | PBCDLM           | 0DPDHE_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/321fa7f7-cff6-441b-b5a9-5891c443ae0e   | IGM_PBCDLM-0DPDHE_20230824_normal.aln.cram       | d9fdce101660aefdebbcbce219d99c9c | CRAM        |
|  523 | PBCDLM           | 0DPDHE_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/7411fe77-efe3-41af-8d87-274a2eebe885   | IGM_PBCDLM-0DPDHE_20230824_normal.aln.cram.crai  | c3e39709d77d6d80332dfb10e5554f33 | CRAI        |
|  524 | PBCDLM           | 0DPDHN_RNA  | Soft Tissue Sarcoma    | Tumor          | RNA           | dg.4DFC/58729b76-fde7-45a7-bdeb-14663e45d34b   | IGM_PBCDLM-0DPDHN_20230828_bam.bam               | 4f4322290fa7e5a19b855eb9eca772c2 | BAM         |
|  525 | PBCDLM           | 0DPDHN_RNA  | Soft Tissue Sarcoma    | Tumor          | RNA           | dg.4DFC/8f3a84af-d188-424a-83bc-cb124af29063   | IGM_PBCDLM-0DPDHN_20230828_bam.bai               | 8ff35d330e9bd500ce2514340820515b | BAI         |
|  526 | PBCDLM           | 0DPDHN_RNA  | Soft Tissue Sarcoma    | Tumor          | RNA           | dg.4DFC/3a56d155-ecf1-4fa7-9d98-48525241c68b   | IGM_PBCDLM-0DPDHN_20230828_merged_bam.bam        | 38630d67b88c80cb93c909ee8a299e04 | BAM         |
|  527 | PBCDLM           | 0DPDHN_RNA  | Soft Tissue Sarcoma    | Tumor          | RNA           | dg.4DFC/cefa6971-68ed-4519-8c77-1d61eae72bec   | IGM_PBCDLM-0DPDHN_20230828_merged_bam.bai        | 79aaf38f2c302d5e19f98ad82152795a | BAI         |
|  528 | PBCECF           | 0DPZUC_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f155ccca-11e4-4696-a093-573d6b02b6a8   | IGM_PBCECF-0DPZUC_20230913_normal.aln.cram       | 2e555a4d5435c412c43027454e362296 | CRAM        |
|  529 | PBCECF           | 0DPZUC_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f570fc89-4a55-498b-96c0-5a95fb188ad9   | IGM_PBCECF-0DPZUC_20230913_normal.aln.cram.crai  | c16fdf477046dd8f92e78c9a44fad7d0 | CRAI        |
|  530 | PBCECF           | 0DPZTC_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/5f705f3c-4b10-4e8a-9b13-a1e4e0ccf223   | IGM_PBCECF-0DPZTC_20230918_merged_bam.bam        | 4e06a22b7e0107f867fa8b5a262626d5 | BAM         |
|  531 | PBCECF           | 0DPZTC_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e3b669f2-93b5-49c1-a2aa-634bd642443d   | IGM_PBCECF-0DPZTC_20230918_bam.bam               | fa2467cf819793f255a7ea3e396c8ec7 | BAM         |
|  532 | PBCECF           | 0DPZTC_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b2d41f81-efec-4103-a7cb-139bdca62692   | IGM_PBCECF-0DPZTC_20230918_merged_bam.bai        | 0cb1488e5c3a974f498628dc59bc7707 | BAI         |
|  533 | PBCECF           | 0DPZTC_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/faa62e7b-a2d4-4e54-b484-5194e8232b73   | IGM_PBCECF-0DPZTC_20230918_bam.bai               | 8455de4b76815882095c9c390e3ef518 | BAI         |
|  534 | PBCECF           | 0DPZTK_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/15a59f83-0890-47f1-83a4-4c8d1f6fb91e   | IGM_PBCECF-0DPZTK_20230913_tumor.aln.cram        | 846e52ccbbf0df7cda4a098ccbeea724 | CRAM        |
|  535 | PBCECF           | 0DPZTK_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/b65d014b-c368-4041-a4d3-0406fab11ec0   | IGM_PBCECF-0DPZTK_20230913_tumor.aln.cram.crai   | 0f283bb5c762e7b108f8d27a59dd838b | CRAI        |
|  536 | PBCEFF           | 0DQ7WL_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2ff914bb-6a52-47b8-ba84-6a7810ee5b11   | IGM_PBCEFF-0DQ7WL_20230919_merged_bam.bai        | 449a18e5e485e6906ea9ab0b02bc627a | BAI         |
|  537 | PBCEFF           | 0DQ7WL_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c8d03418-c3e4-4a9b-a270-f27fe10867bf   | IGM_PBCEFF-0DQ7WL_20230919_merged_bam.bam        | 4e9ce9e43d7ec7eca3a862cd45708ee9 | BAM         |
|  538 | PBCEFF           | 0DQ7WL_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/881ee944-5ed5-4d36-8cee-774a70647e2f   | IGM_PBCEFF-0DQ7WL_20230919_bam.bam               | aaf959767de1136f80880feb693ee13b | BAM         |
|  539 | PBCEFF           | 0DQ7WL_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a008d1b8-0b21-4fce-b3af-d35fdfee425c   | IGM_PBCEFF-0DQ7WL_20230919_bam.bai               | dd974ead6550ce6670dd5514bc8d7c4f | BAI         |
|  540 | PBCEHF           | 0DQC1T_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/740fb3bc-c277-4794-b79d-54f1ac4de558   | IGM_PBCEHF-0DQC1T_20230920_bam.bai               | f56990c6a0f2bcee3b01a417403197c0 | BAI         |
|  541 | PBCEHF           | 0DQC1T_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/1070a86a-4ffa-471c-8624-2d403b802c1f   | IGM_PBCEHF-0DQC1T_20230920_bam.bam               | 1d7d3382a5512f18f3baf05728079d8a | BAM         |
|  542 | PBCEHF           | 0DQC1T_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8c450608-7445-497f-8141-fde2c0d22f30   | IGM_PBCEHF-0DQC1T_20230920_merged_bam.bai        | f5fbe2705118be5b5b17661749a090b8 | BAI         |
|  543 | PBCEHF           | 0DQC1T_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8f1525af-1e8c-431c-915e-2bc26b479cae   | IGM_PBCEHF-0DQC1T_20230920_merged_bam.bam        | 8fec5257cb8cce825fe890b905f98427 | BAM         |
|  544 | PBCBYH           | 0DP8QG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3396ad6d-4550-47b8-bceb-15188b28f5eb   | IGM_PBCBYH-0DP8QG_20230827_bam.bai               | 97341c1cf25b264c8830e622de87cc9c | BAI         |
|  545 | PBCBYH           | 0DP8QG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a2a4654d-30f8-43ae-9295-88cba75c49b9   | IGM_PBCBYH-0DP8QG_20230827_bam.bam               | eb448d3ec7acc5b97f54b1b8dd6f71ea | BAM         |
|  546 | PBCBYH           | 0DP8QG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/bc8a4c8c-da87-49b8-8d27-fcc031e80d73   | IGM_PBCBYH-0DP8QG_20230827_merged_bam.bam        | be5881c5e9ee4626165501eeebb67f6a | BAM         |
|  547 | PBCBYH           | 0DP8QG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c44b1c16-9f9b-4f00-ae4e-4d50f6b7f61f   | IGM_PBCBYH-0DP8QG_20230827_merged_bam.bai        | 127aa1a42e1699342f43c18ad11d32cb | BAI         |
|  548 | PBCAET           | 0DPZU7_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f5641930-f1df-4ffc-a6d2-cf6b49dc5834   | IGM_PBCAET-0DPZU7_20230911_normal.aln.cram       | 842bc9a93cc998243b0a2a97f8a77898 | CRAM        |
|  549 | PBCAET           | 0DPZU7_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/619934bb-3aca-4c20-b739-897f728f4b63   | IGM_PBCAET-0DPZU7_20230911_normal.aln.cram.crai  | 9eb9e55c9793af5b72838e07710d4286 | CRAI        |
|  550 | PBCAET           | 0DPZT6_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/40636e0d-6ede-4525-9bda-bee0b381d6a9   | IGM_PBCAET-0DPZT6_20230918_merged_bam.bam        | 9e32db078300041b3713739c4dfc1fb8 | BAM         |
|  551 | PBCAET           | 0DPZT6_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/79d5fc6c-4eb0-4e04-9de1-eb003d803679   | IGM_PBCAET-0DPZT6_20230918_merged_bam.bai        | c0d3bf952e6121484ec5cd66891579e7 | BAI         |
|  552 | PBCAET           | 0DPZT6_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f7185a1e-d063-46ed-9854-32ab15c69c9a   | IGM_PBCAET-0DPZT6_20230918_bam.bai               | 5aad48c98366c0b0b754bf6ffa2e78bd | BAI         |
|  553 | PBCAET           | 0DPZT6_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/715e315b-0c1f-4d37-93fd-ed170caa42fc   | IGM_PBCAET-0DPZT6_20230918_bam.bam               | bec8de8873536ece91db12de0720c52f | BAM         |
|  554 | PBCAET           | 0DPZTI_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/9e60687a-2c20-40d5-ba82-f6c839eaf986   | IGM_PBCAET-0DPZTI_20230911_tumor.aln.cram        | d632ec67c375d67038fefc3749109c13 | CRAM        |
|  555 | PBCAET           | 0DPZTI_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/4d05c347-1ef1-4245-a2be-3a733e40ad7e   | IGM_PBCAET-0DPZTI_20230911_tumor.aln.cram.crai   | 432b1887eb8f703ee257ab36ea867e7b | CRAI        |
|  556 | PBCCVE           | 0DP4PC_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2a0a246c-09e7-4720-8640-5e7429209b73   | IGM_PBCCVE-0DP4PC_20230815_merged_bam.bam        | 2cc131249d1d679847ec6e5a82774ea5 | BAM         |
|  557 | PBCCVE           | 0DP4PC_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/441c371c-fc78-4db1-8915-0f47e01da8df   | IGM_PBCCVE-0DP4PC_20230815_bam.bai               | 894993588cfc0b06c763bc399838e6d1 | BAI         |
|  558 | PBCCVE           | 0DP4PC_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8dceb831-897c-4ce9-b68c-04457b58987d   | IGM_PBCCVE-0DP4PC_20230815_bam.bam               | d4ffba101ea7f23ff50c420de1c0120a | BAM         |
|  559 | PBCCVE           | 0DP4PC_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8f2e8d39-12e3-49be-9a0c-55a22fed0908   | IGM_PBCCVE-0DP4PC_20230815_merged_bam.bai        | 14d75ce63733d5958b76a5ade613dd8f | BAI         |
|  560 | PBCCVE           | 0DP4Q1_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/f99ac948-4cea-430e-9c59-6bcdb0d9c102   | IGM_PBCCVE-0DP4Q1_20230811_tumor.aln.cram        | 6437d44115dce49acef5d39c7a6f0a9b | CRAM        |
|  561 | PBCCVE           | 0DP4Q1_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/b9fd9d70-f19c-462c-9d74-4930fb0cebf0   | IGM_PBCCVE-0DP4Q1_20230811_tumor.aln.cram.crai   | e41f47b41d354339aadea1a7f4615ed1 | CRAI        |
|  562 | PBCCVE           | 0DP4OY_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/55b223d6-30b4-4c06-b6a3-2718e32c90a6   | IGM_PBCCVE-0DP4OY_20230811_normal.aln.cram.crai  | e779291a16cc9e9adcb4bfc8ac30c8ed | CRAI        |
|  563 | PBCCVE           | 0DP4OY_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/1f16d92c-e9e3-4cdd-b617-297fdac4f195   | IGM_PBCCVE-0DP4OY_20230811_normal.aln.cram       | 55e4568cf93178759a829070b88f5038 | CRAM        |
|  564 | PBCCMF           | 0DPE9Z_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/1a564808-077d-4f24-8ef0-0becd6e4d00d   | IGM_PBCCMF-0DPE9Z_20230824_tumor.aln.cram        | 231119c3c1f57769a2adcb50b9bfb64e | CRAM        |
|  565 | PBCCMF           | 0DPE9Z_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/e3d545b0-e27c-4bfd-b69c-ad8d5e6a7cbe   | IGM_PBCCMF-0DPE9Z_20230824_tumor.aln.cram.crai   | 6244ca4e989920e359da073f46243084 | CRAI        |
|  566 | PBCCMF           | 0DPE8Z_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/d535a0e5-02b5-41be-83cd-ba763726ac8a   | IGM_PBCCMF-0DPE8Z_20230824_normal.aln.cram       | 184f7f0b4e73caf396c01e246780da20 | CRAM        |
|  567 | PBCCMF           | 0DPE8Z_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/dfd9343f-818c-483e-ae48-546daf9be334   | IGM_PBCCMF-0DPE8Z_20230824_normal.aln.cram.crai  | 1064cc673b0c080b2bacd9174ca1e666 | CRAI        |
|  568 | PBCCMF           | 0DP8QI_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d29f1c28-328c-4abd-bb5e-711e6fd774f6   | IGM_PBCCMF-0DP8QI_20230819_bam.bam               | 6d1bcca2fe6bcf47e5930029475a6b74 | BAM         |
|  569 | PBCCMF           | 0DP8QI_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/454f37d8-5f81-4349-8379-739858ebc479   | IGM_PBCCMF-0DP8QI_20230819_bam.bai               | 4f9c782b2c2a6e899ab8e55feb0490d6 | BAI         |
|  570 | PBCCMF           | 0DP8QI_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/619e11e1-3a3b-46b3-ab5e-96841eb46835   | IGM_PBCCMF-0DP8QI_20230819_merged_bam.bai        | 4aeb13724d72668c57aad39ce7b8de84 | BAI         |
|  571 | PBCCMF           | 0DP8QI_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7dc963bd-f196-484c-bb0a-af15024f864c   | IGM_PBCCMF-0DP8QI_20230819_merged_bam.bam        | 1de446dc0615a67f64181f674880dd69 | BAM         |
|  572 | PBCCEK           | 0DODO2_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/cd98d10e-9bb5-41e3-a45b-635a5553fb6d   | IGM_PBCCEK-0DODO2_20230728_tumor.aln.cram.crai   | 81d01cb25918a6560a7e82b90fb570bc | CRAI        |
|  573 | PBCCEK           | 0DODO2_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/ee445fe0-cc67-4ab2-b102-7ba56bc069cc   | IGM_PBCCEK-0DODO2_20230728_tumor.aln.cram        | 94b72e59fad163c5702e4e1ded39890d | CRAM        |
|  574 | PBCCEK           | 0DODMA_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f5ecb360-2803-4e55-801c-f424c1e309f7   | IGM_PBCCEK-0DODMA_20230728_normal.aln.cram       | 7ab4acee6bfbfbd28c708a3b33e04003 | CRAM        |
|  575 | PBCCEK           | 0DODMA_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/95fc4d7e-5e5c-4af9-93a6-b6d957a06cf1   | IGM_PBCCEK-0DODMA_20230728_normal.aln.cram.crai  | 5dad4684967a3d690c6b6f36c686acce | CRAI        |
|  576 | PBCCEK           | 0DODN1_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/bf76169d-51a9-4747-aa8d-112a25021a3c   | IGM_PBCCEK-0DODN1_20230728_bam.bam               | 5c5bd0dcd93b61e47ac87f97bf60c9aa | BAM         |
|  577 | PBCCEK           | 0DODN1_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4ba0a551-59ce-4e65-ae9d-c777363d4ec0   | IGM_PBCCEK-0DODN1_20230728_merged_bam.bam        | 2af56355a20c5469aa60ffa2034dbc3c | BAM         |
|  578 | PBCCEK           | 0DODN1_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/de49d2ed-c92c-4927-a1ce-27f279c2d190   | IGM_PBCCEK-0DODN1_20230728_bam.bai               | 6f9fa21f02b976273426706baba8fe66 | BAI         |
|  579 | PBCCEK           | 0DODN1_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c20cb65f-0444-4d12-8deb-a75d0b931581   | IGM_PBCCEK-0DODN1_20230728_merged_bam.bai        | b97cb60af2c2270485016dc50bbfd178 | BAI         |
|  580 | PBCDET           | 0DP8O4_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/973607e2-d3fb-479c-bc1e-eb766924a41a   | IGM_PBCDET-0DP8O4_20230815_merged_bam.bai        | 773bbe1a7cd68e653d2b3a0c6ae18b9f | BAI         |
|  581 | PBCDET           | 0DP8O4_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/142099fc-88f1-4d34-a5f2-ed56a287c1e2   | IGM_PBCDET-0DP8O4_20230815_bam.bai               | d3d50095bef2a39158ec0eb39d7f339c | BAI         |
|  582 | PBCDET           | 0DP8O4_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7b36443e-ac85-463b-bac6-e117850670b4   | IGM_PBCDET-0DP8O4_20230815_bam.bam               | 29e63bd8e4b45e8a330d694a7ddf9612 | BAM         |
|  583 | PBCDET           | 0DP8O4_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8520b138-8b13-405d-935b-8c89bd57a294   | IGM_PBCDET-0DP8O4_20230815_merged_bam.bam        | 457629a9db85edad16d0d3a1b21b82a4 | BAM         |
|  584 | PBCDET           | 0DP8O0_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/96f21ec6-dd26-4ddc-ac8f-b834253d8d76   | IGM_PBCDET-0DP8O0_20230818_normal.aln.cram.crai  | c869827ee79d6d47635123bdc8634031 | CRAI        |
|  585 | PBCDET           | 0DP8O0_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/ee85681a-7fca-450a-b286-5fadc8e7e243   | IGM_PBCDET-0DP8O0_20230818_normal.aln.cram       | c748c3f6471794558d2d726b8be26045 | CRAM        |
|  586 | PBCDET           | 0DP8OB_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/3c778d7c-2ab0-4b0d-9f51-6816c59962db   | IGM_PBCDET-0DP8OB_20230818_tumor.aln.cram.crai   | 304feedd5196b60305a01d967216cb4b | CRAI        |
|  587 | PBCDET           | 0DP8OB_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/2ab32951-feed-41e7-ba88-ed6949786f30   | IGM_PBCDET-0DP8OB_20230818_tumor.aln.cram        | be30a94f0b7146b92971594ce7fec029 | CRAM        |
|  588 | PBCCJT           | 0DOA28_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/30c05463-bb98-4f01-923c-d4e2aa1b7e88   | IGM_PBCCJT-0DOA28_20230725_normal.aln.cram.crai  | b30cb534f92e5fc61a8a8ae72626360c | CRAI        |
|  589 | PBCCJT           | 0DOA28_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/1fbd53e7-d875-4599-a2f7-dfac6cc37a22   | IGM_PBCCJT-0DOA28_20230725_normal.aln.cram       | 056a3c340b18a4591c5080a6239b00f0 | CRAM        |
|  590 | PBCCJT           | 0DOA3Q_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/653429f5-6e1a-4d82-8c7a-f96f60fec145   | IGM_PBCCJT-0DOA3Q_20230725_tumor.aln.cram.crai   | 38574b6ee0a157bcaa4babb87125b702 | CRAI        |
|  591 | PBCCJT           | 0DOA3Q_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/25a1a1e4-c8fc-45e2-9e7e-63cc561c6d27   | IGM_PBCCJT-0DOA3Q_20230725_tumor.aln.cram        | 91601e1a6114965cde8b28c4524ea919 | CRAM        |
|  592 | PBCCJT           | 0DOA2Z_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b8d62eb6-a8a4-448d-8d01-a9afcac2054b   | IGM_PBCCJT-0DOA2Z_20230727_bam.bai               | 6bd1c5eac9188ea7ff86703b3fe45098 | BAI         |
|  593 | PBCCJT           | 0DOA2Z_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3e4fba45-5b8f-4c3a-8810-34444b673cd8   | IGM_PBCCJT-0DOA2Z_20230727_bam.bam               | 3a996bcbe586c57da4b5b3b37808a09b | BAM         |
|  594 | PBCCJT           | 0DOA2Z_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a24dc5c0-ca57-4cfe-b417-bf9a69bb84cb   | IGM_PBCCJT-0DOA2Z_20230727_merged_bam.bai        | 07fc716190878c6879c7e03672f79e72 | BAI         |
|  595 | PBCCJT           | 0DOA2Z_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/fe656d44-1bbc-40e4-86db-97dbe3e0a1f5   | IGM_PBCCJT-0DOA2Z_20230727_merged_bam.bam        | 195e29d208ed7509f49fbe875faa247d | BAM         |
|  596 | PBCCKE           | 0DP8QH_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/6d5d08e7-ce89-46d8-b58d-2c594fe7409f   | IGM_PBCCKE-0DP8QH_20230819_bam.bam               | d4681bbb6f11f2e9f38497c910e03627 | BAM         |
|  597 | PBCCKE           | 0DP8QH_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/1dd28548-313a-4faa-8686-973d9923919b   | IGM_PBCCKE-0DP8QH_20230819_merged_bam.bam        | e4010f565e12f5639b9406a1467835bd | BAM         |
|  598 | PBCCKE           | 0DP8QH_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/1ff8c6cf-79a8-4e12-98f5-c996c4cd26c9   | IGM_PBCCKE-0DP8QH_20230819_bam.bai               | a974a1bbad2dc6ae527803e5c94021d8 | BAI         |
|  599 | PBCCKE           | 0DP8QH_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7123d60f-fac6-42d3-88cd-839376c2ec5a   | IGM_PBCCKE-0DP8QH_20230819_merged_bam.bai        | c60741059f6c5a181845c206c3ada4fd | BAI         |
|  600 | PBCANM           | 0DNRTU_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f5de1b67-a8f6-450c-995a-ef6ec9ab3ca7   | IGM_PBCANM-0DNRTU_20230717_bam.bai               | 71b7dd2fa15a4bee1a79d180e90ad4cf | BAI         |
|  601 | PBCANM           | 0DNRTU_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/5a5d7180-1116-4222-a423-75b39cc3dd1c   | IGM_PBCANM-0DNRTU_20230717_merged_bam.bam        | 6e7222676441dc8a960d1b2484dea252 | BAM         |
|  602 | PBCANM           | 0DNRTU_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/1f0b3be2-f3f9-4338-b821-178d0dcfa4d6   | IGM_PBCANM-0DNRTU_20230717_merged_bam.bai        | adb6a3c1f03e1b5cae4d463268d4d274 | BAI         |
|  603 | PBCANM           | 0DNRTU_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/95b42bb3-760b-499a-9789-cac3c5a615d2   | IGM_PBCANM-0DNRTU_20230717_bam.bam               | 75672124759cec640f1fb383e2782db4 | BAM         |
|  604 | PBCANM           | 0DNRTN_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f47b80bc-11ff-435d-ae87-7f2d4e3bc68f   | IGM_PBCANM-0DNRTN_20230713_normal.aln.cram.crai  | daf6d1767bbd00c6af232c9f443b8279 | CRAI        |
|  605 | PBCANM           | 0DNRTN_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/d842d7e2-09e6-48ae-9e1f-d166f2a48ce8   | IGM_PBCANM-0DNRTN_20230713_normal.aln.cram       | e41991fd41bd1a35ea9dddeb3da9c2e3 | CRAM        |
|  606 | PBCANM           | 0DNRUW_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/bf437b79-80a9-4784-afed-819de61a1bf3   | IGM_PBCANM-0DNRUW_20230713_tumor.aln.cram        | 9164653e6c063d0854dcde67da2d3812 | CRAM        |
|  607 | PBCANM           | 0DNRUW_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/18301593-a9e7-40ee-abc0-2571a4f91aca   | IGM_PBCANM-0DNRUW_20230713_tumor.aln.cram.crai   | 6347ad478a456ae33e188639235c834d | CRAI        |
|  608 | PBCCCJ           | 0DNYNQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/23c29386-35b1-4512-a1c5-23f62c7f1c44   | IGM_PBCCCJ-0DNYNQ_20230720_merged_bam.bai        | 09f3fb1caa9b930f683f166e6eed84bc | BAI         |
|  609 | PBCCCJ           | 0DNYNQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8f0b8f9c-b4d2-41d0-9121-710f5bc51015   | IGM_PBCCCJ-0DNYNQ_20230720_merged_bam.bam        | a3252d9a3d6b14ec7f122fa0dc6137c7 | BAM         |
|  610 | PBCCCJ           | 0DNYNQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0d8de74d-97fa-42af-9f95-e9d54b839a53   | IGM_PBCCCJ-0DNYNQ_20230720_bam.bai               | 1315ea1238924864e6520e11696739cb | BAI         |
|  611 | PBCCCJ           | 0DNYNQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b95fd66a-6925-4a76-ab15-b4eb647c6883   | IGM_PBCCCJ-0DNYNQ_20230720_bam.bam               | 14b8c68237a46ca91be27ffec8883b98 | BAM         |
|  612 | PBCCCJ           | 0DNYNK_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/5f2b9033-2890-4984-ae39-48b6b2c7b02b   | IGM_PBCCCJ-0DNYNK_20230720_normal.aln.cram.crai  | 213a14b465bb66dc78b5a0a8a35fd820 | CRAI        |
|  613 | PBCCCJ           | 0DNYNK_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/88df0d4b-d158-4fc2-808d-a86f15c37b86   | IGM_PBCCCJ-0DNYNK_20230720_normal.aln.cram       | 6cb9827ea14d62cb1d179da9fea412db | CRAM        |
|  614 | PBCCCJ           | 0DNYO1_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/a3cd51df-4d1a-4778-a7ca-385a4feba0b4   | IGM_PBCCCJ-0DNYO1_20230720_tumor.aln.cram.crai   | 389ecb1cc5f75a41b1025a7f65d2b72a | CRAI        |
|  615 | PBCCCJ           | 0DNYO1_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/015f53ab-5c73-4c02-9bf1-bf120ffbf1f6   | IGM_PBCCCJ-0DNYO1_20230720_tumor.aln.cram        | 904696277cdbce3932eb0712669471a3 | CRAM        |
|  616 | PBCDLD           | 0DPAOD_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/91d160d5-4514-4209-a19e-d313a1ae2ff8   | IGM_PBCDLD-0DPAOD_20230821_normal.aln.cram       | a7c0ac6076a26061746e76c887ec7352 | CRAM        |
|  617 | PBCDLD           | 0DPAOD_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b5732610-3be6-4634-b0f4-afb48f4bc5b2   | IGM_PBCDLD-0DPAOD_20230821_normal.aln.cram.crai  | f523362f57ce764849a280860e56a757 | CRAI        |
|  618 | PBCDLD           | 0DPAOH_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/5e520efd-cdde-4607-a0c2-9405dede93d3   | IGM_PBCDLD-0DPAOH_20230821_tumor.aln.cram        | 1e7cf828a210638bcabc4c4fc5314ea6 | CRAM        |
|  619 | PBCDLD           | 0DPAOH_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/3947791c-9348-48a1-b581-736fb9d958de   | IGM_PBCDLD-0DPAOH_20230821_tumor.aln.cram.crai   | 512f95dae30fe1d64b6924bcb1581c5c | CRAI        |
|  620 | PBCDGG           | 0DP6HL_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/354c9dde-c79c-49f9-82c3-696c90a7ffd8   | IGM_PBCDGG-0DP6HL_20230816_normal.aln.cram.crai  | 099cebcd8c83f025c0a6f32053af8dc7 | CRAI        |
|  621 | PBCDGG           | 0DP6HL_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/afa8bc31-ad4d-44f2-b53f-98c59aa3776c   | IGM_PBCDGG-0DP6HL_20230816_normal.aln.cram       | 44cda506813569419920b9e91b60ac70 | CRAM        |
|  622 | PBCDGG           | 0DP6IB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/24231e62-b5fa-4c3d-9389-dca36382df57   | IGM_PBCDGG-0DP6IB_20230815_bam.bam               | 176f8bcd4eea9145ea4efebc4d55e2ea | BAM         |
|  623 | PBCDGG           | 0DP6IB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/bf044e54-e6cc-4d77-a6c5-4eedf3e78047   | IGM_PBCDGG-0DP6IB_20230815_bam.bai               | e3ccb811ad332288167c83e5eb257e04 | BAI         |
|  624 | PBCDGG           | 0DP6IB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/38fa8516-ded2-4e29-a68d-06b65b97dd99   | IGM_PBCDGG-0DP6IB_20230815_merged_bam.bai        | 81a2224320e0b390933237e780e0013e | BAI         |
|  625 | PBCDGG           | 0DP6IB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/269207a3-ea9e-4fb1-8d13-a3d0ad493e6e   | IGM_PBCDGG-0DP6IB_20230815_merged_bam.bam        | 571fbf358ea2b06a217dec9a3e065ece | BAM         |
|  626 | PBCDGG           | 0DP6I5_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/9b8ad85e-99f1-455b-bab9-27e1fef04c9d   | IGM_PBCDGG-0DP6I5_20230816_tumor.aln.cram.crai   | 80ef20cfb12f5ddc252a769368735620 | CRAI        |
|  627 | PBCDGG           | 0DP6I5_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/072f63e6-2f31-4712-ba34-6c181c2b0218   | IGM_PBCDGG-0DP6I5_20230816_tumor.aln.cram        | f0b9709f4cf1dcff312a437bec3c7ed0 | CRAM        |
|  628 | PBCCYH           | 0DP4P1_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/3801141e-5c7c-4d9f-88de-024b1a0812df   | IGM_PBCCYH-0DP4P1_20230810_normal.aln.cram.crai  | 21cbf52175a1d14cda9f8f1c684a7227 | CRAI        |
|  629 | PBCCYH           | 0DP4P1_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f26d4c6f-8cfb-4302-8e03-f9ed1854e0de   | IGM_PBCCYH-0DP4P1_20230810_normal.aln.cram       | c7898fd7e4d413c7ff8f84ddc66949ab | CRAM        |
|  630 | PBCCYH           | 0DP4Q4_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/c1db10b9-8ca5-4c5e-879c-b3feb9a42832   | IGM_PBCCYH-0DP4Q4_20230810_tumor.aln.cram.crai   | d741a3926657cf24bc7a38490ae8f6c0 | CRAI        |
|  631 | PBCCYH           | 0DP4Q4_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/9c729b6d-e6c8-4865-98ee-667eb39b0c1b   | IGM_PBCCYH-0DP4Q4_20230810_tumor.aln.cram        | 69e295dbc4b111b98586c791eb0c87a0 | CRAM        |
|  632 | PBCCYH           | 0DP4PF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/bea2d863-4919-48a7-86d4-2469bc7763f0   | IGM_PBCCYH-0DP4PF_20230814_merged_bam.bam        | c6b6f7d52bd2b81b662b8bc956ce33df | BAM         |
|  633 | PBCCYH           | 0DP4PF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/839355fd-d47b-48c6-bbc0-ed67f099ef87   | IGM_PBCCYH-0DP4PF_20230814_bam.bam               | a6a64c23d75a5510cc7820d7400fc693 | BAM         |
|  634 | PBCCYH           | 0DP4PF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/bc6f91bb-1ddd-4123-aae4-00f7f0bc7501   | IGM_PBCCYH-0DP4PF_20230814_bam.bai               | dd1c96984f219218462f47d7b4d5b3e8 | BAI         |
|  635 | PBCCYH           | 0DP4PF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/59400e12-fcd7-41ae-ba6d-f445feea25aa   | IGM_PBCCYH-0DP4PF_20230814_merged_bam.bai        | 432db6ad8521b6005761412082de2bdf | BAI         |
|  636 | PBCCXW           | 0DP4Q3_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/52862811-1990-4902-a9a4-aeb2596d71b6   | IGM_PBCCXW-0DP4Q3_20230814_tumor.aln.cram        | 50dfba22e84e26340b789ab497d51e63 | CRAM        |
|  637 | PBCCXW           | 0DP4Q3_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/10a46883-5219-4c10-afca-b0a25e0e51e5   | IGM_PBCCXW-0DP4Q3_20230814_tumor.aln.cram.crai   | 880d6ce3a279d4e21fa0b62641019b20 | CRAI        |
|  638 | PBCCXW           | 0DP4PE_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/28fe7c3d-b910-4f92-9b84-4744d06de253   | IGM_PBCCXW-0DP4PE_20230814_merged_bam.bai        | c6ed244e2fe0cc75555947d9d5686d1f | BAI         |
|  639 | PBCCXW           | 0DP4PE_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/1a143349-ba84-404c-8565-429434ebd7ff   | IGM_PBCCXW-0DP4PE_20230814_bam.bam               | 2ada910eab3349b1a2cb841e456182d6 | BAM         |
|  640 | PBCCXW           | 0DP4PE_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/cfa0065f-32fb-4bad-abe3-beea2f162bfe   | IGM_PBCCXW-0DP4PE_20230814_merged_bam.bam        | c08f58f2f764c71efd8d35a1e7ea8a6e | BAM         |
|  641 | PBCCXW           | 0DP4PE_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/6c306b1c-0631-44e7-b543-8ce42c099d7c   | IGM_PBCCXW-0DP4PE_20230814_bam.bai               | 0dd56e8ced7222ffc00b05eec303718e | BAI         |
|  642 | PBCCXW           | 0DP4P0_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/4f0cd4d2-256c-44de-8073-a07a5fe2052f   | IGM_PBCCXW-0DP4P0_20230814_normal.aln.cram.crai  | d74d876edb0fcbc7552dda29000187ae | CRAI        |
|  643 | PBCCXW           | 0DP4P0_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/1de84083-5c9e-4645-8e91-bf1c9476e76b   | IGM_PBCCXW-0DP4P0_20230814_normal.aln.cram       | 7053dfc6f38986a9753fbfd318ee00bc | CRAM        |
|  644 | PBCDAC           | 0DOXGR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/76449106-0180-47bc-8ecc-abb7acbbac2e   | IGM_PBCDAC-0DOXGR_20230809_merged_bam.bam        | 0d0028aa4c62ec0596a0fe2138fe066c | BAM         |
|  645 | PBCDAC           | 0DOXGR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/fe01b5be-69f5-4e48-9c81-656a04af3548   | IGM_PBCDAC-0DOXGR_20230809_bam.bai               | 27fae503900ff31542915eceaa0dbe41 | BAI         |
|  646 | PBCDAC           | 0DOXGR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/fdd9d829-48dc-4a46-961a-e1f23f3d268c   | IGM_PBCDAC-0DOXGR_20230809_merged_bam.bai        | ae2a21a9fa06ed35a5aca389aff57af2 | BAI         |
|  647 | PBCDAC           | 0DOXGR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/658aad3d-bafc-4d51-938b-d02267057f66   | IGM_PBCDAC-0DOXGR_20230809_bam.bam               | babeba23a3dcf4e3d97a637db9c17855 | BAM         |
|  648 | PBCDAC           | 0DOXGY_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/cec1f9e4-4b33-4273-864b-3c1c239b1cc7   | IGM_PBCDAC-0DOXGY_20230809_tumor.aln.cram.crai   | 56e00392509b2e4cd5d231d6e6b9a6bf | CRAI        |
|  649 | PBCDAC           | 0DOXGY_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/501c33e8-3c2d-4de2-ac1f-661112e1a666   | IGM_PBCDAC-0DOXGY_20230809_tumor.aln.cram        | fa531a603464b7d9bfc89ff2e9a11c3b | CRAM        |
|  650 | PBCDAC           | 0DOXG4_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/6c02166a-feb2-4dac-b95a-1ddd009e81b6   | IGM_PBCDAC-0DOXG4_20230809_normal.aln.cram       | 1744201c47819a028fdbcd8aff23ae12 | CRAM        |
|  651 | PBCDAC           | 0DOXG4_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/589a20dd-3801-490d-b2d1-4d3c7c7bc1f9   | IGM_PBCDAC-0DOXG4_20230809_normal.aln.cram.crai  | 08ea38d5fcb1c87173a861a364718790 | CRAI        |
|  652 | PBCDFH           | 0DP8OC_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/8d2603d7-f63a-4d0e-b4bf-98c5d2ba7715   | IGM_PBCDFH-0DP8OC_20230818_tumor.aln.cram        | a5e6e953b66eb9b70e082aea836f9cf5 | CRAM        |
|  653 | PBCDFH           | 0DP8OC_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/58a3a69a-2f86-4673-b9f1-b0ef8e28b2c8   | IGM_PBCDFH-0DP8OC_20230818_tumor.aln.cram.crai   | c721d5221e2f703f62f1fae9cab896e5 | CRAI        |
|  654 | PBCDFH           | 0DP8O5_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/9226738c-c901-4165-8aa3-e5470039d62c   | IGM_PBCDFH-0DP8O5_20230819_bam.bai               | 1b3ef517f26c344bb5e25d83c339ad40 | BAI         |
|  655 | PBCDFH           | 0DP8O5_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b5774cfa-05fe-4c99-b3ae-739f7d0496f0   | IGM_PBCDFH-0DP8O5_20230819_bam.bam               | 152261fb56b78d585c52468cac3b6d1b | BAM         |
|  656 | PBCDFH           | 0DP8O5_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/cf4d4ae8-842e-426d-9215-cf8f0e54580a   | IGM_PBCDFH-0DP8O5_20230819_merged_bam.bam        | 4653168c668c37afb45cf2f5a3475786 | BAM         |
|  657 | PBCDFH           | 0DP8O5_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8e190077-9941-4b30-b8a6-5d1003b81c67   | IGM_PBCDFH-0DP8O5_20230819_merged_bam.bai        | 3d7bb33ed95e03e9fa9a396328fe08b6 | BAI         |
|  658 | PBCDFH           | 0DP8O1_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/0c856f79-baa3-4373-a4ed-8968b1b36b63   | IGM_PBCDFH-0DP8O1_20230818_normal.aln.cram.crai  | 2410dd9c7ec7dffc6c78f6001ab6462f | CRAI        |
|  659 | PBCDFH           | 0DP8O1_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/9ecbc564-d08a-4ef7-8764-172fed7f13d9   | IGM_PBCDFH-0DP8O1_20230818_normal.aln.cram       | dae29b1a26fb6a385245870d180520fd | CRAM        |
|  660 | PBCCFA           | 0DNYO0_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/6c08c1ba-73c6-431e-9a4b-c43999efd7fd   | IGM_PBCCFA-0DNYO0_20230719_tumor.aln.cram.crai   | 36fbcd1a0629ca45ac45da4bd358b2a3 | CRAI        |
|  661 | PBCCFA           | 0DNYO0_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/783a7862-9e0b-4ce4-99ca-9c5b47e92238   | IGM_PBCCFA-0DNYO0_20230719_tumor.aln.cram        | ae8e4ad4b5020c1dd9c8f99e4a4cff43 | CRAM        |
|  662 | PBCCFA           | 0DNYNL_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/702d689b-6dbd-41b0-933c-5128a014e4df   | IGM_PBCCFA-0DNYNL_20230719_normal.aln.cram.crai  | 8c2b12fea83e7c93a03bd3d29fcc4c63 | CRAI        |
|  663 | PBCCFA           | 0DNYNL_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/249644e3-86fa-4f5c-a96e-212a00de6052   | IGM_PBCCFA-0DNYNL_20230719_normal.aln.cram       | aea5624d9f4411b0f565f3d0d2244312 | CRAM        |
|  664 | PBCCXK           | 0DP4OZ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/095fbfca-7d7d-4098-9369-4172cd89ff2e   | IGM_PBCCXK-0DP4OZ_20230811_normal.aln.cram       | 88fad403fd67650e2b5bdbf385f08cfc | CRAM        |
|  665 | PBCCXK           | 0DP4OZ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/d41a95b7-c872-4ad7-86dd-cf863ffc932e   | IGM_PBCCXK-0DP4OZ_20230811_normal.aln.cram.crai  | 96906d044db2413e5f66adab884989ba | CRAI        |
|  666 | PBCCXK           | 0DP4PD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/84b08491-fa14-409f-be29-908668537684   | IGM_PBCCXK-0DP4PD_20230814_bam.bai               | a8e0fc864c40ce583cdd4459fa074359 | BAI         |
|  667 | PBCCXK           | 0DP4PD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/55a2da75-038c-4f47-bef5-402858294474   | IGM_PBCCXK-0DP4PD_20230814_bam.bam               | 6de823868e769860fe462a912a7c9389 | BAM         |
|  668 | PBCCXK           | 0DP4PD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/832c269f-0ca9-4fc1-92a4-c3d5d058d5de   | IGM_PBCCXK-0DP4PD_20230814_merged_bam.bai        | 406bb562a5e848aced47cb2e2dfbb127 | BAI         |
|  669 | PBCCXK           | 0DP4PD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/76531f9d-acec-40e6-bd21-aaaced888bb3   | IGM_PBCCXK-0DP4PD_20230814_merged_bam.bam        | 3a59dda5abe065a2cc9471f0c406f14e | BAM         |
|  670 | PBCCXK           | 0DP4Q2_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/b916ec6a-6543-4012-8636-d452f9951936   | IGM_PBCCXK-0DP4Q2_20230811_tumor.aln.cram        | 0e6d2f09ab992e013d707faaab93ba22 | CRAM        |
|  671 | PBCCXK           | 0DP4Q2_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/fa4eff57-b6a6-4eb2-aefd-31d6267bd754   | IGM_PBCCXK-0DP4Q2_20230811_tumor.aln.cram.crai   | cf96c9bfd175fca0b9bb788b9c2223fe | CRAI        |
|  672 | PBCBHX           | 0DOG3D_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/81dd61ee-4de3-4a71-a038-ac99f3659dc1   | IGM_PBCBHX-0DOG3D_20230731_tumor.aln.cram.crai   | f8550fb8a9b71e9daa1c0810d6dbdf75 | CRAI        |
|  673 | PBCBHX           | 0DOG3D_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/bc65d9f6-99cc-4d71-88a7-c488175af70d   | IGM_PBCBHX-0DOG3D_20230731_tumor.aln.cram        | 9f7ce3bbc98077a08b63206a204b322c | CRAM        |
|  674 | PBCBHX           | 0DOG2U_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/24a63817-c339-4aa5-9577-6adad4732aba   | IGM_PBCBHX-0DOG2U_20230731_normal.aln.cram       | 39f6316ffb7fed4e2f705e4badeccc5a | CRAM        |
|  675 | PBCBHX           | 0DOG2U_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/19990835-f88d-43b3-9212-80648859d589   | IGM_PBCBHX-0DOG2U_20230731_normal.aln.cram.crai  | ef7f89673717fda088f2ba5140c2e56d | CRAI        |
|  676 | PBCBHX           | 0DOG3N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f8b433ae-92a3-45c0-8269-cfb3ecd35e60   | IGM_PBCBHX-0DOG3N_20230731_merged_bam.bam        | 61249a57485ace78900c7ca038d73c73 | BAM         |
|  677 | PBCBHX           | 0DOG3N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ce0781fc-42a9-4250-8378-36498120221f   | IGM_PBCBHX-0DOG3N_20230731_merged_bam.bai        | 95e003ab970763340d81f377e8482024 | BAI         |
|  678 | PBCBHX           | 0DOG3N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3cd33294-c3e9-4281-b332-46e03a83dfa5   | IGM_PBCBHX-0DOG3N_20230731_bam.bai               | b8225b70aa7dd429248b35f6be5143a0 | BAI         |
|  679 | PBCBHX           | 0DOG3N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/44fdbc82-30a9-49b5-aae0-0e0ee5c37ae3   | IGM_PBCBHX-0DOG3N_20230731_bam.bam               | 0088b7d5a020f028087a034e1f4bdf54 | BAM         |
|  680 | PBCCIP           | 0DO5FM_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/7b9d00cf-ca26-4a07-8230-fa2507626c4b   | IGM_PBCCIP-0DO5FM_20230724_tumor.aln.cram.crai   | eccccf53ae83f70a255e6c43ef88f024 | CRAI        |
|  681 | PBCCIP           | 0DO5FM_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/0920d993-cbb8-483d-94c6-72a1d015456d   | IGM_PBCCIP-0DO5FM_20230724_tumor.aln.cram        | 5202ffc3864c99b02d1bceb86fcee9fc | CRAM        |
|  682 | PBCCIP           | 0DO5EZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/18e057d9-f850-4c31-a2c3-fde718746110   | IGM_PBCCIP-0DO5EZ_20230724_merged_bam.bai        | 7828aaec8e6b386a296616070345a8bb | BAI         |
|  683 | PBCCIP           | 0DO5EZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/031f63cb-1999-45a1-a0e3-099406332df7   | IGM_PBCCIP-0DO5EZ_20230724_bam.bam               | e4540e7b1e1e432de6edea19bfb9e0fe | BAM         |
|  684 | PBCCIP           | 0DO5EZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e7d6f5ba-206d-4446-9f70-d49703d56a1c   | IGM_PBCCIP-0DO5EZ_20230724_merged_bam.bam        | 8fdc3bcd940e862becebe3d864ebc390 | BAM         |
|  685 | PBCCIP           | 0DO5EZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f52f699f-3851-47ac-bbed-88efd6cc222b   | IGM_PBCCIP-0DO5EZ_20230724_bam.bai               | a721b10d976c381574862c96d9232810 | BAI         |
|  686 | PBCCIP           | 0DO5E0_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/000ea848-7707-4943-83ca-95aa2b54f04b   | IGM_PBCCIP-0DO5E0_20230724_normal.aln.cram       | ea9a94dd4410c4f24e03ca04a1cd428a | CRAM        |
|  687 | PBCCIP           | 0DO5E0_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/cc5c7a57-755f-4261-beb7-19187f3d3378   | IGM_PBCCIP-0DO5E0_20230724_normal.aln.cram.crai  | 9833e66aafde28126f7d4dbc10eacb08 | CRAI        |
|  688 | PBBYKX           | 0DNGGJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/406f755d-6ada-4ece-8edc-01cc3ae5fda0   | IGM_PBBYKX-0DNGGJ_20230706_merged_bam.bai        | 025cdc53daaf9841059b80463fa30118 | BAI         |
|  689 | PBBYKX           | 0DNGGJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/69cdda5f-da94-494a-9288-f3fae91e1a3a   | IGM_PBBYKX-0DNGGJ_20230706_merged_bam.bam        | 4e3469f06e6735dd09b020564158f755 | BAM         |
|  690 | PBBYKX           | 0DNGGJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/06ddb0de-fba1-4002-ae4c-110a886a9b92   | IGM_PBBYKX-0DNGGJ_20230706_bam.bai               | 95e2da09a3c2629299ec1782cb4d3eac | BAI         |
|  691 | PBBYKX           | 0DNGGJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/5c3df36c-9a6d-437d-8afc-61f48ca5cbc5   | IGM_PBBYKX-0DNGGJ_20230706_bam.bam               | 61007c6a23014f9750d4e37fac4bd499 | BAM         |
|  692 | PBBYKX           | 0DNGGB_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/57e5ebcf-e48c-4817-bb6d-455d7ffcefc8   | IGM_PBBYKX-0DNGGB_20230706_normal.aln.cram.crai  | adb352bff85ed1cb224554486e713ec7 | CRAI        |
|  693 | PBBYKX           | 0DNGGB_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/e2f2aad1-f752-4b24-a142-e4f4e01e29f4   | IGM_PBBYKX-0DNGGB_20230706_normal.aln.cram       | d961221a26e6b18f837bce47fc35a52f | CRAM        |
|  694 | PBBYKX           | 0DNGGT_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/a706c8b0-e20f-4f26-8f71-77c093636d94   | IGM_PBBYKX-0DNGGT_20230706_tumor.aln.cram        | 17d350835e10831f7da577b38bbcf6ca | CRAM        |
|  695 | PBBYKX           | 0DNGGT_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/e8785f71-5776-4d64-8f6e-ebac80f1ef62   | IGM_PBBYKX-0DNGGT_20230706_tumor.aln.cram.crai   | 3b20f8d2e3b73fbfad7db10423011203 | CRAI        |
|  696 | PBCBFB           | 0DN3ST_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/21dde086-f329-4dbf-a54a-7807ee50cae2   | IGM_PBCBFB-0DN3ST_20230627_tumor.aln.cram        | a881c9e0efacf6272f8f881da824b9bb | CRAM        |
|  697 | PBCBFB           | 0DN3ST_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/a1f4e7fe-5efc-47a8-807d-5ecf7a1ff23b   | IGM_PBCBFB-0DN3ST_20230627_tumor.aln.cram.crai   | 57558239defa2b97a28e8a086afc8211 | CRAI        |
|  698 | PBCBFB           | 0DN3R6_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/bf641d22-1258-415f-9665-927d37d9ed1c   | IGM_PBCBFB-0DN3R6_20230627_normal.aln.cram       | db3c92d330f75909fcecf777a273938b | CRAM        |
|  699 | PBCBFB           | 0DN3R6_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/54640e4d-35a3-450d-96a9-de5dccd50beb   | IGM_PBCBFB-0DN3R6_20230627_normal.aln.cram.crai  | 36ce9a9ea5d881af2c13ed5acf8cd8f0 | CRAI        |
|  700 | PBCBFB           | 0DN3SB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a9355017-0bf0-4d24-9006-12f51ddee23c   | IGM_PBCBFB-0DN3SB_20230626_merged_bam.bam        | 06cca7ba9f95c016bb5dbc4bdb07e9a5 | BAM         |
|  701 | PBCBFB           | 0DN3SB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4427c21b-84df-4065-9313-0f4cb2ddc67b   | IGM_PBCBFB-0DN3SB_20230626_bam.bai               | 3920662e1246b29bc3973ee4737ada93 | BAI         |
|  702 | PBCBFB           | 0DN3SB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/466b6ee8-cfc1-4886-93a4-982612c97fe2   | IGM_PBCBFB-0DN3SB_20230626_bam.bam               | 217a492fe5c21e5b9e64a37065113dbf | BAM         |
|  703 | PBCBFB           | 0DN3SB_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/21bc0156-3a7c-4915-b177-a2e3993fef1f   | IGM_PBCBFB-0DN3SB_20230626_merged_bam.bai        | 380710e701ac904920acb8fbbd6eacf3 | BAI         |
|  704 | PBCBDL           | 0DN3SE_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0e587e84-afb1-4293-a504-45b81d0f2f08   | IGM_PBCBDL-0DN3SE_20230625_merged_bam.bam        | 52ec1dde8e22a197afc7e6f4dc7e9cf0 | BAM         |
|  705 | PBCBDL           | 0DN3SE_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7b6845ad-efb8-4298-9dc3-9e9b1c67dc2f   | IGM_PBCBDL-0DN3SE_20230625_merged_bam.bai        | 1cbe253cbff99e2395e47a87b318bdec | BAI         |
|  706 | PBCBDL           | 0DN3SE_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/82817db2-19ca-4ddb-b9c8-c65ac5e27deb   | IGM_PBCBDL-0DN3SE_20230625_bam.bam               | 6a3d6931161b76d80e309a478c8015ff | BAM         |
|  707 | PBCBDL           | 0DN3SE_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/24520b26-10bb-466a-9b6e-c70dedff51e4   | IGM_PBCBDL-0DN3SE_20230625_bam.bai               | c7ad81e77cf8ffc389d3fb876c5a3726 | BAI         |
|  708 | PBCBDL           | 0DN3SQ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/fc31b231-dfd1-4b92-a3af-1cb41deab051   | IGM_PBCBDL-0DN3SQ_20230623_tumor.aln.cram        | 24f5880c7f7dbb7bdd3c6dddbbd80774 | CRAM        |
|  709 | PBCBDL           | 0DN3SQ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/3d59b73a-e452-40ed-8bdb-8378d79e285d   | IGM_PBCBDL-0DN3SQ_20230623_tumor.aln.cram.crai   | 78d7b051c1dba17aca94c2b6ad2e564f | CRAI        |
|  710 | PBCBDL           | 0DN3R5_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/c6baca82-1d42-4758-8dd1-e642085b4f20   | IGM_PBCBDL-0DN3R5_20230623_normal.aln.cram.crai  | 17070afa06a2824448673fc33b87772f | CRAI        |
|  711 | PBCBDL           | 0DN3R5_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/938061b1-338e-4c03-9b25-ca49dc2d0059   | IGM_PBCBDL-0DN3R5_20230623_normal.aln.cram       | aec9754196c769d0fcd04e55fa2b8b65 | CRAM        |
|  712 | PBCCJG           | 0DO7MI_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/6b09ccdd-6e5e-4f7e-89b0-88724fcecd0b   | IGM_PBCCJG-0DO7MI_20230725_tumor.aln.cram.crai   | c5f3a4d2887ea5669842667d436568a7 | CRAI        |
|  713 | PBCCJG           | 0DO7MI_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/0c32636c-9bba-49d5-ba92-582ef7bcafd9   | IGM_PBCCJG-0DO7MI_20230725_tumor.aln.cram        | 6b2a396fc9882e196b3ccbfd02331b60 | CRAM        |
|  714 | PBCCJG           | 0DO7MA_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2af7fcae-b7dd-4b04-9917-d065abe8e8ab   | IGM_PBCCJG-0DO7MA_20230725_bam.bai               | 06413b6a3fa4f73738ba0de789b6c944 | BAI         |
|  715 | PBCCJG           | 0DO7MA_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f08b996f-a1db-40bc-8dbb-8d1a7305b86a   | IGM_PBCCJG-0DO7MA_20230725_merged_bam.bam        | a08b0022aa878f224af64ccb5b1546e3 | BAM         |
|  716 | PBCCJG           | 0DO7MA_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2eee2055-6fcf-4de6-a018-367c11db03c4   | IGM_PBCCJG-0DO7MA_20230725_merged_bam.bai        | 417bd4361443b716adb493a9fadb7646 | BAI         |
|  717 | PBCCJG           | 0DO7MA_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b3af905e-fbdd-4f50-9bdf-2e2de6c1f5c9   | IGM_PBCCJG-0DO7MA_20230725_bam.bam               | a4e13760f9bc8f8416c1ffae70f1aec9 | BAM         |
|  718 | PBCCJG           | 0DO7LZ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/bf5aa057-7f97-4057-acf8-215bf667eddb   | IGM_PBCCJG-0DO7LZ_20230725_normal.aln.cram.crai  | cacec640847f9418449d033a0547afda | CRAI        |
|  719 | PBCCJG           | 0DO7LZ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f5a5f4d7-d3a6-4b72-85f8-62af3c03efab   | IGM_PBCCJG-0DO7LZ_20230725_normal.aln.cram       | bf444e67529e9515990918f8d9f5cb71 | CRAM        |
|  720 | PBCBYV           | 0DNWJR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/059c9527-c6ea-4d7e-a8ea-830c8598a1a3   | IGM_PBCBYV-0DNWJR_20230714_merged_bam.bam        | 471fac5259e3b80b421f8cb675c3e66d | BAM         |
|  721 | PBCBYV           | 0DNWJR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8136071f-3ee3-44dd-8d11-4f2152539bcf   | IGM_PBCBYV-0DNWJR_20230714_bam.bam               | db2392a2506b9bf6bdbfc8404544b2b1 | BAM         |
|  722 | PBCBYV           | 0DNWJR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/6c4de42d-8277-47fd-a088-50b347b09d85   | IGM_PBCBYV-0DNWJR_20230714_merged_bam.bai        | 2197509753707cccf021aefa5b4ef916 | BAI         |
|  723 | PBCBYV           | 0DNWJR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/1194de59-1948-4ad6-bf60-ef5bf14e1360   | IGM_PBCBYV-0DNWJR_20230714_bam.bai               | 4e4276d8c0fda3ea126ebe3df2904fb7 | BAI         |
|  724 | PBCBYV           | 0DNWKV_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/d922383a-ea9d-4d5a-9551-fa04b5335796   | IGM_PBCBYV-0DNWKV_20230717_tumor.aln.cram        | 0e54f97376756e5c9dc0cf0bb9ae6be6 | CRAM        |
|  725 | PBCBYV           | 0DNWKV_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/e620860e-1202-4047-bbc4-54a57f5e3a7e   | IGM_PBCBYV-0DNWKV_20230717_tumor.aln.cram.crai   | 91b469d228b6c927d35ca9b1a4bd8920 | CRAI        |
|  726 | PBCBYV           | 0DNWJJ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/0b27f95c-7fb2-4b34-b7aa-b4b763ea2aa8   | IGM_PBCBYV-0DNWJJ_20230717_normal.aln.cram       | 6ea0a6801ade95b00a9f01ceb1447d4f | CRAM        |
|  727 | PBCBYV           | 0DNWJJ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/802b07ca-8e5b-4da0-af04-fd0d11114b26   | IGM_PBCBYV-0DNWJJ_20230717_normal.aln.cram.crai  | d3fe910fd2169516f757cd6816b3768e | CRAI        |
|  728 | PBCCLV           | 0DOV9I_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/caf5c9ff-42eb-4083-b521-9862cdfe76e0   | IGM_PBCCLV-0DOV9I_20230808_tumor.aln.cram        | 36d0f38aa99f3f4347672a53ec6a3fc4 | CRAM        |
|  729 | PBCCLV           | 0DOV9I_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/27d76d8c-37da-4343-b719-1cd19451faba   | IGM_PBCCLV-0DOV9I_20230808_tumor.aln.cram.crai   | 01d4bbfaa3f6bf3fb9eaf014d39bde8c | CRAI        |
|  730 | PBCCLV           | 0DOV8I_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/c8b84149-f7b4-47da-9b79-512eaea7e19c   | IGM_PBCCLV-0DOV8I_20230808_normal.aln.cram.crai  | 1c87fbe2172d5a67d5c8e22c891b6aa0 | CRAI        |
|  731 | PBCCLV           | 0DOV8I_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/395ef78f-77e7-4955-a232-f818e6323f7b   | IGM_PBCCLV-0DOV8I_20230808_normal.aln.cram       | c489c4daa9afa7b3a2c3fb760f7bbbef | CRAM        |
|  732 | PBCCLV           | 0DOV8Y_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7362e0f9-239d-4b7a-991e-9229ca83ddac   | IGM_PBCCLV-0DOV8Y_20230808_bam.bai               | 1614290ecc432ef57e8aff461aff3185 | BAI         |
|  733 | PBCCLV           | 0DOV8Y_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/823ac660-fd64-497a-b749-b2e4ca376baa   | IGM_PBCCLV-0DOV8Y_20230808_bam.bam               | c92616e29623bee8579bff0eb8c814c5 | BAM         |
|  734 | PBCCLV           | 0DOV8Y_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/9d63494b-0661-4704-8679-eaa28ae9d76d   | IGM_PBCCLV-0DOV8Y_20230808_merged_bam.bam        | 5a0b67e89ed02bb28aa059f33f89b680 | BAM         |
|  735 | PBCCLV           | 0DOV8Y_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7bd24ca6-c03c-497a-8e11-e8469cd959e4   | IGM_PBCCLV-0DOV8Y_20230808_merged_bam.bai        | 10a083b8bc7e67be76a1740c19068fda | BAI         |
|  736 | PBCBBE           | 0DN1B6_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/0f4edb5f-7802-43f7-bb1e-bec8976cc75a   | IGM_PBCBBE-0DN1B6_20230622_normal.aln.cram       | 1e32ba75d47804deeae9f2df44bb0945 | CRAM        |
|  737 | PBCBBE           | 0DN1B6_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/64ee2636-350a-46d7-9134-894caad71692   | IGM_PBCBBE-0DN1B6_20230622_normal.aln.cram.crai  | a97aa44d2eb52755c61b7a3bb1913841 | CRAI        |
|  738 | PBCBBE           | 0DN1BV_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a19b5fa5-f2e7-4cd9-a090-9e16d89846fb   | IGM_PBCBBE-0DN1BV_20230623_bam.bam               | 027eee7251716d1a9f34cad31a6d5786 | BAM         |
|  739 | PBCBBE           | 0DN1BV_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/51e05885-3cd1-420c-82d0-57632fe4b61c   | IGM_PBCBBE-0DN1BV_20230623_merged_bam.bai        | 615ba528541a2b40282e003a224a7f8f | BAI         |
|  740 | PBCBBE           | 0DN1BV_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3f079a7a-c259-49de-8f74-211e365e001c   | IGM_PBCBBE-0DN1BV_20230623_merged_bam.bam        | 5487b5f557399ea384b4d30ee3d33de6 | BAM         |
|  741 | PBCBBE           | 0DN1BV_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/93ec9260-bd99-4a19-bfc6-eb54bc7fb883   | IGM_PBCBBE-0DN1BV_20230623_bam.bai               | 20f5d8400a09068205bfaf5ec8b6e2cd | BAI         |
|  742 | PBCBBE           | 0DN1D4_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/f06bccc3-cac8-46cb-9360-eda67fb6c59e   | IGM_PBCBBE-0DN1D4_20230622_tumor.aln.cram.crai   | 6d91789b25510c83c0ba62fe43eb8324 | CRAI        |
|  743 | PBCBBE           | 0DN1D4_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/741eab5e-9104-4fa3-b81e-8e36bfefb2f6   | IGM_PBCBBE-0DN1D4_20230622_tumor.aln.cram        | f357a282062dcf023a232f93b8903516 | CRAM        |
|  744 | PBCASF           | 0DN51E_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/8748469f-7d9d-4e38-b421-022026741ae4   | IGM_PBCASF-0DN51E_20230628_normal.aln.cram.crai  | 06a093bfbef0069603f15e5c06f790cb | CRAI        |
|  745 | PBCASF           | 0DN51E_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/44423322-8402-4747-beec-755a58cd00c6   | IGM_PBCASF-0DN51E_20230628_normal.aln.cram       | 9f60c22d6611d01d45e2a86167a03684 | CRAM        |
|  746 | PBCASF           | 0DN51N_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/c3edcb13-9c5d-4210-9cac-8dab210a2abe   | IGM_PBCASF-0DN51N_20230628_tumor.aln.cram.crai   | 343afd0d274f6425da8a212738d9de87 | CRAI        |
|  747 | PBCASF           | 0DN51N_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/bc1f9ece-736d-4954-8e71-f9bb0b981875   | IGM_PBCASF-0DN51N_20230628_tumor.aln.cram        | b6de4eaf642339597a458bf9b1edaf7f | CRAM        |
|  748 | PBCASF           | 0DN524_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/596db7d3-dcba-407d-989d-20e3dc001089   | IGM_PBCASF-0DN524_20230627_merged_bam.bai        | 64849d781d562092103606a9cd2e65ab | BAI         |
|  749 | PBCASF           | 0DN524_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/6d6cbf17-6788-45e6-bf2c-ab1e5cd73a5d   | IGM_PBCASF-0DN524_20230627_bam.bai               | 770f7447ead075dad9af442a1a331cd0 | BAI         |
|  750 | PBCASF           | 0DN524_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/158a9e1c-38c9-496d-91f3-2048d18b0395   | IGM_PBCASF-0DN524_20230627_merged_bam.bam        | 3820433b13acd3dbb7446c4eed6afbf0 | BAM         |
|  751 | PBCASF           | 0DN524_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/db22b819-7497-4daa-81f8-0dc74c837df2   | IGM_PBCASF-0DN524_20230627_bam.bam               | b011c35ae8880c250702f41051a0d261 | BAM         |
|  752 | PBCAKG           | 0DMKKU_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/bfcadff6-fca4-4fdb-adfb-7c69f27254a9   | IGM_PBCAKG-0DMKKU_20230614_bam.bam               | 5acc105daa277b1279afdf1f6b52f8ec | BAM         |
|  753 | PBCAKG           | 0DMKKU_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2fa95037-f532-4485-b2e3-d4967a22d02d   | IGM_PBCAKG-0DMKKU_20230614_merged_bam.bai        | e235717c48f1bf55b4a712856209e681 | BAI         |
|  754 | PBCAKG           | 0DMKKU_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0a8ec3e4-f50a-44ac-bd29-f924278de2c6   | IGM_PBCAKG-0DMKKU_20230614_merged_bam.bam        | e3d585e9ab66eb07991ca3ade40b5416 | BAM         |
|  755 | PBCAKG           | 0DMKKU_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/56a9dfc0-4c1b-4ad2-ba11-faa8478142de   | IGM_PBCAKG-0DMKKU_20230614_bam.bai               | 6fce5fe4cff19f44fa2662c4cc2b2b02 | BAI         |
|  756 | PBCAWA           | 0DMOXO_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/0dd34fb9-c18e-4569-bc7f-f3a8e80c7d45   | IGM_PBCAWA-0DMOXO_20230616_normal.aln.cram.crai  | 168f289571c8685cec72335140a6d006 | CRAI        |
|  757 | PBCAWA           | 0DMOXO_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/0d41c310-269b-4918-a0aa-9b537a59c295   | IGM_PBCAWA-0DMOXO_20230616_normal.aln.cram       | 0d66b5bd310cdc61e65d79a7079ec062 | CRAM        |
|  758 | PBCAWA           | 0DMOXJ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/6094d665-234f-4848-967c-0542015dd809   | IGM_PBCAWA-0DMOXJ_20230616_tumor.aln.cram        | c0eec2316155ac4a68d963d0f846038b | CRAM        |
|  759 | PBCAWA           | 0DMOXJ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/585baf1b-1444-41ed-8eec-534916b2d3ea   | IGM_PBCAWA-0DMOXJ_20230616_tumor.aln.cram.crai   | 23df6285902a8ec60b33f45abc95cff4 | CRAI        |
|  760 | PBCAWA           | 0DMOX3_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/87b12e4a-c2fc-4357-8360-77e60a835626   | IGM_PBCAWA-0DMOX3_20230614_merged_bam.bai        | 5bd31605bd2b012e64656bf646672e30 | BAI         |
|  761 | PBCAWA           | 0DMOX3_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b7a2f578-d2ef-4f6d-b627-a42532ee8a0b   | IGM_PBCAWA-0DMOX3_20230614_bam.bai               | 359cfc07388bdffdadf2ff4573933a1e | BAI         |
|  762 | PBCAWA           | 0DMOX3_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4bf41f7c-e897-47c5-8184-482c7e2fe4cd   | IGM_PBCAWA-0DMOX3_20230614_merged_bam.bam        | 164be74e0df28621af28ac49d4f1de05 | BAM         |
|  763 | PBCAWA           | 0DMOX3_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/359fac82-4daa-41ae-8524-448af675d1b1   | IGM_PBCAWA-0DMOX3_20230614_bam.bam               | 1db9ffa28209870af38c03b11cfcd17e | BAM         |
|  764 | PBCAFX           | 0DMIOH_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/e793d8db-b039-4f77-8a50-fe405780867c   | IGM_PBCAFX-0DMIOH_20230608_tumor.aln.cram        | ded4acd98299999a495c8819fb14673f | CRAM        |
|  765 | PBCAFX           | 0DMIOH_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/b9e59611-702e-46fe-9cbe-f8f1436115a7   | IGM_PBCAFX-0DMIOH_20230608_tumor.aln.cram.crai   | 76e4251fd0b07ea017fb1f713a6217ec | CRAI        |
|  766 | PBCAFX           | 0DMIO0_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3624e121-4d6d-4089-9681-08d7dcb9588c   | IGM_PBCAFX-0DMIO0_20230609_bam.bai               | 108f3bf188d6bebe52fe92a8f798a67d | BAI         |
|  767 | PBCAFX           | 0DMIO0_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e02d2d24-d50e-4dfd-aaeb-79e7c4bb5855   | IGM_PBCAFX-0DMIO0_20230609_bam.bam               | c25192a9018f96abbaec42cde0e9788b | BAM         |
|  768 | PBCAFX           | 0DMIO0_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ccc4c5b1-2d98-4589-ba48-ae4cb39ea472   | IGM_PBCAFX-0DMIO0_20230609_merged_bam.bam        | 9dc2a95331e2468735609cba19f2a958 | BAM         |
|  769 | PBCAFX           | 0DMIO0_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e765a26f-37c5-44ef-b8e9-60b126284c37   | IGM_PBCAFX-0DMIO0_20230609_merged_bam.bai        | 19bae7cde89e9316c8dac5567343de36 | BAI         |
|  770 | PBCAFX           | 0DMIPH_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/4f9e4843-0458-4ff5-b9a2-8dd7b07d22da   | IGM_PBCAFX-0DMIPH_20230608_normal.aln.cram.crai  | f89e05e93eba47ed5de779e6d6b97710 | CRAI        |
|  771 | PBCAFX           | 0DMIPH_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/2ceaff97-3928-49fd-b246-d6ee73f534c6   | IGM_PBCAFX-0DMIPH_20230608_normal.aln.cram       | ce20636bab8d2a8e5d1ba7c6f2e59285 | CRAM        |
|  772 | PBBZGH           | 0DLBM1_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/bdc7cf04-a0f2-4a9a-bff4-0fca13a12aeb   | IGM_PBBZGH-0DLBM1_20230510_normal.aln.cram.crai  | 31bb6e68996ae16417cc4812e6b093ad | CRAI        |
|  773 | PBBZGH           | 0DLBM1_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/29489638-22ee-4ded-823f-5f760bb24aa9   | IGM_PBBZGH-0DLBM1_20230510_normal.aln.cram       | 1763de14b88408365e9c3123c913e0d9 | CRAM        |
|  774 | PBBZGH           | 0DLBMJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/57335064-02a4-4ee3-8036-9c50bfb5bdab   | IGM_PBBZGH-0DLBMJ_20230504_bam.bam               | 31b1cc2b21eda42ab832a7de4ff701c8 | BAM         |
|  775 | PBBZGH           | 0DLBMJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/5c8b2177-41e7-4894-b54d-753a13228972   | IGM_PBBZGH-0DLBMJ_20230504_bam.bai               | c6cfe55fb87edfff01d66902ac8cdd2e | BAI         |
|  776 | PBBZGH           | 0DLBMJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/56cec8a3-5406-420c-95f7-c598515e6c25   | IGM_PBBZGH-0DLBMJ_20230504_merged_bam.bam        | 4bbe104b42f3d415a166894f8f0974a7 | BAM         |
|  777 | PBBZGH           | 0DLBMJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/35843655-bac2-47cc-807a-61a33ccc5449   | IGM_PBBZGH-0DLBMJ_20230504_merged_bam.bai        | ae66c6014e3daa423c9f90b59428d0fe | BAI         |
|  778 | PBBZGH           | 0DLBMT_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/ac95f93c-290f-4fad-8288-99a076fdd37b   | IGM_PBBZGH-0DLBMT_20230510_tumor.aln.cram.crai   | d57ab5936b35ea3409fbb00b2e1b7a43 | CRAI        |
|  779 | PBBZGH           | 0DLBMT_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/b5b5dc03-26c6-446f-99c6-ccfeb397b6de   | IGM_PBBZGH-0DLBMT_20230510_tumor.aln.cram        | f92e2fb9b2fd8b3aaef7b2818f0fb108 | CRAM        |
|  780 | PBBZKS           | 0DLIFQ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/0b19e19a-7c28-414b-a20e-422adf0f55f3   | IGM_PBBZKS-0DLIFQ_20230515_tumor.aln.cram        | 3d71103cd8e64c8dbd27e35974f5d7e2 | CRAM        |
|  781 | PBBZKS           | 0DLIFQ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/7c9c9fa9-e243-4822-924b-a78fc7f3e220   | IGM_PBBZKS-0DLIFQ_20230515_tumor.aln.cram.crai   | 1c3d40044e0f610c3dbc0973b5418100 | CRAI        |
|  782 | PBBZKS           | 0DLIFA_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/d4c734f0-f83e-461b-ab76-cbe01506a6e2   | IGM_PBBZKS-0DLIFA_20230515_normal.aln.cram.crai  | ad5375cb49477ccf8c7710d0a198ba8b | CRAI        |
|  783 | PBBZKS           | 0DLIFA_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f74f6e82-11c5-4205-9ad7-0e4e6ff03a8c   | IGM_PBBZKS-0DLIFA_20230515_normal.aln.cram       | 77b27e23a18f6e0edc6c98e08fa3b5ad | CRAM        |
|  784 | PBBZKS           | 0DLIF3_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d78485ed-f0be-4feb-a421-2f8aabbcba31   | IGM_PBBZKS-0DLIF3_20230509_merged_bam.bai        | 24bba291f158a5ad66aedabc661c0a23 | BAI         |
|  785 | PBBZKS           | 0DLIF3_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a217432a-16c1-42bb-80db-74a59db4a6f8   | IGM_PBBZKS-0DLIF3_20230509_bam.bai               | 463833844730fa8e83abdbe84a9edc67 | BAI         |
|  786 | PBBZKS           | 0DLIF3_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/9d37df41-538a-4052-b1db-c05ba9ac1144   | IGM_PBBZKS-0DLIF3_20230509_bam.bam               | af8a3e1a1c5f1bc1c4da7db3da916a4c | BAM         |
|  787 | PBBZKS           | 0DLIF3_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a2505c87-939c-4d3b-b6b5-33e71a8f65e7   | IGM_PBBZKS-0DLIF3_20230509_merged_bam.bam        | a3588f5dd035aade77fb9c4e93000e4a | BAM         |
|  788 | PBBWUR           | 0DK843_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/dc743618-5051-4ded-8274-6fa3d2c5e7b0   | IGM_PBBWUR-0DK843_20230413_tumor.aln.cram.crai   | 4f85b49c8f6b872ef0da6a74b2eca96f | CRAI        |
|  789 | PBBWUR           | 0DK843_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/4889ca38-90f3-4814-8d0a-f7ed49fdf594   | IGM_PBBWUR-0DK843_20230413_tumor.aln.cram        | a170f0f9cb4a32158ca7267a1e48f7f5 | CRAM        |
|  790 | PBBWUR           | 0DK83S_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/ada6cef4-c7c0-4fdc-bfb3-b1d9a14163f5   | IGM_PBBWUR-0DK83S_20230413_normal.aln.cram       | 2af66cb4646f652a002f96354be224fd | CRAM        |
|  791 | PBBWUR           | 0DK83S_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/bbd5fa4e-70cf-4273-a8a9-c43b1d10ca3f   | IGM_PBBWUR-0DK83S_20230413_normal.aln.cram.crai  | 3d6656f64cb77781972d7d546f4c566f | CRAI        |
|  792 | PBBWUR           | 0DK83Y_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a4b76deb-0194-48ed-9856-8b497f3defc6   | IGM_PBBWUR-0DK83Y_20230407_merged_bam.bai        | 0165e9495bb8a70296619fc1a04a0a7f | BAI         |
|  793 | PBBWUR           | 0DK83Y_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ed31a9a0-baf1-45bf-83de-f9aa8ec07036   | IGM_PBBWUR-0DK83Y_20230407_bam.bam               | 311b61b8467dff907b20a43bd18122a2 | BAM         |
|  794 | PBBWUR           | 0DK83Y_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/34f9b633-98a0-4e94-9ee6-11ad9fb7284b   | IGM_PBBWUR-0DK83Y_20230407_merged_bam.bam        | 35660cb0598aa5d95456530f783e6c52 | BAM         |
|  795 | PBBWUR           | 0DK83Y_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d089e5dd-4d9c-441b-bd29-87f412deb680   | IGM_PBBWUR-0DK83Y_20230407_bam.bai               | 90a6c9f9534f5d499a29e37851d9ef02 | BAI         |
|  796 | PBBXRV           | 0DK0AK_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/c5119e35-82d4-405f-ac7e-cae1c179d9e2   | IGM_PBBXRV-0DK0AK_20230331_tumor.aln.cram        | aee49e9313bf01be92bc4eaa2a7861c7 | CRAM        |
|  797 | PBBXRV           | 0DK0AK_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/a8a18abe-5032-49dd-b887-63901b9e3e27   | IGM_PBBXRV-0DK0AK_20230331_tumor.aln.cram.crai   | f15e7910813f7437b98c61bde98049a5 | CRAI        |
|  798 | PBBXRV           | 0DK0AD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/6fa658e4-1749-4326-b3ef-1cc6744af1d2   | IGM_PBBXRV-0DK0AD_20230330_merged_bam.bai        | 1268a5d287f13291ac8c0ed87ebc00e5 | BAI         |
|  799 | PBBXRV           | 0DK0AD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/eae18e1e-11da-4766-8fcc-3005ffadf0ed   | IGM_PBBXRV-0DK0AD_20230330_merged_bam.bam        | 420c53de77218b11d2c036c228930bfe | BAM         |
|  800 | PBBXRV           | 0DK0AD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/94b6a479-0d16-419c-a47e-b88ee7d56ae0   | IGM_PBBXRV-0DK0AD_20230330_bam.bam               | 6e2650ac91eb11528e466c4fac2e5abe | BAM         |
|  801 | PBBXRV           | 0DK0AD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3cb438b7-efd0-43b7-9771-c6644dd059a0   | IGM_PBBXRV-0DK0AD_20230330_bam.bai               | 30b67a69f5a1371b48cdf7a62e497e94 | BAI         |
|  802 | PBBXRV           | 0DK0AR_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/2f7c6891-2d10-442c-a66c-b8ddfae8d28b   | IGM_PBBXRV-0DK0AR_20230331_normal.aln.cram       | ff270b17e24c85ad352b7bdcdc6f53e4 | CRAM        |
|  803 | PBBXRV           | 0DK0AR_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/5aaa2df4-7d4f-40d3-ac6a-fb742a352a47   | IGM_PBBXRV-0DK0AR_20230331_normal.aln.cram.crai  | 18289b3d0c80c3ea3a2ce8479ad59262 | CRAI        |
|  804 | PBBYAL           | 0DKF9O_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/c1c03963-979e-4596-b5b2-b7bfc880ec6f   | IGM_PBBYAL-0DKF9O_20230418_normal.aln.cram       | 96c56b8768a51ccbf2ddb1ba63f7d1dc | CRAM        |
|  805 | PBBYAL           | 0DKF9O_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/22ef36b0-512d-4ddc-abb1-9b836f3b44f0   | IGM_PBBYAL-0DKF9O_20230418_normal.aln.cram.crai  | 78ea109c463635b6c1c790f9b221ddbd | CRAI        |
|  806 | PBBYAL           | 0DKFA2_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/f92a870f-f0fd-49d8-88b7-e3fedac3a86b   | IGM_PBBYAL-0DKFA2_20230418_tumor.aln.cram.crai   | 449a0a73ffeaf14ca61337287651dddf | CRAI        |
|  807 | PBBYAL           | 0DKFA2_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/520e97de-31e8-43bc-8c80-631d03a2e3dd   | IGM_PBBYAL-0DKFA2_20230418_tumor.aln.cram        | 8307dc8993ee66d11ebfe750091edd10 | CRAM        |
|  808 | PBBYAL           | 0DKF9V_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b763b1e3-57c4-4264-8a1a-935db14a728c   | IGM_PBBYAL-0DKF9V_20230410_merged_bam.bam        | 6d2efd5103105998667f6a85ba087f1b | BAM         |
|  809 | PBBYAL           | 0DKF9V_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/292e44a2-331e-4e24-8a77-d4ac965e1cfa   | IGM_PBBYAL-0DKF9V_20230410_bam.bam               | 6d881043670a598d694ab8633dc23a05 | BAM         |
|  810 | PBBYAL           | 0DKF9V_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/58c73562-2257-4b8e-9181-cf121fbcc003   | IGM_PBBYAL-0DKF9V_20230410_merged_bam.bai        | f88a84c715eda649425427fd45fb4aae | BAI         |
|  811 | PBBYAL           | 0DKF9V_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/01e51988-8c0c-4014-ba79-e8c81f56737c   | IGM_PBBYAL-0DKF9V_20230410_bam.bai               | a8404bbbcd29f78e8347d172741446ff | BAI         |
|  812 | PBBWDB           | 0DK963_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a68b5df1-c575-437e-82c8-178ff02a973e   | IGM_PBBWDB-0DK963_20230407_bam.bam               | 2d750efe66d5861e776a435a3c3b1e4a | BAM         |
|  813 | PBBWDB           | 0DK963_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/047b4465-fd81-43ff-9bad-5834cfa652b5   | IGM_PBBWDB-0DK963_20230407_bam.bai               | da9471ce24223c510ae5b4e4dce27e23 | BAI         |
|  814 | PBBWDB           | 0DK963_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8b0ecbca-3464-45ed-b983-ed01b3f76d10   | IGM_PBBWDB-0DK963_20230407_merged_bam.bai        | 10afeabbc42533a12e4102aaeac8a1c5 | BAI         |
|  815 | PBBWDB           | 0DK963_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/70717e3e-7a86-48db-a1f1-03140be2a05c   | IGM_PBBWDB-0DK963_20230407_merged_bam.bam        | 73b28e33e43903d43d316e8391c52f0f | BAM         |
|  816 | PBBWDB           | 0DJ6KK_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/8f614935-fdca-4965-81eb-8207a6efc4e3   | IGM_PBBWDB-0DJ6KK_20230314_tumor.aln.cram        | 30c78704cbe180941bacf0893056ae46 | CRAM        |
|  817 | PBBWDB           | 0DJ6KK_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/4edf3382-536f-47b2-acf8-532f87438191   | IGM_PBBWDB-0DJ6KK_20230314_tumor.aln.cram.crai   | f5b9af3859d40430db569422c1bc5130 | CRAI        |
|  818 | PBBWDB           | 0DJ6K0_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b7b47a20-e472-4bce-a167-09e5517f369e   | IGM_PBBWDB-0DJ6K0_20230314_normal.aln.cram.crai  | 6951f6915111c1c46ac847ac1e7aaf94 | CRAI        |
|  819 | PBBWDB           | 0DJ6K0_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/a45ec61b-0fb0-4384-9e5c-b7790a7e30b8   | IGM_PBBWDB-0DJ6K0_20230314_normal.aln.cram       | e24edffb78760e814bfdc3d72a8be4cb | CRAM        |
|  820 | PBBVKH           | 0DIOKE_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/8fa08912-bb39-4902-80db-187fd7d99b58   | IGM_PBBVKH-0DIOKE_20230308_tumor.aln.cram        | a0bd5eb88064dd89168072734e958681 | CRAM        |
|  821 | PBBVKH           | 0DIOKE_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/ec1648b3-8aa2-47ac-bd01-fa810adefe00   | IGM_PBBVKH-0DIOKE_20230308_tumor.aln.cram.crai   | 3da520fece444fd7166760720f418e6d | CRAI        |
|  822 | PBBVKH           | 0DIOK6_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a72c05cb-36cd-424b-9884-ff6fe0b1fc49   | IGM_PBBVKH-0DIOK6_20230224_merged_bam.bai        | 83a2fefc42d5d60a1e1d34eb7add5752 | BAI         |
|  823 | PBBVKH           | 0DIOK6_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e6aa5caf-d9cb-495c-94db-a15db4839e6c   | IGM_PBBVKH-0DIOK6_20230224_bam.bam               | 8add4a6cd3d049bd72f182773c8bbd4c | BAM         |
|  824 | PBBVKH           | 0DIOK6_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/fceca893-a619-4baa-bbf5-af1c22a41245   | IGM_PBBVKH-0DIOK6_20230224_bam.bai               | fda6f217e52d13bf5f17c0b544f3bc53 | BAI         |
|  825 | PBBVKH           | 0DIOK6_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/28071f14-35fd-4080-b0fc-a8adf29f7f8d   | IGM_PBBVKH-0DIOK6_20230224_merged_bam.bam        | e735ec195325ff5316a921d1baa7a4b1 | BAM         |
|  826 | PBBVKH           | 0DIOJX_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/d2f49d93-e3a8-4a5f-8661-81046aaa8b45   | IGM_PBBVKH-0DIOJX_20230308_normal.aln.cram.crai  | 925b10864c841615a51bc94a7d4a574e | CRAI        |
|  827 | PBBVKH           | 0DIOJX_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/def2607f-ac9c-4c3f-9175-6d2f2507e7aa   | IGM_PBBVKH-0DIOJX_20230308_normal.aln.cram       | 8a72a0b8ac0a30793102d7537858da19 | CRAM        |
|  828 | PBBXAT           | 0DJHEK_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f4ed3752-22bb-4337-a05d-073672bfc4d7   | IGM_PBBXAT-0DJHEK_20230321_bam.bam               | c3f16de75cbec29be8de2d9cf700d08c | BAM         |
|  829 | PBBXAT           | 0DJHEK_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/89232cf5-9971-44d9-bf03-fadbe75538c8   | IGM_PBBXAT-0DJHEK_20230321_merged_bam.bam        | 49cb3df37ccf737d92f821afa598c0c2 | BAM         |
|  830 | PBBXAT           | 0DJHEK_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2141d401-6d24-486f-a610-52e32a853e00   | IGM_PBBXAT-0DJHEK_20230321_bam.bai               | 7b5015a163ab656aae0714bf71bb9899 | BAI         |
|  831 | PBBXAT           | 0DJHEK_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/168d92ae-ee5b-4397-af3e-369831f8f8b3   | IGM_PBBXAT-0DJHEK_20230321_merged_bam.bai        | ee3dead215a6a8ee0ee4ac5c8dbc159a | BAI         |
|  832 | PBBXAT           | 0DJHG4_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/86da6ab8-cfbc-4546-adbb-3c269644250a   | IGM_PBBXAT-0DJHG4_20230319_tumor.aln.cram        | 899f6a3ae990f9257f7d60dd2a762066 | CRAM        |
|  833 | PBBXAT           | 0DJHG4_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/3c91aa08-94a6-4db4-a8d8-0f6681fa5e6b   | IGM_PBBXAT-0DJHG4_20230319_tumor.aln.cram.crai   | 1a4e5eb75eaf34c943c7e07f6dee5bac | CRAI        |
|  834 | PBBXAT           | 0DJHE2_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/ac8db617-23b6-4071-9f13-c1a73b485d8c   | IGM_PBBXAT-0DJHE2_20230319_normal.aln.cram       | a6fa0144a9dc9498198146753858f56d | CRAM        |
|  835 | PBBXAT           | 0DJHE2_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b74532a9-d977-488c-bc05-aa2b3a55f054   | IGM_PBBXAT-0DJHE2_20230319_normal.aln.cram.crai  | 615d4065f6e01ef5cc5ae0036b0d0e88 | CRAI        |
|  836 | PBBXGC           | 0DJO59_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4562273e-616c-443c-bc57-333d36227474   | IGM_PBBXGC-0DJO59_20230328_bam.bam               | e1bb2ca028ff4ce0568f2065a4f82bdc | BAM         |
|  837 | PBBXGC           | 0DJO59_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4563deb2-8309-4341-ae45-3265b5d2ba4e   | IGM_PBBXGC-0DJO59_20230328_merged_bam.bai        | ee924189abc3ffc10170067f9f6b01be | BAI         |
|  838 | PBBXGC           | 0DJO59_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/15214682-4864-42d5-88ba-fcffd840f9f2   | IGM_PBBXGC-0DJO59_20230328_merged_bam.bam        | f9ad2cab9eca78a66444ff39a4e1c646 | BAM         |
|  839 | PBBXGC           | 0DJO59_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f75a1d68-a8ff-4df6-8a9d-4efb5b4417b5   | IGM_PBBXGC-0DJO59_20230328_bam.bai               | 4f350c1f1c2d7127e068e624e844c9e4 | BAI         |
|  840 | PBBXGC           | 0DJO4U_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/1ec9cc79-da86-4ece-82e9-7130ed732305   | IGM_PBBXGC-0DJO4U_20230328_normal.aln.cram       | 6dcea6be191d4f7d64c33d82deb54896 | CRAM        |
|  841 | PBBXGC           | 0DJO4U_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f0442476-cbab-445d-ad72-7019e4297d5d   | IGM_PBBXGC-0DJO4U_20230328_normal.aln.cram.crai  | 7fb261c99121b88878c64cd633a6c901 | CRAI        |
|  842 | PBBXGC           | 0DJO5M_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/b98eca3e-2fdd-42aa-9401-db8d3b913f1c   | IGM_PBBXGC-0DJO5M_20230328_tumor.aln.cram.crai   | 0ce450047d9ee30eb955455d797acd62 | CRAI        |
|  843 | PBBXGC           | 0DJO5M_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/f59e7216-a77f-4b60-a14a-e539655195ce   | IGM_PBBXGC-0DJO5M_20230328_tumor.aln.cram        | 8951db2a83a91d30d30b19c018bbca8d | CRAM        |
|  844 | PBBVXS           | 0DIOKN_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/7deb7df6-8f76-40fe-91db-73189488bab4   | IGM_PBBVXS-0DIOKN_20230228_tumor.aln.cram        | 0ab8894a4065e54ffaab37bd5e384bc4 | CRAM        |
|  845 | PBBVXS           | 0DIOKN_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/a80a28e3-4164-4d87-a8ce-8ed2b950abaf   | IGM_PBBVXS-0DIOKN_20230228_tumor.aln.cram.crai   | e58b20939f76a8df209454566ac3154d | CRAI        |
|  846 | PBBVXS           | 0DIOK2_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/dde6669c-d53d-4188-9399-fa7bdf181aab   | IGM_PBBVXS-0DIOK2_20230228_normal.aln.cram.crai  | 33e68a6b5af0de3b198ad047a215aa56 | CRAI        |
|  847 | PBBVXS           | 0DIOK2_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/c5d43b44-3ac5-480e-b1a6-39c71d20f4a0   | IGM_PBBVXS-0DIOK2_20230228_normal.aln.cram       | 5f4a5424a151a1ef3fab595aecaff55e | CRAM        |
|  848 | PBBWAC           | 0DJGDY_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/0d03d139-650a-494f-863e-18ae7f77ed12   | IGM_PBBWAC-0DJGDY_20230320_tumor.aln.cram        | a0cd54ff0123e5bef620e87068f65178 | CRAM        |
|  849 | PBBWAC           | 0DJGDY_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/d1c7f087-9635-44b7-b6ca-0e3879739b6f   | IGM_PBBWAC-0DJGDY_20230320_tumor.aln.cram.crai   | cbac473695d05cdc653f3a2506952159 | CRAI        |
|  850 | PBBWAC           | 0DJG6W_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/42307a9e-fdfc-47d3-8cb0-dda6a01bd088   | IGM_PBBWAC-0DJG6W_20230320_normal.aln.cram.crai  | 195a5f8f54ce67902e17bdfb337febe3 | CRAI        |
|  851 | PBBWAC           | 0DJG6W_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/1d62bc21-d290-431e-9d41-3fab41815ccd   | IGM_PBBWAC-0DJG6W_20230320_normal.aln.cram       | 5bf408fddb157639b67733931c76b7ea | CRAM        |
|  852 | PBBWLY           | 0DJC3A_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/a39120e2-8fe7-4a7e-9fd8-c147fc0b33ea   | IGM_PBBWLY-0DJC3A_20230316_tumor.aln.cram.crai   | e996bd648348b78cd67f6b1422066001 | CRAI        |
|  853 | PBBWLY           | 0DJC3A_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/860112b6-b059-4f39-8845-48837ea94fcc   | IGM_PBBWLY-0DJC3A_20230316_tumor.aln.cram        | 92af8c6dfa8e22d3318c97bed6b00101 | CRAM        |
|  854 | PBBWLY           | 0DJC33_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2577694d-6fad-4dc3-b96a-e4fcf4dc178e   | IGM_PBBWLY-0DJC33_20230315_bam.bai               | ce8cfd84756a9e1e06cc25a1bae9c225 | BAI         |
|  855 | PBBWLY           | 0DJC33_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/aabb7ed6-7e86-49f5-a8a9-2fa94d910120   | IGM_PBBWLY-0DJC33_20230315_merged_bam.bam        | e48ce59bfe7da2f5c7ea3d5dd0480dd1 | BAM         |
|  856 | PBBWLY           | 0DJC33_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/04f721c8-ba7f-4874-9ba6-c9336a511664   | IGM_PBBWLY-0DJC33_20230315_merged_bam.bai        | 4feb49d4e8b2647917db9e8c7d62efeb | BAI         |
|  857 | PBBWLY           | 0DJC33_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/01f3b2be-335c-4458-b67b-78c9432cb391   | IGM_PBBWLY-0DJC33_20230315_bam.bam               | 6d22a5234bae6d3595f13ddf135a6cce | BAM         |
|  858 | PBBWLY           | 0DJBLQ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/847e6527-a925-40e4-9364-2bbaa00c0af6   | IGM_PBBWLY-0DJBLQ_20230316_normal.aln.cram.crai  | 27840076dfd3d03681894ff0fc1a24e0 | CRAI        |
|  859 | PBBWLY           | 0DJBLQ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/6289b5a4-9c84-4689-8abf-fb5f7244158c   | IGM_PBBWLY-0DJBLQ_20230316_normal.aln.cram       | d370f1f3c3610f1db243b2ae8452a741 | CRAM        |
|  860 | PBBWWF           | 0DJE3D_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/14b9f49b-f053-41e5-878b-a17c2357cc72   | IGM_PBBWWF-0DJE3D_20230316_tumor.aln.cram.crai   | 5d548e991fd093e6b75687249556bb45 | CRAI        |
|  861 | PBBWWF           | 0DJE3D_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/c6f12889-f3a5-4ddb-9748-6c14ebd98abd   | IGM_PBBWWF-0DJE3D_20230316_tumor.aln.cram        | ea8771d0fbb1e4b6936154f7921dac02 | CRAM        |
|  862 | PBBWWF           | 0DJE3J_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/9400cdb7-44a5-4430-aad2-f420d3949f8a   | IGM_PBBWWF-0DJE3J_20230316_normal.aln.cram       | 407778fd11db0365d497082e08c79246 | CRAM        |
|  863 | PBBWWF           | 0DJE3J_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/1e4726f6-fc0d-4375-ba59-873cd74c8263   | IGM_PBBWWF-0DJE3J_20230316_normal.aln.cram.crai  | faf0703b385d707e58bd0e477512d46f | CRAI        |
|  864 | PBBWWF           | 0DJE35_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2ad0b516-9aea-470f-911d-f58f19e6f0ba   | IGM_PBBWWF-0DJE35_20230313_bam.bai               | 86c3c7385435495c9a96c710e5e69665 | BAI         |
|  865 | PBBWWF           | 0DJE35_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/637c6c1f-284e-4181-a988-78de433336bb   | IGM_PBBWWF-0DJE35_20230313_merged_bam.bai        | 59e5ac9d2184ad458c1b7f4b22bca7de | BAI         |
|  866 | PBBWWF           | 0DJE35_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/33a0c13a-ec54-43d2-848c-f8e45c8e8314   | IGM_PBBWWF-0DJE35_20230313_bam.bam               | 3954afdb3c2671901183af413ebb801e | BAM         |
|  867 | PBBWWF           | 0DJE35_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a82870e1-c6f3-4925-8ae4-b215059adaab   | IGM_PBBWWF-0DJE35_20230313_merged_bam.bam        | 6b461b4ba4cce514f1d7868bdde2a4ea | BAM         |
|  868 | PBBWVB           | 0DJG8Q_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2baa9fd1-69ef-47e6-b0bb-bd17407045d9   | IGM_PBBWVB-0DJG8Q_20230313_bam.bai               | 93d01a8790271792875c47b612ce8d8e | BAI         |
|  869 | PBBWVB           | 0DJG8Q_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/668b2855-7ff8-4c52-95ae-4b541b74fa17   | IGM_PBBWVB-0DJG8Q_20230313_merged_bam.bam        | cd93bba94bc0cd9213d043e08d889e3c | BAM         |
|  870 | PBBWVB           | 0DJG8Q_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d3998fe1-bfb9-441a-9082-9e4464afb048   | IGM_PBBWVB-0DJG8Q_20230313_merged_bam.bai        | 2b8701bd7ee755f8aafe1cf0c0bb1a42 | BAI         |
|  871 | PBBWVB           | 0DJG8Q_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/36df34be-748d-44ed-b37c-7a8ea2093284   | IGM_PBBWVB-0DJG8Q_20230313_bam.bam               | 2349141d3d1f31fdbefcf74fe6906746 | BAM         |
|  872 | PBBWVB           | 0DJGE1_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/3f6495d8-61fe-4025-8034-e13a38e439a7   | IGM_PBBWVB-0DJGE1_20230320_tumor.aln.cram.crai   | a610b740928e506362afa03c9ce3e0cc | CRAI        |
|  873 | PBBWVB           | 0DJGE1_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/45a0a3b5-1fbf-41bd-8d01-2e1f051a22fb   | IGM_PBBWVB-0DJGE1_20230320_tumor.aln.cram        | d774956a10e5331baacd1cdbdc6c1ed0 | CRAM        |
|  874 | PBBWVB           | 0DJG70_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/0ca74053-ec6c-424d-a270-6de3d41c6dc5   | IGM_PBBWVB-0DJG70_20230320_normal.aln.cram.crai  | dba3ffe1cb6a6ae5bfb96a3a8acdc991 | CRAI        |
|  875 | PBBWVB           | 0DJG70_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/92ef5b1c-b142-466a-a8ff-aa5893d30c25   | IGM_PBBWVB-0DJG70_20230320_normal.aln.cram       | f66f10a336a893f9acc5916918937c2b | CRAM        |
|  876 | PBBVRV           | 0DII3B_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/75f494b4-6d45-4136-a045-0b67527ace3e   | IGM_PBBVRV-0DII3B_20230303_normal.aln.cram       | bca7b3ca2434ebf15a91aff091516207 | CRAM        |
|  877 | PBBVRV           | 0DII3B_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/3a3c7720-e8a6-4c07-af07-fb0c83196915   | IGM_PBBVRV-0DII3B_20230303_normal.aln.cram.crai  | 24bb95089b782257574d86a0c484115c | CRAI        |
|  878 | PBBVRV           | 0DII4C_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/de403a6b-607f-43c5-9911-b03d6976e1fa   | IGM_PBBVRV-0DII4C_20230303_tumor.aln.cram.crai   | de33bb27f10facca94a5fec6e258648f | CRAI        |
|  879 | PBBVRV           | 0DII4C_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/9892a2ed-ed73-4a1b-9fc1-5e7b861de512   | IGM_PBBVRV-0DII4C_20230303_tumor.aln.cram        | fc37c9d6b5a377be08faf9db9bee70ed | CRAM        |
|  880 | PBBVRV           | 0DII3N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/fefde1ab-6c7e-4d93-b19d-1911a720d691   | IGM_PBBVRV-0DII3N_20230218_merged_bam.bai        | ab6d310c7299614ffe998dc40abed6ff | BAI         |
|  881 | PBBVRV           | 0DII3N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3c0da32f-7c9d-4f1d-8cf8-175b3d969db9   | IGM_PBBVRV-0DII3N_20230218_bam.bam               | b02a86bb041a3fba109919d84ec5cac7 | BAM         |
|  882 | PBBVRV           | 0DII3N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/283392fa-9b3f-4729-9626-c10c1436b585   | IGM_PBBVRV-0DII3N_20230218_bam.bai               | e44a3ee7ea6d1c696faad3c7772a96cc | BAI         |
|  883 | PBBVRV           | 0DII3N_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ea88be34-b32c-4776-b7bc-793d23d7d0cd   | IGM_PBBVRV-0DII3N_20230218_merged_bam.bam        | e376dbadff3e5661ed5d54f9c448c98c | BAM         |
|  884 | PBBVUP           | 0DIMWR_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/ec1dca11-9c94-4beb-8979-7d35eef584ac   | IGM_PBBVUP-0DIMWR_20230308_normal.aln.cram.crai  | 22089cb3e6b70b812ab1134ee431c707 | CRAI        |
|  885 | PBBVUP           | 0DIMWR_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/43d167f3-ebcc-4f8d-8c7d-15fc60449845   | IGM_PBBVUP-0DIMWR_20230308_normal.aln.cram       | 47ed011fc4361511156f598b9e2c8c97 | CRAM        |
|  886 | PBBVUP           | 0DIMVW_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/609810d0-ec56-4fb9-a37b-47f22723d5b4   | IGM_PBBVUP-0DIMVW_20230223_merged_bam.bam        | 8bb034b2020f9bdd795451161b0d1ab3 | BAM         |
|  887 | PBBVUP           | 0DIMVW_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/bab6e409-0853-4fc8-bdf1-0658341f570d   | IGM_PBBVUP-0DIMVW_20230223_merged_bam.bai        | bf0458db41a6db8df9c2ea54d5596aec | BAI         |
|  888 | PBBVUP           | 0DIMVW_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/29acffd1-e8f0-4940-b8bd-7309edfed601   | IGM_PBBVUP-0DIMVW_20230223_bam.bai               | 740c7e1d24ee58228417ff03391bd5fe | BAI         |
|  889 | PBBVUP           | 0DIMVW_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/06c20f2b-c136-4c8a-806e-513e24c2cd7e   | IGM_PBBVUP-0DIMVW_20230223_bam.bam               | 19156910af68c59775a849dd29edbc19 | BAM         |
|  890 | PBBVUP           | 0DIMWU_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/3b626748-48c9-43cc-a0f3-08d50dc0e985   | IGM_PBBVUP-0DIMWU_20230308_tumor.aln.cram.crai   | e4041948e97c6f195394deed976ee262 | CRAI        |
|  891 | PBBVUP           | 0DIMWU_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/5f368ceb-88d2-401c-a9e0-36bbffaff1b2   | IGM_PBBVUP-0DIMWU_20230308_tumor.aln.cram        | f6c30db99adea164b839f77ef2fe633a | CRAM        |
|  892 | PBBVKF           | 0DI7SD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/49c36c11-82aa-499e-9b57-68880ee2d5c7   | IGM_PBBVKF-0DI7SD_20230212_merged_bam.bam        | 11b878c7e319befd4f4f3d40a57be08e | BAM         |
|  893 | PBBVKF           | 0DI7SD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4b77e0f1-a83c-4609-babf-b71fad4772aa   | IGM_PBBVKF-0DI7SD_20230212_bam.bai               | bfdcc00e5b1dcc3de097be34759e4bca | BAI         |
|  894 | PBBVKF           | 0DI7SD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/06d76acb-520b-4fad-b7f1-f74773b9d928   | IGM_PBBVKF-0DI7SD_20230212_merged_bam.bai        | bae8a0491c413cefff67a4ea250d4529 | BAI         |
|  895 | PBBVKF           | 0DI7SD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/cb0853a2-d808-488b-bf41-51727c75f4d7   | IGM_PBBVKF-0DI7SD_20230212_bam.bam               | aaa1b9fe5dce5686dd4b1f9ba70779b7 | BAM         |
|  896 | PBBVKF           | 0DI7VH_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/a64dfe6b-2ce8-426e-818c-cad1ccf62026   | IGM_PBBVKF-0DI7VH_20230209_normal.aln.cram.crai  | 4a2ee63cea3a98fbbae54d384a76e795 | CRAI        |
|  897 | PBBVKF           | 0DI7VH_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f433da62-b624-49bf-a622-8b6a46be9c63   | IGM_PBBVKF-0DI7VH_20230209_normal.aln.cram       | 890f71005d976eb9dfebc75a8520671d | CRAM        |
|  898 | PBBVKF           | 0DI7V0_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/81df4492-38db-4e4b-9385-140f8ff0628f   | IGM_PBBVKF-0DI7V0_20230209_tumor.aln.cram        | b95ef39e898be4f093425da3af7b9ba4 | CRAM        |
|  899 | PBBVKF           | 0DI7V0_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/3a677416-7a87-4418-8202-b4b1d0b1b58a   | IGM_PBBVKF-0DI7V0_20230209_tumor.aln.cram.crai   | 583f7165d5c2edd7b1ec4bca2f1ca625 | CRAI        |
|  900 | PBBUYK           | 0DHQDJ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/21b2efb0-79af-410b-a105-ee39c4da09f9   | IGM_PBBUYK-0DHQDJ_20230201_normal.aln.cram       | 05ce338f7b3a1436e61f7fa8e4b4d737 | CRAM        |
|  901 | PBBUYK           | 0DHQDJ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/ab4b4eb7-bb95-4bcf-ab48-eb2926bd13ef   | IGM_PBBUYK-0DHQDJ_20230201_normal.aln.cram.crai  | 5c2d22a7b28622c050e842950044ee3c | CRAI        |
|  902 | PBBUYK           | 0DHQFF_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/4e41ec67-3e34-4a53-bd70-f12ef2ecca3a   | IGM_PBBUYK-0DHQFF_20230201_tumor.aln.cram.crai   | 706bf9738b5747ad3388a2bb927807ab | CRAI        |
|  903 | PBBUYK           | 0DHQFF_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/2f032b63-781f-4276-8c27-c9ef9b6a2ff7   | IGM_PBBUYK-0DHQFF_20230201_tumor.aln.cram        | e9bc68e1eb80f3393ec675757f456921 | CRAM        |
|  904 | PBBUYK           | 0DHQE8_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3e3943f6-da22-4b22-a40d-912491eecec4   | IGM_PBBUYK-0DHQE8_20230127_bam.bam               | 7fd2158cf278cd62faaf0e90d6419892 | BAM         |
|  905 | PBBUYK           | 0DHQE8_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/98ec696d-5425-436d-8250-565083eb516d   | IGM_PBBUYK-0DHQE8_20230127_bam.bai               | c81dea190ce17d5d0c375ace68f3a261 | BAI         |
|  906 | PBBUYK           | 0DHQE8_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/cb337be1-0663-4ded-b7ea-4c7182c15be9   | IGM_PBBUYK-0DHQE8_20230127_merged_bam.bam        | b39804498f42c9d2fec7b5f718369a5f | BAM         |
|  907 | PBBUYK           | 0DHQE8_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d8362f62-4227-4ba4-b107-e5cde429e407   | IGM_PBBUYK-0DHQE8_20230127_merged_bam.bai        | af2deb6eb0bd0d8da15f3167fe93ac01 | BAI         |
|  908 | PBBSKD           | 0DFYQ7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c1928447-7d28-4bbd-b953-4974cc837d52   | IGM_PBBSKD-0DFYQ7_20221213_bam.bam               | 920e18cb3a138721a1811a7966b63610 | BAM         |
|  909 | PBBSKD           | 0DFYQ7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/58276304-bf45-414c-bbfc-a5b1060df343   | IGM_PBBSKD-0DFYQ7_20221213_merged_bam.bam        | 510edb5d250d94ecacaae6864983b1da | BAM         |
|  910 | PBBSKD           | 0DFYQ7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/5086ff88-2923-451f-83e7-4a23458ffc9b   | IGM_PBBSKD-0DFYQ7_20221213_bam.bai               | 1935bb2bf23f9979233fadbe5ac5c7dc | BAI         |
|  911 | PBBSKD           | 0DFYQ7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c2e4bb66-2641-4218-8812-bb1ee9f073f9   | IGM_PBBSKD-0DFYQ7_20221213_merged_bam.bai        | c91f1687f9f4c34cbbd69902266cdf93 | BAI         |
|  912 | PBBSKD           | 0DG0G9_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/e37af897-202e-48da-ba1b-beac279419cd   | IGM_PBBSKD-0DG0G9_20221229_tumor.aln.cram.crai   | 7520a8bd614a769cf1c49a7a19942cd0 | CRAI        |
|  913 | PBBSKD           | 0DG0G9_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/1e5bbcaa-e061-4926-b44b-42ee545fb714   | IGM_PBBSKD-0DG0G9_20221229_tumor.aln.cram        | edd8b079bebdf2ca785245bf928fdc6c | CRAM        |
|  914 | PBBSKD           | 0DFYPK_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/dc8a971a-e2d8-4bcd-9b61-3b41d35bd45e   | IGM_PBBSKD-0DFYPK_20221229_normal.aln.cram       | da0124ba5226d0306f108da78ae852d5 | CRAM        |
|  915 | PBBSKD           | 0DFYPK_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b69cea20-e495-4907-9c99-33ee00dfda78   | IGM_PBBSKD-0DFYPK_20221229_normal.aln.cram.crai  | cb471c8514c9e7ad5d8df4ffc19c20fe | CRAI        |
|  916 | PBBSRT           | 0DFXEZ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/cedc6693-2424-442c-8026-9fb434e97b88   | IGM_PBBSRT-0DFXEZ_20221213_normal.aln.cram       | 29b08946c83809a3eebdbb3b11150e39 | CRAM        |
|  917 | PBBSRT           | 0DFXEZ_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b1256e5c-b92f-49e1-a857-3c2493985e54   | IGM_PBBSRT-0DFXEZ_20221213_normal.aln.cram.crai  | c9ab87f14fa2b4f6bfd9119c15139ddf | CRAI        |
|  918 | PBBSRT           | 0DFXFG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/40415811-1d55-4657-ad5a-88473aec8822   | IGM_PBBSRT-0DFXFG_20221217_merged_bam.bam        | e4d96578f0ceda58c36fdd94293c6bfc | BAM         |
|  919 | PBBSRT           | 0DFXFG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/9145aac0-01d2-436d-a383-c6eeb8fcd291   | IGM_PBBSRT-0DFXFG_20221217_merged_bam.bai        | d2e421c207f49da95c94fcde289b0f8f | BAI         |
|  920 | PBBSRT           | 0DFXFG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7fd02744-622a-4051-a89d-690c11f0cf65   | IGM_PBBSRT-0DFXFG_20221217_bam.bam               | 6b3112971b4ef7f8aba68cc890b88121 | BAM         |
|  921 | PBBSRT           | 0DFXFG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/09d3ab51-7d54-4baa-b394-6cc9e1a8fb53   | IGM_PBBSRT-0DFXFG_20221217_bam.bai               | c025ce792515e0500a140ae175133b06 | BAI         |
|  922 | PBBSRT           | 0DFXH4_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/2c6b8611-1da9-4f3e-bc8a-fcc5fcd9f3c3   | IGM_PBBSRT-0DFXH4_20221213_tumor.aln.cram.crai   | 2727da514745129b1ca6a04b683fa0f4 | CRAI        |
|  923 | PBBSRT           | 0DFXH4_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/721192c4-d7e4-4b62-b76a-f25ffa98f041   | IGM_PBBSRT-0DFXH4_20221213_tumor.aln.cram        | 40118c99bca53fb601c263862e343d21 | CRAM        |
|  924 | PBBUKM           | 0DHX36_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/a6497c0d-562b-40f0-8473-2082ac97e4d9   | IGM_PBBUKM-0DHX36_20230217_tumor.aln.cram        | e3cba18dc4ab1fd10ad6d29fe5b5a62a | CRAM        |
|  925 | PBBUKM           | 0DHX36_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/fe04074c-712d-4419-a3dc-3eaacdce54b8   | IGM_PBBUKM-0DHX36_20230217_tumor.aln.cram.crai   | 7fd3d930d060a4e121b20ac770c8b850 | CRAI        |
|  926 | PBBUKM           | 0DHX1V_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/81cc6e38-22b6-4dde-aab4-a86c175c839d   | IGM_PBBUKM-0DHX1V_20230217_normal.aln.cram       | d40b6933248b8569f48afe0507aee37f | CRAM        |
|  927 | PBBUKM           | 0DHX1V_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/e6e75c4d-fe90-471c-a2f2-b9fbc23bda26   | IGM_PBBUKM-0DHX1V_20230217_normal.aln.cram.crai  | 9ef1979e8d270d41eef1a2f18efd7c3c | CRAI        |
|  928 | PBBTVK           | 0DH28Q_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0e92b842-12d1-4bb0-b9a2-a0bd314703dd   | IGM_PBBTVK-0DH28Q_20230110_merged_bam.bam        | 2d6a0f75824f65621f464981fdd4106a | BAM         |
|  929 | PBBTVK           | 0DH28Q_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/432a2b81-954c-41c9-9dab-a9e1e98263f9   | IGM_PBBTVK-0DH28Q_20230110_bam.bam               | 55ede4d8184a0346581f4281f78a1b81 | BAM         |
|  930 | PBBTVK           | 0DH28Q_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/96b0204b-c956-4463-8de2-fbaed0539c6d   | IGM_PBBTVK-0DH28Q_20230110_bam.bai               | 4622c91c7e2c0bf2ee6899dc1355cc09 | BAI         |
|  931 | PBBTVK           | 0DH28Q_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/6e23886a-3711-4908-aed8-ceb47e54805d   | IGM_PBBTVK-0DH28Q_20230110_merged_bam.bai        | 49fbd8f945dfe3e01de3436a54bb3b5c | BAI         |
|  932 | PBBTVK           | 0DH27J_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/c176b771-aa9b-4074-880e-79e2439e98ef   | IGM_PBBTVK-0DH27J_20230122_normal.aln.cram       | bf1ff0536591cab8748f6c3b391f7d47 | CRAM        |
|  933 | PBBTVK           | 0DH27J_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/0111b979-7c48-435b-a965-151fe5e17ddd   | IGM_PBBTVK-0DH27J_20230122_normal.aln.cram.crai  | 4c3362c31cbd399c97d7f4594afd9f9c | CRAI        |
|  934 | PBBTVK           | 0DH293_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/dc35de68-7b7c-4294-a1ac-399f765a760d   | IGM_PBBTVK-0DH293_20230122_tumor.aln.cram.crai   | 9d07c0cc767ea4ae3de158a1558b5b76 | CRAI        |
|  935 | PBBTVK           | 0DH293_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/0d804312-6423-4921-93ac-dde8e6d0ad3b   | IGM_PBBTVK-0DH293_20230122_tumor.aln.cram        | 90d681b43618e96730e46dbc35fab181 | CRAM        |
|  936 | PBBTLT           | 0DGOB6_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/41cc23d2-2871-4af6-a47c-5d86db8a16f3   | IGM_PBBTLT-0DGOB6_20230112_normal.aln.cram       | a86f6e2383ec65c88e9bb46323354025 | CRAM        |
|  937 | PBBTLT           | 0DGOB6_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/6302229e-1783-4d58-b00e-8b44a8838608   | IGM_PBBTLT-0DGOB6_20230112_normal.aln.cram.crai  | d45fbc822151d564b0d6350ad24f27f6 | CRAI        |
|  938 | PBBTLT           | 0DGOAU_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/193dd125-500a-434c-ad8d-fc164408980f   | IGM_PBBTLT-0DGOAU_20230112_tumor.aln.cram.crai   | e586534a3e7cb43d5c07aab2a5c7d7d8 | CRAI        |
|  939 | PBBTLT           | 0DGOAU_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/ac37d366-a8e4-4aa3-8e72-fe784042f31d   | IGM_PBBTLT-0DGOAU_20230112_tumor.aln.cram        | 5b4682e3c3ebdb62c10a5b3344a6a17d | CRAM        |
|  940 | PBBTLT           | 0DGO9Y_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/de58d389-9ec0-4c54-9dd0-91dc37e1e821   | IGM_PBBTLT-0DGO9Y_20221231_bam.bam               | a00c98c1fe5bb91a87f06eeaab8925bf | BAM         |
|  941 | PBBTLT           | 0DGO9Y_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/35b4065f-3a77-4442-9619-c06b98d41132   | IGM_PBBTLT-0DGO9Y_20221231_merged_bam.bai        | 96416d660a97ad5262dcf8cc02e5369e | BAI         |
|  942 | PBBTLT           | 0DGO9Y_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a0a51091-1090-401b-a539-8e4a25bf7c0f   | IGM_PBBTLT-0DGO9Y_20221231_merged_bam.bam        | a367db10dc091c98a7c77a38680056bb | BAM         |
|  943 | PBBTLT           | 0DGO9Y_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8d53558c-c066-4633-ae7d-2f74c20fc07b   | IGM_PBBTLT-0DGO9Y_20221231_bam.bai               | 449426ff9744f08c8b69f2f3d1fcc7e5 | BAI         |
|  944 | PBBTRI           | 0DGOBC_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f09c270a-d1ef-40de-b002-6bac0a92ec13   | IGM_PBBTRI-0DGOBC_20230106_normal.aln.cram       | b7c2077ae1e3f9a8e189fca5142e8830 | CRAM        |
|  945 | PBBTRI           | 0DGOBC_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/370b129c-eb41-4394-bee3-971cbd8fbc1c   | IGM_PBBTRI-0DGOBC_20230106_normal.aln.cram.crai  | 9662d71b5a2e5d7b122c873f55604432 | CRAI        |
|  946 | PBBTRI           | 0DGOA1_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c9419552-8483-4c56-9bba-f290c1166201   | IGM_PBBTRI-0DGOA1_20221229_bam.bai               | 5a0b3d8ae52cc739885b9de4f6d19529 | BAI         |
|  947 | PBBTRI           | 0DGOA1_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/84366436-745e-429f-8d19-2aa859d48ed8   | IGM_PBBTRI-0DGOA1_20221229_merged_bam.bam        | 279d1b7c230c749f5b4794fc8f9a1937 | BAM         |
|  948 | PBBTRI           | 0DGOA1_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8ebf9661-c6a1-4d01-a52b-1af12917fb91   | IGM_PBBTRI-0DGOA1_20221229_merged_bam.bai        | 0536d29f1d43509b4ef827d8368b2748 | BAI         |
|  949 | PBBTRI           | 0DGOA1_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/204cd8ab-712b-48d3-b003-1d243df1a64b   | IGM_PBBTRI-0DGOA1_20221229_bam.bam               | c0a1236e54d8ae8056ba49b40fdd6720 | BAM         |
|  950 | PBBTRI           | 0DGOAP_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/93002ddf-ffd3-40fa-9eb6-dd5319de8a6d   | IGM_PBBTRI-0DGOAP_20230106_tumor.aln.cram        | 994466f440ec0e4c04893e54af38dd31 | CRAM        |
|  951 | PBBTRI           | 0DGOAP_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/2235b37f-540c-4090-bc25-eb3464145107   | IGM_PBBTRI-0DGOAP_20230106_tumor.aln.cram.crai   | 66d1d84ad09bff4a6320f838eb323960 | CRAI        |
|  952 | PBBTAY           | 0DGB9M_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3deefc94-97e3-47fd-b1d1-2377f68eba1e   | IGM_PBBTAY-0DGB9M_20221222_bam.bai               | 3641539600549ec74be81cd92da85c82 | BAI         |
|  953 | PBBTAY           | 0DGB9M_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/53db0ef3-b2cf-4b52-9fa7-ff8bbc3d9f4d   | IGM_PBBTAY-0DGB9M_20221222_merged_bam.bai        | a7651bc56785a8ec9e21b25b396c680f | BAI         |
|  954 | PBBTAY           | 0DGB9M_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/286adf10-72f2-491f-ad8a-095c7b33b21f   | IGM_PBBTAY-0DGB9M_20221222_bam.bam               | 61ce68f24088512d53290da8188bb478 | BAM         |
|  955 | PBBTAY           | 0DGB9M_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/9accb6eb-8744-42b9-9e7f-de390bb5aaf5   | IGM_PBBTAY-0DGB9M_20221222_merged_bam.bam        | 9b8e68a586bf9914ae31e5dbfce16139 | BAM         |
|  956 | PBBTAY           | 0DGB91_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/8c3e9c8c-58cc-421d-885b-c3a108efa0f8   | IGM_PBBTAY-0DGB91_20230109_normal.aln.cram.crai  | 06f0787e1847373213c26fec2811b990 | CRAI        |
|  957 | PBBTAY           | 0DGB91_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f8c49e0a-9cc3-44da-abf2-5662260a7e5a   | IGM_PBBTAY-0DGB91_20230109_normal.aln.cram       | b16f99b1475b58cd7d197b6e6641dff1 | CRAM        |
|  958 | PBBTAY           | 0DGBAC_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/3de9ad19-ace9-426c-87f3-b91f79618ac8   | IGM_PBBTAY-0DGBAC_20230109_tumor.aln.cram.crai   | 58b359e4db2ae140e469016353061122 | CRAI        |
|  959 | PBBTAY           | 0DGBAC_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/0a066c78-9a3c-4c40-8366-e1def9541d33   | IGM_PBBTAY-0DGBAC_20230109_tumor.aln.cram        | c778157548c21cec25c3059713691c3f | CRAM        |
|  960 | PBBSRD           | 0DFWC9_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/c8ec077c-f197-4d41-9725-8cbb8895c3d6   | IGM_PBBSRD-0DFWC9_20221211_normal.aln.cram.crai  | ed5adc9c511699916c2398a9dfb44c19 | CRAI        |
|  961 | PBBSRD           | 0DFWC9_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/0094d4bc-426d-4373-8679-e3fffa5f79c2   | IGM_PBBSRD-0DFWC9_20221211_normal.aln.cram       | 565bce499e00dd574b01ab3748771860 | CRAM        |
|  962 | PBBSRD           | 0DFWC7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/48742bb9-dcbb-4b56-babd-cc3c47f7d0ee   | IGM_PBBSRD-0DFWC7_20221215_merged_bam.bai        | 2e8ed0e604b4f7810f92af30ddba1888 | BAI         |
|  963 | PBBSRD           | 0DFWC7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/35514544-e878-4c5c-8bed-4f26f90e29d3   | IGM_PBBSRD-0DFWC7_20221215_bam.bam               | 4afc3abd1fbbbbca158add49dd7abd94 | BAM         |
|  964 | PBBSRD           | 0DFWC7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/6f4042bd-9240-4252-8eb6-f4a67dc9bbc2   | IGM_PBBSRD-0DFWC7_20221215_merged_bam.bam        | 5ec568d7d13d1bef1bcc8f1af31e4a3b | BAM         |
|  965 | PBBSRD           | 0DFWC7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4e8b1837-4ded-41bb-bfd1-7457fb40b912   | IGM_PBBSRD-0DFWC7_20221215_bam.bai               | de0b5ee7f165fe680e513b4759a97af3 | BAI         |
|  966 | PBBSRD           | 0DFWC8_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/7f9ac37f-654a-461a-8670-8b280d550ced   | IGM_PBBSRD-0DFWC8_20221211_tumor.aln.cram        | 1640b4cd4dc79e4900b00cd622236f9c | CRAM        |
|  967 | PBBSRD           | 0DFWC8_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/3892838d-b528-4040-8b6d-5b6b6db66d68   | IGM_PBBSRD-0DFWC8_20221211_tumor.aln.cram.crai   | ed0b2558954af97a87bdcd31a0d0a949 | CRAI        |
|  968 | PBBTSW           | 0DHBCT_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/69fdefe7-23ce-4a17-9571-b2a1daa175bb   | IGM_PBBTSW-0DHBCT_20230126_normal.aln.cram.crai  | c3af46031c1eececb7cdc219365710ad | CRAI        |
|  969 | PBBTSW           | 0DHBCT_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/27173999-f894-4fcb-a6b4-b6e7d1980f9a   | IGM_PBBTSW-0DHBCT_20230126_normal.aln.cram       | b57699c92a9342ea926f6c2aa6f91887 | CRAM        |
|  970 | PBBTSW           | 0DHBD2_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/9d14d8fa-40bb-4661-9e30-30b72da4ffe8   | IGM_PBBTSW-0DHBD2_20230126_tumor.aln.cram        | 73790190d35ca799bd8c34c94fe44d1e | CRAM        |
|  971 | PBBTSW           | 0DHBD2_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/b662e1cf-497b-4ed7-a3ff-99263eb159c8   | IGM_PBBTSW-0DHBD2_20230126_tumor.aln.cram.crai   | e3dfdb218fe1820aca14a853f66b7f56 | CRAI        |
|  972 | PBBTSD           | 0DGQAQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/32b28bab-8cb6-4ab1-9f32-202afbde12ae   | IGM_PBBTSD-0DGQAQ_20221229_merged_bam.bam        | 2c4598262e1ba5984e5c416212743db3 | BAM         |
|  973 | PBBTSD           | 0DGQAQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b614e83d-269e-429a-8297-d57a0d664625   | IGM_PBBTSD-0DGQAQ_20221229_bam.bam               | 568f6402ef25a9ad4d9b082cd0b968c0 | BAM         |
|  974 | PBBTSD           | 0DGQAQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f30d4d15-779a-4364-99f6-d85e2787e896   | IGM_PBBTSD-0DGQAQ_20221229_bam.bai               | 455efe5adcdcbc090afe31410037fa2f | BAI         |
|  975 | PBBTSD           | 0DGQAQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/bc5ff66a-3e05-4757-9915-277a06a86dab   | IGM_PBBTSD-0DGQAQ_20221229_merged_bam.bai        | 06361226e5c52fe0f1b5ac510dc3a469 | BAI         |
|  976 | PBBTSD           | 0DGQC2_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/3ca16067-b5fb-4733-b838-567251003b94   | IGM_PBBTSD-0DGQC2_20230105_tumor.aln.cram        | 43371e3076d9ea68c52f16def8f10b67 | CRAM        |
|  977 | PBBTSD           | 0DGQC2_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/b7857016-e755-44cb-baef-70229dee0f96   | IGM_PBBTSD-0DGQC2_20230105_tumor.aln.cram.crai   | 5721fbf6292a227c08d60282a77da128 | CRAI        |
|  978 | PBBTSD           | 0DGQAG_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/7ff1f46f-7357-46c5-95ec-727a935d07e0   | IGM_PBBTSD-0DGQAG_20230105_normal.aln.cram.crai  | 2f9207ff072568a1fa787a5af3cf1fde | CRAI        |
|  979 | PBBTSD           | 0DGQAG_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/6227bbd7-954b-442e-9897-59de0e396c63   | IGM_PBBTSD-0DGQAG_20230105_normal.aln.cram       | 2ccaf6a2db99f5fe8da55e93cee74482 | CRAM        |
|  980 | PBBSZR           | 0DG5A5_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/fb15cd45-a3eb-4bc8-b79b-73e7f6342da6   | IGM_PBBSZR-0DG5A5_20221220_merged_bam.bam        | 9a131f527fff650a312be0895a97e4ab | BAM         |
|  981 | PBBSZR           | 0DG5A5_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/9864ad9c-6700-41d3-93dc-c31132fa8226   | IGM_PBBSZR-0DG5A5_20221220_bam.bam               | e07654c359724b70d2dd325cd320bdbb | BAM         |
|  982 | PBBSZR           | 0DG5A5_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d322c936-40ed-4750-a8cb-f49d1fcd290f   | IGM_PBBSZR-0DG5A5_20221220_bam.bai               | 7b5ff9d16a28dd6ee104d892a141cb7b | BAI         |
|  983 | PBBSZR           | 0DG5A5_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c780eec4-f722-48f1-87db-abcaf70efb85   | IGM_PBBSZR-0DG5A5_20221220_merged_bam.bai        | 12ce25280529e7ff5af850386c53f1d2 | BAI         |
|  984 | PBBSZR           | 0DG5AD_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/4dde735e-0624-4fdf-a324-448961ed3b10   | IGM_PBBSZR-0DG5AD_20221217_normal.aln.cram.crai  | debb5ba2252effa89193251c5facccd6 | CRAI        |
|  985 | PBBSZR           | 0DG5AD_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/906ebd62-1a7b-4170-9420-c9e0c9522a84   | IGM_PBBSZR-0DG5AD_20221217_normal.aln.cram       | fbd7c25c71f044f6354da0424ae13935 | CRAM        |
|  986 | PBBSZR           | 0DG5AN_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/81235246-afca-4df7-a5e8-c6c3ab90926c   | IGM_PBBSZR-0DG5AN_20221217_tumor.aln.cram.crai   | 00011ce8abef9ac344b8487836998dc9 | CRAI        |
|  987 | PBBSZR           | 0DG5AN_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/0aca5332-1d4a-42e5-8b28-2a524ce2ac84   | IGM_PBBSZR-0DG5AN_20221217_tumor.aln.cram        | 8a8ef2558247540cb3c3487445b009bc | CRAM        |
|  988 | PBBTTG           | 0DGO9W_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/39e518d8-62f0-47a9-b71e-dc02896a4572   | IGM_PBBTTG-0DGO9W_20221229_merged_bam.bai        | cd26891d75a2de783ddc3d7157616fa8 | BAI         |
|  989 | PBBTTG           | 0DGO9W_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c743f4e5-a617-42fb-b3e3-c9be198efadb   | IGM_PBBTTG-0DGO9W_20221229_bam.bai               | 4f1a24194b269f02224d37f99461cbc1 | BAI         |
|  990 | PBBTTG           | 0DGO9W_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/db36551e-dd71-417a-9571-81f3d8fe1a7f   | IGM_PBBTTG-0DGO9W_20221229_merged_bam.bam        | 46306d000ffa48c8650573893f598971 | BAM         |
|  991 | PBBTTG           | 0DGO9W_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0a54c94f-1578-4d74-b8c6-9d9cd7254e9f   | IGM_PBBTTG-0DGO9W_20221229_bam.bam               | 994a0cae8f0fa40a2461f4a63789fceb | BAM         |
|  992 | PBBTTG           | 0DGOBD_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/131fa2ed-db5b-4b8f-b101-fa6f1a4b701f   | IGM_PBBTTG-0DGOBD_20221229_normal.aln.cram.crai  | d246e6b2bbd4ed4606e4cc8637e2d1e2 | CRAI        |
|  993 | PBBTTG           | 0DGOBD_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/9fa9e830-a004-48e6-938c-c6627efaac90   | IGM_PBBTTG-0DGOBD_20221229_normal.aln.cram       | 68de1aff91cf176af3466148a6c092fa | CRAM        |
|  994 | PBBTTG           | 0DGOAS_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/3bcd624f-9fc8-4930-bb4f-71bba36d2510   | IGM_PBBTTG-0DGOAS_20221229_tumor.aln.cram.crai   | 751475c452720d83696306740d8b5b11 | CRAI        |
|  995 | PBBTTG           | 0DGOAS_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/16d34917-015e-4b43-b598-8acc148e0977   | IGM_PBBTTG-0DGOAS_20221229_tumor.aln.cram        | bf84fd7faf1a06c522aac86a700ba2bf | CRAM        |
|  996 | PBBSXH           | 0DFYQ9_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7a5e1e8e-5a5d-4b52-a021-9947c58ab5ff   | IGM_PBBSXH-0DFYQ9_20221213_merged_bam.bai        | f078f4409b360856e1a4eeafc2895adf | BAI         |
|  997 | PBBSXH           | 0DFYQ9_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a6338f48-394b-4f0f-871b-3665e823e36f   | IGM_PBBSXH-0DFYQ9_20221213_bam.bai               | ba608395f64c2c93812a6da7bc6e85b7 | BAI         |
|  998 | PBBSXH           | 0DFYQ9_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/226a65f1-f570-4a6b-aa0e-fa165f9e09f4   | IGM_PBBSXH-0DFYQ9_20221213_bam.bam               | 66b3089c73bfa949bd2b8169ddb43df8 | BAM         |
|  999 | PBBSXH           | 0DFYQ9_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3ec47483-5d4f-4c67-84a3-f59e214e4276   | IGM_PBBSXH-0DFYQ9_20221213_merged_bam.bam        | 0bc2d8bb401428e1aa6cda4264c6d040 | BAM         |
| 1000 | PBBSXH           | 0DFYPG_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/abd32984-d4a4-45af-91df-ae4143e605d0   | IGM_PBBSXH-0DFYPG_20221215_normal.aln.cram       | a2814eb21b377da6810e6a651d797e9d | CRAM        |
| 1001 | PBBSXH           | 0DFYPG_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b1c0beac-10c2-4bad-9951-6622ea39e0ba   | IGM_PBBSXH-0DFYPG_20221215_normal.aln.cram.crai  | a561aad4c3fc5b067fcdd872ef1d5110 | CRAI        |
| 1002 | PBBSXH           | 0DFYQP_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/77c6fdcc-d5f5-4a86-bbe7-3a27b719663d   | IGM_PBBSXH-0DFYQP_20221215_tumor.aln.cram        | b5f606c97ed91c4a720dfd15f52bdcdd | CRAM        |
| 1003 | PBBSXH           | 0DFYQP_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/d8fe86d1-d9d2-4e3c-b917-0be817841f06   | IGM_PBBSXH-0DFYQP_20221215_tumor.aln.cram.crai   | 8d1c1d0cda1c3c5995d9b098f19177dc | CRAI        |
| 1004 | PBBNSW           | 0DDRN9_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/a2af6b56-62a4-4eb2-ac4c-6199243c7984   | IGM_PBBNSW-0DDRN9_20221010_normal.aln.cram.crai  | 945ec1382ae27ba3654fb34367120abe | CRAI        |
| 1005 | PBBNSW           | 0DDRN9_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/cee7ec00-0f0d-44d3-aee5-899ce3aca92f   | IGM_PBBNSW-0DDRN9_20221010_normal.aln.cram       | 62349daacf397aa367b45d70f62c132b | CRAM        |
| 1006 | PBBNSW           | 0DDRN7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/dc1122d9-4f8e-4daa-982a-de41cfb20126   | IGM_PBBNSW-0DDRN7_20221007_bam.bai               | 5ff77379071106cb19d83d6a0cb1b532 | BAI         |
| 1007 | PBBNSW           | 0DDRN7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/18818f8e-75ae-4860-96ba-42b1e67dbf37   | IGM_PBBNSW-0DDRN7_20221007_bam.bam               | bb6c86ae292af3364fe0c38c3fb93341 | BAM         |
| 1008 | PBBNSW           | 0DDRN7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/55152fca-e13d-4e7c-91be-1a577aa42323   | IGM_PBBNSW-0DDRN7_20221007_merged_bam.bam        | a48d10d01a3e94ea3fe9a8686b757d52 | BAM         |
| 1009 | PBBNSW           | 0DDRN7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/68a8c1d4-726c-4d56-9cbc-782c5b7e09c5   | IGM_PBBNSW-0DDRN7_20221007_merged_bam.bai        | 333ced2807ceb29f0414c682ef623a21 | BAI         |
| 1010 | PBBNSW           | 0DDRN8_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/e9d2e98b-a204-4800-810f-5c39f0db51c1   | IGM_PBBNSW-0DDRN8_20221010_tumor.aln.cram        | 3e582186ddb3f305f3f7ed063c5d8ac6 | CRAM        |
| 1011 | PBBNSW           | 0DDRN8_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/c032f1d4-721a-49e5-8909-6f81d39a396f   | IGM_PBBNSW-0DDRN8_20221010_tumor.aln.cram.crai   | 69aef22239cda9f9b2093fa211175ff3 | CRAI        |
| 1012 | PBBPGJ           | 0DDTDV_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/3c6b40c4-d307-4b1b-be53-8747aea00cc2   | IGM_PBBPGJ-0DDTDV_20221012_tumor.aln.cram        | 88e0d2a3285fab739620fd79590fada9 | CRAM        |
| 1013 | PBBPGJ           | 0DDTDV_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/6063eb31-0aa6-4361-ad7d-d8abfd8b1ef9   | IGM_PBBPGJ-0DDTDV_20221012_tumor.aln.cram.crai   | bf328e3e5a7a0bdd483328c842729bd2 | CRAI        |
| 1014 | PBBPGJ           | 0DDTDG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0fac4898-666d-4de3-8748-ddddedbba8a0   | IGM_PBBPGJ-0DDTDG_20221012_bam.bai               | 083e6e740eee00a12d0102b41d80c687 | BAI         |
| 1015 | PBBPGJ           | 0DDTDG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4c4be2d9-4a65-43d4-9fa5-c4e7517ec81b   | IGM_PBBPGJ-0DDTDG_20221012_merged_bam.bam        | b9954963d7c6141d7b3b1d9675f12257 | BAM         |
| 1016 | PBBPGJ           | 0DDTDG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2dffe353-c33a-437e-9677-1b4a074ae024   | IGM_PBBPGJ-0DDTDG_20221012_bam.bam               | 0e83a89415364f7f714a2e1d7f0623f9 | BAM         |
| 1017 | PBBPGJ           | 0DDTDG_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/5d2d6862-9d3f-4e17-928c-b1f78ea9b0cf   | IGM_PBBPGJ-0DDTDG_20221012_merged_bam.bai        | 9140c68a1dcf17bc1de9c848728b731b | BAI         |
| 1018 | PBBPGJ           | 0DDTDW_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/07f22c3d-21bc-4f7a-b2a6-538979491547   | IGM_PBBPGJ-0DDTDW_20221012_normal.aln.cram.crai  | 665f8e29e0a3cdd401df2b3d674ac775 | CRAI        |
| 1019 | PBBPGJ           | 0DDTDW_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/dde03d9b-6177-4cb9-b0e9-9603d5995065   | IGM_PBBPGJ-0DDTDW_20221012_normal.aln.cram       | e266b2ca3becff334b868a55397237fe | CRAM        |
| 1020 | PBBPAP           | 0DDLPJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/13991504-b94d-4443-881d-bf31d6d2b9b2   | IGM_PBBPAP-0DDLPJ_20221006_bam.bai               | 648a6006d149077997c788592a55ec2f | BAI         |
| 1021 | PBBPAP           | 0DDLPJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/6eefdf2f-fd66-4908-b82a-bcfc0a656fa4   | IGM_PBBPAP-0DDLPJ_20221006_merged_bam.bam        | bcceada0b1086c1beb91a7abd0729580 | BAM         |
| 1022 | PBBPAP           | 0DDLPJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8dfaa1bb-9d12-429c-9d97-ad5e1a14c578   | IGM_PBBPAP-0DDLPJ_20221006_bam.bam               | 2748ac208761efa939c73e5be435976b | BAM         |
| 1023 | PBBPAP           | 0DDLPJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7fb4d9b7-11da-4941-b4dd-107651672421   | IGM_PBBPAP-0DDLPJ_20221006_merged_bam.bai        | dbf69af1062adbb15e9cb776814c51c2 | BAI         |
| 1024 | PBBPAP           | 0DDLQ1_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/9ae18feb-2a46-4d21-b979-20dcd2d97465   | IGM_PBBPAP-0DDLQ1_20221006_tumor.aln.cram        | 1589346df8478374ae5a812cdbc38608 | CRAM        |
| 1025 | PBBPAP           | 0DDLQ1_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/fef1180b-92dc-4e63-81c9-72ef2fab407e   | IGM_PBBPAP-0DDLQ1_20221006_tumor.aln.cram.crai   | 796ad921e455c6e49b5c9a17034018fc | CRAI        |
| 1026 | PBBPAP           | 0DDLPF_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/a794e4ed-ff95-42ba-8354-ef7459ae674b   | IGM_PBBPAP-0DDLPF_20221006_normal.aln.cram       | 49c88016e74e00c14391aad332bf78ce | CRAM        |
| 1027 | PBBPAP           | 0DDLPF_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/05a908fd-1915-4cca-9d3d-a6df6e1c351b   | IGM_PBBPAP-0DDLPF_20221006_normal.aln.cram.crai  | 71c10c003d7a046e1c9820a0586b1395 | CRAI        |
| 1028 | PBBPAA           | 0DDKCR_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/0e587af2-34fe-4072-a935-17ab3ae7fa19   | IGM_PBBPAA-0DDKCR_20221005_tumor.aln.cram.crai   | 080e34b2329c7983639d69230169064d | CRAI        |
| 1029 | PBBPAA           | 0DDKCR_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/28d11c51-7645-47d9-b68d-ee06be65f245   | IGM_PBBPAA-0DDKCR_20221005_tumor.aln.cram        | 01960112d2cf4f672d5527038df39e31 | CRAM        |
| 1030 | PBBPAA           | 0DDKDB_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/77f05ef3-d5c8-4c57-8ba9-20ef71c48f1f   | IGM_PBBPAA-0DDKDB_20221005_normal.aln.cram.crai  | 0b93338db5d50a8a5aa0dd6fa5debbff | CRAI        |
| 1031 | PBBPAA           | 0DDKDB_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/38ea97eb-8024-45c1-b45b-6e6c5ad14500   | IGM_PBBPAA-0DDKDB_20221005_normal.aln.cram       | e8b02c85a18fd82b1f27be5389b78cdc | CRAM        |
| 1032 | PBBPAA           | 0DDKCQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d2361879-cff0-48d8-a6a9-77b79b84e621   | IGM_PBBPAA-0DDKCQ_20221003_bam.bam               | 3bd474190ea2b7f4df02ca81e4e9de62 | BAM         |
| 1033 | PBBPAA           | 0DDKCQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/706359bf-11c7-46b6-9497-5df13c28c918   | IGM_PBBPAA-0DDKCQ_20221003_bam.bai               | 943a4aa918e928574d4784823781b0d2 | BAI         |
| 1034 | PBBPAA           | 0DDKCQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ba9cc43e-114d-4d75-936c-64f53f9a68d6   | IGM_PBBPAA-0DDKCQ_20221003_merged_bam.bai        | 503543b0c6cdeab0f37fbbaae8812487 | BAI         |
| 1035 | PBBPAA           | 0DDKCQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/de47ad74-9161-4804-a4a2-ccde00352713   | IGM_PBBPAA-0DDKCQ_20221003_merged_bam.bam        | e03701a88f54a0d49531e7f87793df7c | BAM         |
| 1036 | PBBNTY           | 0DDDTN_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/a65a97c2-97f6-413e-99d4-3feadbe02420   | IGM_PBBNTY-0DDDTN_20220929_normal.aln.cram.crai  | c3b814519ea5d1fdc87c1d3bf0f55da3 | CRAI        |
| 1037 | PBBNTY           | 0DDDTN_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/e14d6203-8829-407b-9b8b-a63a73eac609   | IGM_PBBNTY-0DDDTN_20220929_normal.aln.cram       | e18aeb3265ee5e24f62140e372f8c0e4 | CRAM        |
| 1038 | PBBNTY           | 0DDDTL_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3d1b0ca6-0054-4dfb-8771-1762d266017d   | IGM_PBBNTY-0DDDTL_20220929_merged_bam.bam        | eb3438278bf78d318dac76e069e3dd78 | BAM         |
| 1039 | PBBNTY           | 0DDDTL_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/317b4661-8b26-42b0-8ae1-03d07838505a   | IGM_PBBNTY-0DDDTL_20220929_bam.bam               | 9cd469b5c4a5a479c857c38a090da8be | BAM         |
| 1040 | PBBNTY           | 0DDDTL_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/89ed88e6-e7b5-4344-a3f4-085e34bd83f3   | IGM_PBBNTY-0DDDTL_20220929_merged_bam.bai        | 97b29b89dc7d45cde1b7ff1a383f1ad8 | BAI         |
| 1041 | PBBNTY           | 0DDDTL_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2d80605b-1e0b-4919-8a02-fcc3622846b7   | IGM_PBBNTY-0DDDTL_20220929_bam.bai               | bb518cf772ac200cdf309b6f2399d81a | BAI         |
| 1042 | PBBNTY           | 0DDDTM_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/cf3834a0-c021-4a44-83c9-76d0254b339c   | IGM_PBBNTY-0DDDTM_20220929_tumor.aln.cram        | c52509be5e31e7bcb0f5b17ab620c3b3 | CRAM        |
| 1043 | PBBNTY           | 0DDDTM_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/feabc522-7922-494b-ab2c-ad3e46b3a7de   | IGM_PBBNTY-0DDDTM_20220929_tumor.aln.cram.crai   | f19ec089c55196d7636cc89545f507de | CRAI        |
| 1044 | PBBKPA           | 0DAWAF_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/d7d8ffdd-727c-48d2-92a1-cf4734aa0716   | IGM_PBBKPA-0DAWAF_20220902a_normal.aln.cram      | cb142cb0c0477825a9be204a11717392 | CRAM        |
| 1045 | PBBKPA           | 0DAWAF_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/4dec110f-9ef3-4b96-9e77-35e183001c27   | IGM_PBBKPA-0DAWAF_20220902a_normal.aln.cram.crai | 3bf68641134f206a58832a26a89e41fd | CRAI        |
| 1046 | PBBKPA           | 0DAWAT_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/5a22befe-899e-473f-ba68-2842f9442aab   | IGM_PBBKPA-0DAWAT_20220708_bam.bam               | 4205953f282d213470a21a3b0c39618e | BAM         |
| 1047 | PBBKPA           | 0DAWAT_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/cf2e392f-69fb-41f7-b093-f3a413c2d3a3   | IGM_PBBKPA-0DAWAT_20220708_merged_bam.bam        | 1584aac910babe6e444972667549f1b3 | BAM         |
| 1048 | PBBKPA           | 0DAWAT_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/25156cbc-609e-4b66-b360-e8e8b705b509   | IGM_PBBKPA-0DAWAT_20220708_bam.bai               | c19993e6dfa198a12097c19e6d2abc5e | BAI         |
| 1049 | PBBKPA           | 0DAWAT_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/eaf5737c-7870-4c4c-a56e-6f937a7afcd5   | IGM_PBBKPA-0DAWAT_20220708_merged_bam.bai        | dcedf8b516441ad48a35a15fec2b122c | BAI         |
| 1050 | PBBKPA           | 0DAWAU_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/4ff0fe32-223d-4de3-bdeb-125976af51cb   | IGM_PBBKPA-0DAWAU_20220902a_tumor.aln.cram       | f119dfb15db790f0b1137793e33c6c58 | CRAM        |
| 1051 | PBBKPA           | 0DAWAU_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/567a5f09-01ae-442e-a0d5-3c868fb0a16b   | IGM_PBBKPA-0DAWAU_20220902a_tumor.aln.cram.crai  | b19386bc5360519812b63d1ff80b9c21 | CRAI        |
| 1052 | PBBJVU           | 0DAAZA_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/df92b33d-a4a5-4996-8d89-8aa3e6a60165   | IGM_PBBJVU-0DAAZA_20220902a_tumor.aln.cram       | f4a5de5772cecf83605154ec892d89fb | CRAM        |
| 1053 | PBBJVU           | 0DAAZA_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/10048985-7647-4edc-96dc-c8a5f0ef4b6a   | IGM_PBBJVU-0DAAZA_20220902a_tumor.aln.cram.crai  | 9d177ca801f52496b46d8c4adaf9b77b | CRAI        |
| 1054 | PBBJVU           | 0DAAZL_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/2228af5c-9a3b-48b2-a3db-117f222e4ff7   | IGM_PBBJVU-0DAAZL_20220902a_normal.aln.cram      | 6cd6873faf853ac79568d9d8c6c91e47 | CRAM        |
| 1055 | PBBJVU           | 0DAAZL_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/1f87cbae-72ec-470e-a6f1-329a7d2d0fd0   | IGM_PBBJVU-0DAAZL_20220902a_normal.aln.cram.crai | 37af935bc235cde775b4df08dead77e4 | CRAI        |
| 1056 | PBBJVU           | 0DAAZ7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/049fe56c-660f-4852-bc36-8c6fea28a30f   | IGM_PBBJVU-0DAAZ7_20220622_bam.bai               | 2d88249e51e61e981df3cb92dae841de | BAI         |
| 1057 | PBBJVU           | 0DAAZ7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4b848bbd-19e6-4e51-896b-a667542e8e5c   | IGM_PBBJVU-0DAAZ7_20220622_merged_bam.bam        | af042f5e0946931a1af2e896391f1932 | BAM         |
| 1058 | PBBJVU           | 0DAAZ7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0077d7b4-3058-4252-934a-a0baf1abf3f2   | IGM_PBBJVU-0DAAZ7_20220622_merged_bam.bai        | a1be0e6de14cc4abf77096e58d02731c | BAI         |
| 1059 | PBBJVU           | 0DAAZ7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/9091c2c0-7646-4708-8088-8a02b554b46d   | IGM_PBBJVU-0DAAZ7_20220622_bam.bam               | 672e3c1b0cca795da98396187d57d8d0 | BAM         |
| 1060 | PBBJJT           | 0D9XFF_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/5628ce41-6478-4999-8955-2dcdfb0c1d6d   | IGM_PBBJJT-0D9XFF_20220902a_normal.aln.cram.crai | d4c59a9cfed790090f56d14e8eb57d5f | CRAI        |
| 1061 | PBBJJT           | 0D9XFF_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/6381f949-4029-44e1-85c7-9c0bad303a78   | IGM_PBBJJT-0D9XFF_20220902a_normal.aln.cram      | 4d9120751a153cd1a24c90c8058c3f13 | CRAM        |
| 1062 | PBBJJT           | 0D9XFD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/9e098de3-4b64-41ec-b3c1-6b4cf57145cc   | IGM_PBBJJT-0D9XFD_20220919_merged_bam.bam        | 17df4c2309baea3be78869b5eb52f404 | BAM         |
| 1063 | PBBJJT           | 0D9XFD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/01893634-0b9e-4e6e-8503-b78e2983634a   | IGM_PBBJJT-0D9XFD_20220919_bam.bam               | 38d73ea0b826d151baee0cb6897fcf62 | BAM         |
| 1064 | PBBJJT           | 0D9XFD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ad7f1dab-5646-4f4f-99df-1f23154c5bbb   | IGM_PBBJJT-0D9XFD_20220919_bam.bai               | 9449a1b75f655d7917808605d5f1fc2a | BAI         |
| 1065 | PBBJJT           | 0D9XFD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/006745cd-a260-4913-a11a-f107387027e7   | IGM_PBBJJT-0D9XFD_20220919_merged_bam.bai        | 46c608a3966e95d3c1f5ec9e6e93f151 | BAI         |
| 1066 | PBBJJT           | 0D9XFE_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/ba7fd40e-edcc-4133-a3cd-51589e570dae   | IGM_PBBJJT-0D9XFE_20220902a_tumor.aln.cram       | 8f5e52022e6ef7c86eeb98ae2a74a0a3 | CRAM        |
| 1067 | PBBJJT           | 0D9XFE_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/9e2a0c2a-06fe-438a-82dc-14d1eb30b588   | IGM_PBBJJT-0D9XFE_20220902a_tumor.aln.cram.crai  | 399acd8ab966c878d64e684bad5e329f | CRAI        |
| 1068 | PBBMPA           | 0DCM14_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/65ed0393-8bd3-44af-91d7-5d71fb00ef21   | IGM_PBBMPA-0DCM14_20220830_bam.bai               | 576db37a933d23d728ce21ed92962a95 | BAI         |
| 1069 | PBBMPA           | 0DCM14_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/18145a1c-ba35-41d4-8993-cfb1d20170bb   | IGM_PBBMPA-0DCM14_20220830_merged_bam.bai        | 09d1c4cc77a340d69c023f2ab24c2ddc | BAI         |
| 1070 | PBBMPA           | 0DCM14_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/47845a0f-0f9e-4bd4-97e8-4d4e9abf7be9   | IGM_PBBMPA-0DCM14_20220830_merged_bam.bam        | 7e834a813892f65c09a3913310b77d85 | BAM         |
| 1071 | PBBMPA           | 0DCM14_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/5a102dee-97ba-4dd6-b970-432824843d0b   | IGM_PBBMPA-0DCM14_20220830_bam.bam               | 2e0800502d48a8780c6f976b20e7ea2f | BAM         |
| 1072 | PBBMPA           | 0DCM1N_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/b9916d13-06b3-41c2-a007-807c9106dde4   | IGM_PBBMPA-0DCM1N_20220902a_tumor.aln.cram.crai  | 575a510b2bffa39eac21bb878987751f | CRAI        |
| 1073 | PBBMPA           | 0DCM1N_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/0d4b2665-4b52-4c39-b295-7ddae4b5320b   | IGM_PBBMPA-0DCM1N_20220902a_tumor.aln.cram       | 918414316eaf7bbdc4c9b03ff7fe3e6f | CRAM        |
| 1074 | PBBMPA           | 0DCM1C_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/94fe4dd3-f40d-49bd-8dd3-e59bc8c0f765   | IGM_PBBMPA-0DCM1C_20220902a_normal.aln.cram.crai | 2e81b98a4e4a80f225f5c204d9b3ad3c | CRAI        |
| 1075 | PBBMPA           | 0DCM1C_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/575eb422-b678-40ea-9922-007130369f2e   | IGM_PBBMPA-0DCM1C_20220902a_normal.aln.cram      | bf1680463b35051f9e3bec686de13d40 | CRAM        |
| 1076 | PBBJSS           | 0DAF1H_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/2405e45a-a26f-4558-aa80-104da3377e06   | IGM_PBBJSS-0DAF1H_20220902a_normal.aln.cram      | fc7fe0a281e55bd9d7c8e317e60ee1ff | CRAM        |
| 1077 | PBBJSS           | 0DAF1H_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/5f67ee4d-7700-455d-b5ef-202910056a9d   | IGM_PBBJSS-0DAF1H_20220902a_normal.aln.cram.crai | 367e8a3ddfc3d62a5ab748b9624a430e | CRAI        |
| 1078 | PBBJSS           | 0DAF1D_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/66859411-b149-4506-ac24-7ccaec527b25   | IGM_PBBJSS-0DAF1D_20220629_bam.bam               | 04ffe9270a6e1dd3bc6af9d5dbddb32f | BAM         |
| 1079 | PBBJSS           | 0DAF1D_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/381f27a4-0e55-4052-8673-f3543d560d70   | IGM_PBBJSS-0DAF1D_20220629_merged_bam.bai        | b0c5028693fd174b82764a2314ac55b4 | BAI         |
| 1080 | PBBJSS           | 0DAF1D_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/38fe542c-b9d9-4d40-872b-baa466209930   | IGM_PBBJSS-0DAF1D_20220629_merged_bam.bam        | 6472c821e91da2673e1907a35e0f85bb | BAM         |
| 1081 | PBBJSS           | 0DAF1D_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0984e47d-bd1d-4a43-bf47-9222801561b3   | IGM_PBBJSS-0DAF1D_20220629_bam.bai               | 58f2843537ea5ba4e724a13f83c0fa6e | BAI         |
| 1082 | PBBJSS           | 0DAF27_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/4615227c-edc1-42b9-a534-70afb9891ac1   | IGM_PBBJSS-0DAF27_20220902a_tumor.aln.cram.crai  | f121db8cebe5ef704e787118f86e376d | CRAI        |
| 1083 | PBBJSS           | 0DAF27_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/dffa5697-35d4-460d-b2ee-a8619c78c24c   | IGM_PBBJSS-0DAF27_20220902a_tumor.aln.cram       | 978bdd32d4891c2a25c8efe2fc685be4 | CRAM        |
| 1084 | PBBLMC           | 0DBXXZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7273c669-3a05-47ab-a9a2-43a03f6fc51b   | IGM_PBBLMC-0DBXXZ_20220815_bam.bai               | 142a96ce19002e51ce56a2c9b7659f43 | BAI         |
| 1085 | PBBLMC           | 0DBXXZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/9aa182bc-ebb1-48c5-be4d-c5af2cd48e74   | IGM_PBBLMC-0DBXXZ_20220815_bam.bam               | 31dc80c85a8f994fa7d6324ec2040245 | BAM         |
| 1086 | PBBLMC           | 0DBXXZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e0ec89e5-e2c0-46b1-aaf2-bf130c04f1f7   | IGM_PBBLMC-0DBXXZ_20220815_merged_bam.bai        | 2ada09fe4ecf6d94a12065ce69f1e4d7 | BAI         |
| 1087 | PBBLMC           | 0DBXXZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/34f5fe80-caa4-4cff-9b9c-d5535ae6dfe4   | IGM_PBBLMC-0DBXXZ_20220815_merged_bam.bam        | 1a4ce9e30685abccc369fdb55c2c4d6d | BAM         |
| 1088 | PBBLMC           | 0DBXXX_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/9029cb42-647b-49a4-8788-cf07c56cb585   | IGM_PBBLMC-0DBXXX_20220902a_tumor.aln.cram       | 41c293fa6906593da2c791fd45695664 | CRAM        |
| 1089 | PBBLMC           | 0DBXXX_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/4f3395cf-e5c0-410b-9e52-decd447be2c7   | IGM_PBBLMC-0DBXXX_20220902a_tumor.aln.cram.crai  | c417de6497dd0df4b5a9974f25feaf4c | CRAI        |
| 1090 | PBBLMC           | 0DBXXU_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/136adf9b-996a-4892-8b45-8d26c1d26244   | IGM_PBBLMC-0DBXXU_20220902a_normal.aln.cram      | 3f2bca8acf25416cb1cb576c1755303a | CRAM        |
| 1091 | PBBLMC           | 0DBXXU_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/6bf2f53e-cedd-444b-8f18-4d19e5a8f5cd   | IGM_PBBLMC-0DBXXU_20220902a_normal.aln.cram.crai | 85e67159261b4777531f8cd9faf75d14 | CRAI        |
| 1092 | PBBILM           | 0DAAZ6_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/1b334bad-fba0-4174-9856-ed966c761a4d   | IGM_PBBILM-0DAAZ6_20220919_bam.bam               | 252a9bb2787d18175a024d6d4aaa97f0 | BAM         |
| 1093 | PBBILM           | 0DAAZ6_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c93a0882-5ca6-49f3-9f4d-5c14d238da0e   | IGM_PBBILM-0DAAZ6_20220919_merged_bam.bam        | 39e7c93b29cc6c3cc23a0b4a0fcaceae | BAM         |
| 1094 | PBBILM           | 0DAAZ6_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2afde84c-d00e-43d8-929d-ed7e35709a56   | IGM_PBBILM-0DAAZ6_20220919_merged_bam.bai        | 2778946129fd3db8db0d9536bb6c52cd | BAI         |
| 1095 | PBBILM           | 0DAAZ6_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/6796f51e-87b3-4ce2-9bd3-8747ce920e15   | IGM_PBBILM-0DAAZ6_20220919_bam.bai               | 896e0db6700fcc17d6d4c47e7a5b2a59 | BAI         |
| 1096 | PBBHPP           | 0D95IK_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/01c4e515-c79e-4f1d-9d68-3881a0835622   | IGM_PBBHPP-0D95IK_20220902a_normal.aln.cram.crai | 47b5719ee0a8a82becda74bdece59ef1 | CRAI        |
| 1097 | PBBHPP           | 0D95IK_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/41102bca-83a2-412e-a0b6-81e9eabc1e61   | IGM_PBBHPP-0D95IK_20220902a_normal.aln.cram      | 9aae67697ab4792ebc970cf8addc996b | CRAM        |
| 1098 | PBBHPP           | 0D95II_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/5b33f7ba-1947-4eb8-b623-601a4ccfa2a4   | IGM_PBBHPP-0D95II_20220902a_tumor.aln.cram.crai  | ec6a03f090d79b133405ca92db0159b3 | CRAI        |
| 1099 | PBBHPP           | 0D95II_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/5b34f149-5adb-47cb-8028-4ef2115e1ca9   | IGM_PBBHPP-0D95II_20220902a_tumor.aln.cram       | af4e43f52dfd65fdddbee74bf7a9c19d | CRAM        |
| 1100 | PBBHPP           | 0D95IJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f1672d69-ebdc-4bd5-b70c-544ea00c5b07   | IGM_PBBHPP-0D95IJ_20220919_merged_bam.bai        | 3d40e9f57b71863270397806958a10de | BAI         |
| 1101 | PBBHPP           | 0D95IJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7899de3e-c641-4eb3-942d-71eca51cda8c   | IGM_PBBHPP-0D95IJ_20220919_bam.bam               | bf72e7b8697fe6c9b88260b7ef41ef12 | BAM         |
| 1102 | PBBHPP           | 0D95IJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d8adf51a-d2b9-4759-bebb-06c804cd8e0b   | IGM_PBBHPP-0D95IJ_20220919_merged_bam.bam        | 18293006a3ec42706feaf5da744e3b2f | BAM         |
| 1103 | PBBHPP           | 0D95IJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8869593e-1cb7-4323-bb8a-88fa8671332f   | IGM_PBBHPP-0D95IJ_20220919_bam.bai               | d5ebc1c89d0d303e0310828e3317757a | BAI         |
| 1104 | PBBHIL           | 0D8BTF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/103893e5-7764-4d7b-983a-ef1f17cccee5   | IGM_PBBHIL-0D8BTF_20220919_merged_bam.bam        | 4b9a066d1b33411b44e538a098a5d118 | BAM         |
| 1105 | PBBHIL           | 0D8BTF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d1977e9b-6074-44e3-b1bd-08add0c9edb4   | IGM_PBBHIL-0D8BTF_20220919_bam.bai               | ff08751ed40c1fa2535bf4af900b7df9 | BAI         |
| 1106 | PBBHIL           | 0D8BTF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f41c22d5-3718-48a2-a351-98f73be4fbe9   | IGM_PBBHIL-0D8BTF_20220919_bam.bam               | aa924b2abe9c523d0bb69c43dcdf2f4b | BAM         |
| 1107 | PBBHIL           | 0D8BTF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2f4ec81b-2305-4ddb-b7fa-03f2c2835794   | IGM_PBBHIL-0D8BTF_20220919_merged_bam.bai        | 04da6fcf2093efb7d46a1fdb2e2b3be9 | BAI         |
| 1108 | PBBHIL           | 0D8BTL_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/8050f1a9-df52-43fd-850f-0ff88c892400   | IGM_PBBHIL-0D8BTL_20220902a_normal.aln.cram.crai | 4625b0a946f766346cc531767d85deab | CRAI        |
| 1109 | PBBHIL           | 0D8BTL_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/63bccd5c-cfea-4413-a7e4-96680a2513de   | IGM_PBBHIL-0D8BTL_20220902a_normal.aln.cram      | 7db786ac8532f22628cb9f48fa3d893b | CRAM        |
| 1110 | PBBHIL           | 0D8BTK_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/16832f84-7300-4275-9148-82027c40bb58   | IGM_PBBHIL-0D8BTK_20220902a_tumor.aln.cram       | 94a730f30cb67816c08f9e88c6eb96ae | CRAM        |
| 1111 | PBBHIL           | 0D8BTK_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/d61347da-2405-4296-8b63-ffffdda50580   | IGM_PBBHIL-0D8BTK_20220902a_tumor.aln.cram.crai  | 98c819e3050bee3360b28719820cd681 | CRAI        |
| 1112 | PBBJVN           | 0DAGP2_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/2a76273f-b01b-4a0b-b056-5df43cd28e09   | IGM_PBBJVN-0DAGP2_20220902a_tumor.aln.cram.crai  | 542367ed37b19aab4e6ba793ef773e4d | CRAI        |
| 1113 | PBBJVN           | 0DAGP2_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/20938aff-7590-46b9-a5ca-54596a0d325f   | IGM_PBBJVN-0DAGP2_20220902a_tumor.aln.cram       | 0ec2cbd57c2d9e3b9b346f0294e9994f | CRAM        |
| 1114 | PBBJVN           | 0DAGP8_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e2c45ef1-2d33-489c-8605-c5940f40daa7   | IGM_PBBJVN-0DAGP8_20220628_bam.bam               | 43450220ad045cef9eafe4aebb69bad2 | BAM         |
| 1115 | PBBJVN           | 0DAGP8_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/1dfcb20b-aebc-4184-b87a-47b7762ce0ba   | IGM_PBBJVN-0DAGP8_20220628_merged_bam.bam        | 5710a0673f109167342375df47d2a95f | BAM         |
| 1116 | PBBJVN           | 0DAGP8_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/50f4179e-c0d2-4f41-85c6-61483931a648   | IGM_PBBJVN-0DAGP8_20220628_merged_bam.bai        | da712a41da89c4e182da144b03fc1f48 | BAI         |
| 1117 | PBBJVN           | 0DAGP8_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7ba00a51-2b90-4cb1-bde6-58be5704e9e8   | IGM_PBBJVN-0DAGP8_20220628_bam.bai               | d56155c9e628d22abc1379a7727f9eb3 | BAI         |
| 1118 | PBBJVN           | 0DAGOW_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/2c1ff29d-5ea5-4a34-9a7a-ac0f2937b535   | IGM_PBBJVN-0DAGOW_20220902a_normal.aln.cram.crai | 33752a84b6130868702b0e2cbf35631e | CRAI        |
| 1119 | PBBJVN           | 0DAGOW_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/54cc1d6e-83da-4990-b5df-ae0f8fddda08   | IGM_PBBJVN-0DAGOW_20220902a_normal.aln.cram      | 41a062666ae299ca1ee10e9498e58bc3 | CRAM        |
| 1120 | PBBKSG           | 0DCFVU_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/e3294442-8ae6-4815-9bf0-d395d89b9e7d   | IGM_PBBKSG-0DCFVU_20220902a_tumor.aln.cram       | 135a6fa4269d01c6aa276b19058a7a44 | CRAM        |
| 1121 | PBBKSG           | 0DCFVU_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/5881c55f-d8a0-4a14-86b4-00dabfd94555   | IGM_PBBKSG-0DCFVU_20220902a_tumor.aln.cram.crai  | 5ac813eec8b5991a5c34748dcc0f43ef | CRAI        |
| 1122 | PBBKSG           | 0DCFVO_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/c3cf6b28-b92d-43fc-ae43-5c8b18c3ae16   | IGM_PBBKSG-0DCFVO_20220902a_normal.aln.cram.crai | 6380feb85d0a5e70e2ce77c93121e896 | CRAI        |
| 1123 | PBBKSG           | 0DCFVO_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f64eaea1-8701-4033-8db3-58572b2acc85   | IGM_PBBKSG-0DCFVO_20220902a_normal.aln.cram      | d732b72b1c4111864decb79ee28414ee | CRAM        |
| 1124 | PBBKSG           | 0DCFVJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f8671659-09bf-44f0-a556-717ff5455f78   | IGM_PBBKSG-0DCFVJ_20220825_merged_bam.bam        | 493636bd4b7298d3d557793fda9f858b | BAM         |
| 1125 | PBBKSG           | 0DCFVJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/31afc209-1a09-441f-b38b-2cb8b44b48f6   | IGM_PBBKSG-0DCFVJ_20220825_merged_bam.bai        | b1ae58d3c10a65866714027b4000f232 | BAI         |
| 1126 | PBBKSG           | 0DCFVJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/1c2851ca-fa2d-4277-83bf-d9a74162dd3a   | IGM_PBBKSG-0DCFVJ_20220825_bam.bai               | 44785497bdbc3b88fe4ce86478e41732 | BAI         |
| 1127 | PBBKSG           | 0DCFVJ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8760c4ee-a233-4379-888d-0a43e0238e29   | IGM_PBBKSG-0DCFVJ_20220825_bam.bam               | 841cb4d96215aad912edace129c8cb57 | BAM         |
| 1128 | PBBHYH           | 0D9AJY_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/64bab815-5716-4d4c-aa9a-6dd225cd8224   | IGM_PBBHYH-0D9AJY_20220919_merged_bam.bai        | 7fc6508a722d0d0d02119e35dcdfcae1 | BAI         |
| 1129 | PBBHYH           | 0D9AJY_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4f9c5a12-5f62-4a7c-968c-f0a64e307eed   | IGM_PBBHYH-0D9AJY_20220919_merged_bam.bam        | 62b41065b856bfe996e0bf00aada3931 | BAM         |
| 1130 | PBBHYH           | 0D9AJY_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/dac18560-0398-47ea-a98f-0ad1d274bcb7   | IGM_PBBHYH-0D9AJY_20220919_bam.bai               | d9e3a2a8b1f5ac8562fad2ccb47c8796 | BAI         |
| 1131 | PBBHYH           | 0D9AJY_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/dbb63551-c5b0-474a-9027-c600709e3f63   | IGM_PBBHYH-0D9AJY_20220919_bam.bam               | 862d55ca56923549237d4710ea4dc40c | BAM         |
| 1132 | PBBHYH           | 0D9AKM_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/644004fd-6760-4e6c-be86-28c9ff680bc7   | IGM_PBBHYH-0D9AKM_20220902a_tumor.aln.cram.crai  | b79cc692de7dbda045f142621e7b4327 | CRAI        |
| 1133 | PBBHYH           | 0D9AKM_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/8db2f9d0-ba9c-457f-85ba-87f6546e270d   | IGM_PBBHYH-0D9AKM_20220902a_tumor.aln.cram       | 4d4a6c9f94ae8bf0bf656b5f20476857 | CRAM        |
| 1134 | PBBHYH           | 0D9AKN_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/08c63c9f-35ea-423f-baf6-04a0017f1dcc   | IGM_PBBHYH-0D9AKN_20220902a_normal.aln.cram      | 19e778b19ef8374e3da8f92e88f448c4 | CRAM        |
| 1135 | PBBHYH           | 0D9AKN_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/db3d73f4-b016-4478-b6e4-feabe79125fa   | IGM_PBBHYH-0D9AKN_20220902a_normal.aln.cram.crai | 45a692452cd16e0df090ad818a8773a8 | CRAI        |
| 1136 | PBBPWG           | 0DEJZO_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/0b7226ed-415e-4110-b785-f84fcf5c006f   | IGM_PBBPWG-0DEJZO_20221102_normal.aln.cram       | 8ed7475a1bed020a9e81c4947767f017 | CRAM        |
| 1137 | PBBPWG           | 0DEJZO_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/1231a1b5-7326-42bd-930a-d5d67cd52bce   | IGM_PBBPWG-0DEJZO_20221102_normal.aln.cram.crai  | af90fd0d80c7dd99ee2b897a9cfb7e79 | CRAI        |
| 1138 | PBBPWG           | 0DEK11_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d66ecf19-b368-40ad-9f25-2eb3d6c20e70   | IGM_PBBPWG-0DEK11_20221108_bam.bam               | 4a83d6c4487f54b0c517f772d43abce4 | BAM         |
| 1139 | PBBPWG           | 0DEK11_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/79a31526-ca7e-4f11-a1ba-bcc4749761cb   | IGM_PBBPWG-0DEK11_20221108_merged_bam.bai        | f004f203b26e3ccb7ce5149f6a7cc506 | BAI         |
| 1140 | PBBPWG           | 0DEK11_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f31a0915-5ada-4719-84b5-9533ec4fcd74   | IGM_PBBPWG-0DEK11_20221108_bam.bai               | edb3760b0ff86080cfa33c00edb438a4 | BAI         |
| 1141 | PBBPWG           | 0DEK11_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/9611dd86-2fc9-4043-9b99-5eb2709ee04c   | IGM_PBBPWG-0DEK11_20221108_merged_bam.bam        | f4d600a7f1915ee520e88d0e0b47afc6 | BAM         |
| 1142 | PBBPWG           | 0DEK1L_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/42ee5e0a-578e-40be-8b6a-afd0691bdec8   | IGM_PBBPWG-0DEK1L_20221102_tumor.aln.cram.crai   | 37ea709f48361345792558d0077c90b1 | CRAI        |
| 1143 | PBBPWG           | 0DEK1L_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/075fefbd-9318-43e4-bc5f-346013872aa1   | IGM_PBBPWG-0DEK1L_20221102_tumor.aln.cram        | 64ee9909fc4481ee8ac9a577ca7caf5a | CRAM        |
| 1144 | PBBRKP           | 0DEV2G_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/d5b2e432-58f8-4cee-a318-2d3a2b93885c   | IGM_PBBRKP-0DEV2G_20221205_normal.aln.cram.crai  | 51c8e64caaee23b4bfe9ebb84d355cd3 | CRAI        |
| 1145 | PBBRKP           | 0DEV2G_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/8a65321d-07a0-417f-a4c0-820fe3daccd7   | IGM_PBBRKP-0DEV2G_20221205_normal.aln.cram       | 8b6bdbc1f6dba5651bfa2bc775a3621f | CRAM        |
| 1146 | PBBRKP           | 0DEYGK_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/f9aef257-e757-4825-a24f-93dc6118d5bc   | IGM_PBBRKP-0DEYGK_20221205_tumor.aln.cram        | da519027255c134132f6b1d67f60c0b0 | CRAM        |
| 1147 | PBBRKP           | 0DEYGK_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/fb5cc62f-a7fa-40bc-90dc-b120db9d0cc5   | IGM_PBBRKP-0DEYGK_20221205_tumor.aln.cram.crai   | 128fd7698bba494119558436d6572257 | CRAI        |
| 1148 | PBBRKP           | 0DEV3S_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0d126645-5bb6-40ee-801c-2406764eb5ea   | IGM_PBBRKP-0DEV3S_20221116_bam.bam               | 2e749fdd07f854c447265d814c1a7e2c | BAM         |
| 1149 | PBBRKP           | 0DEV3S_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2cb85a4b-331f-4a6c-be69-3f7de752120b   | IGM_PBBRKP-0DEV3S_20221116_merged_bam.bam        | 7ed7fd151353629fce6662f1081c5b9b | BAM         |
| 1150 | PBBRKP           | 0DEV3S_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/729ee991-4880-48af-85f0-a5084b20781e   | IGM_PBBRKP-0DEV3S_20221116_merged_bam.bai        | 4715fb9daf597b8b23f589f74346caca | BAI         |
| 1151 | PBBRKP           | 0DEV3S_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8ca1ce84-6e55-4813-a8ca-9728db080d46   | IGM_PBBRKP-0DEV3S_20221116_bam.bai               | 03372935e71496c823763925e3f3a0ff | BAI         |
| 1152 | PBBPIX           | 0DE7E1_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b406ab77-59a8-4eba-be6c-fc949610857d   | IGM_PBBPIX-0DE7E1_20221109_normal.aln.cram       | 5634aeefd8aff9ca46f278f3e75f685e | CRAM        |
| 1153 | PBBPIX           | 0DE7E1_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/d4968781-7415-4327-a963-f97576fe360e   | IGM_PBBPIX-0DE7E1_20221109_normal.aln.cram.crai  | 19be131aa2616a88b03cbec7cb8e8cde | CRAI        |
| 1154 | PBBPIX           | 0DE7FL_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/16d7501d-6063-4b8d-856a-1c6d4d328e10   | IGM_PBBPIX-0DE7FL_20221109_tumor.aln.cram        | 8896948918f6d8cbb3ba372837ed78d5 | CRAM        |
| 1155 | PBBPIX           | 0DE7FL_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/fbb6e35e-76a0-43c0-ba6e-8ad700a1792a   | IGM_PBBPIX-0DE7FL_20221109_tumor.aln.cram.crai   | fbd7f81c76c6ef3149b1a0517c77b252 | CRAI        |
| 1156 | PBBPIX           | 0DE7CS_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3fc009ad-9c7e-4feb-a3a4-6994ffe131ab   | IGM_PBBPIX-0DE7CS_20221101_merged_bam.bam        | 4c3dc87366ba2de131cbf10ec93ebc3e | BAM         |
| 1157 | PBBPIX           | 0DE7CS_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0030d667-7237-48dd-ac6b-257ef2c0981d   | IGM_PBBPIX-0DE7CS_20221101_bam.bam               | 0abeda764aef1043fcb318e8fb6a5243 | BAM         |
| 1158 | PBBPIX           | 0DE7CS_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/23a3da58-dfd4-40e0-a80d-e6817adea680   | IGM_PBBPIX-0DE7CS_20221101_bam.bai               | 7bc252b09b43f5677f61e2f9e5d877c3 | BAI         |
| 1159 | PBBPIX           | 0DE7CS_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/9f1897cc-fa10-4865-a96d-3ac4592975ca   | IGM_PBBPIX-0DE7CS_20221101_merged_bam.bai        | b67d28b70f4b21147a294e4b61398c24 | BAI         |
| 1160 | PBBPTA           | 0DEFHN_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/fc42d5fd-e14e-4f9b-a28c-bbc27857a73c   | IGM_PBBPTA-0DEFHN_20221029_normal.aln.cram       | a4f91720781d80d48d8d8b2e316ce804 | CRAM        |
| 1161 | PBBPTA           | 0DEFHN_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/4862f3e1-c3a5-4e55-a881-9134c90581a9   | IGM_PBBPTA-0DEFHN_20221029_normal.aln.cram.crai  | 6b0f336419ca33aa6fe1ce7791c0cadb | CRAI        |
| 1162 | PBBPTA           | 0DEFJE_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d99cb707-a943-49fc-ad1b-901173db4130   | IGM_PBBPTA-0DEFJE_20221105_bam.bam               | 659e737a58d801c9756104f6642c38bb | BAM         |
| 1163 | PBBPTA           | 0DEFJE_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/dcedcc08-3388-4435-afa1-ab0b7fc3c89d   | IGM_PBBPTA-0DEFJE_20221105_merged_bam.bam        | 05bc7edfeb1f19ebae8dd19b0f48941a | BAM         |
| 1164 | PBBPTA           | 0DEFJE_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ea663fb2-5c33-4019-86c2-2e2b4527e370   | IGM_PBBPTA-0DEFJE_20221105_bam.bai               | 4decbf319198532b76b239485fffd6ca | BAI         |
| 1165 | PBBPTA           | 0DEFJE_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c934ea68-a441-44cd-a603-a2dcd1698d88   | IGM_PBBPTA-0DEFJE_20221105_merged_bam.bai        | be55172eace052bf58c8bc55fd8f1bd2 | BAI         |
| 1166 | PBBPTA           | 0DEFJZ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/5eb3e435-f7fd-42cb-8404-47f4653f7c09   | IGM_PBBPTA-0DEFJZ_20221029_tumor.aln.cram        | a9f6aa644f9003517f13b242e2e7b73c | CRAM        |
| 1167 | PBBPTA           | 0DEFJZ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/13872503-2e0c-40d8-ac10-9cefbe9a883a   | IGM_PBBPTA-0DEFJZ_20221029_tumor.aln.cram.crai   | bc609e3ccd88cf217db2a9e5af546f34 | CRAI        |
| 1168 | PBBPNU           | 0DEV3C_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/4639537d-f711-4d72-99f7-b22c3c8a8267   | IGM_PBBPNU-0DEV3C_20221116_merged_bam.bam        | 8dbb8c2b83df9554d9e16913742cd826 | BAM         |
| 1169 | PBBPNU           | 0DEV3C_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/574b5d57-fd43-40c8-a552-6dbdb5bcfba0   | IGM_PBBPNU-0DEV3C_20221116_bam.bai               | 2c2cab3eae3363b856447e795f31c306 | BAI         |
| 1170 | PBBPNU           | 0DEV3C_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a9ca45c4-5573-4f69-8e73-248cdd1a2f39   | IGM_PBBPNU-0DEV3C_20221116_merged_bam.bai        | b3b75ca434816b425b4043225e094832 | BAI         |
| 1171 | PBBPNU           | 0DEV3C_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/abe30f96-1ed0-42f5-bd88-e63bd2d779f1   | IGM_PBBPNU-0DEV3C_20221116_bam.bam               | 99361ece2f6edfc9da3e9599339a2e14 | BAM         |
| 1172 | PBBPNU           | 0DEV2A_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/0aac26fb-4ce6-42de-8de3-3bc8839ce1f0   | IGM_PBBPNU-0DEV2A_20221110_normal.aln.cram.crai  | 23936e758bb5c985a5b317fb44caa23f | CRAI        |
| 1173 | PBBPNU           | 0DEV2A_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/500e76a3-8c9b-4545-883e-09e5eb15c101   | IGM_PBBPNU-0DEV2A_20221110_normal.aln.cram       | 6cafd47ff9059e0056ee284e35bc46e5 | CRAM        |
| 1174 | PBBPNU           | 0DEV4P_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/19bc584c-1f3d-4046-b113-4f344cb70ab0   | IGM_PBBPNU-0DEV4P_20221110_tumor.aln.cram.crai   | a9e21b5df66e8b0d5d6cade5d09945cc | CRAI        |
| 1175 | PBBPNU           | 0DEV4P_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/9980f8c5-5d21-4e5a-bc2b-de96129d921f   | IGM_PBBPNU-0DEV4P_20221110_tumor.aln.cram        | 6d389b8af03b5a3d5a6886989cb0a06b | CRAM        |
| 1176 | PBBRKJ           | 0DFBL8_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/88c19453-47ef-4d7c-86e0-775cb109e1f2   | IGM_PBBRKJ-0DFBL8_20221117_normal.aln.cram.crai  | 48f433342461924c0f7f02abaf15eabe | CRAI        |
| 1177 | PBBRKJ           | 0DFBL8_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/54794cdb-6763-422f-a0b6-883ac0dac48d   | IGM_PBBRKJ-0DFBL8_20221117_normal.aln.cram       | ef777bc5900a7bf5b0bddb61cd32db23 | CRAM        |
| 1178 | PBBRKJ           | 0DFBLQ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/28ba3c8e-07d8-45e2-ba0c-10e5b0ff72f3   | IGM_PBBRKJ-0DFBLQ_20221117_tumor.aln.cram        | 0ab90eaae6ccae2abcb90d39b780a82c | CRAM        |
| 1179 | PBBRKJ           | 0DFBLQ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/865ae262-2f5a-4095-8243-c0cef5c4fcc8   | IGM_PBBRKJ-0DFBLQ_20221117_tumor.aln.cram.crai   | 82b14ce88bd69ee783229ea5679d3934 | CRAI        |
| 1180 | PBBRKJ           | 0DFBLD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c07ecf8c-204e-4c32-aec3-6e816714c247   | IGM_PBBRKJ-0DFBLD_20221121_merged_bam.bai        | 05ae7a96b6973df5b947fa9ca8b066e8 | BAI         |
| 1181 | PBBRKJ           | 0DFBLD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/9f412bb9-a4df-4f96-9983-380b6b9d595c   | IGM_PBBRKJ-0DFBLD_20221121_merged_bam.bam        | c4b9c5837464600a747ba2edb80f9e01 | BAM         |
| 1182 | PBBRKJ           | 0DFBLD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c01e37a4-8fea-4f3e-86c8-e91424a94632   | IGM_PBBRKJ-0DFBLD_20221121_bam.bai               | e5c906962545345d8cb53d814bb1c4f8 | BAI         |
| 1183 | PBBRKJ           | 0DFBLD_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f66ea4f9-8974-4d9b-9c28-6e54849bcf9c   | IGM_PBBRKJ-0DFBLD_20221121_bam.bam               | a93ded6c75234a46e27c359e47c2713b | BAM         |
| 1184 | PBBMCC           | 0DESUB_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/8d849597-f55b-490a-b8b5-a87e46b0c9a9   | IGM_PBBMCC-0DESUB_20221130_normal.aln.cram.crai  | 6e8eccee442ffbfa6bf5a2bf818ca7f8 | CRAI        |
| 1185 | PBBMCC           | 0DESUB_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/86663004-d114-4f81-94d8-c1c10b7a324c   | IGM_PBBMCC-0DESUB_20221130_normal.aln.cram       | 7d944a23d1d0dced50bcda6a9edee3d6 | CRAM        |
| 1186 | PBBMCC           | 0DESUA_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/ee407a5c-c8ed-4f7d-b7a3-8f7683cbca1a   | IGM_PBBMCC-0DESUA_20221130_tumor.aln.cram        | 4cd39181d9a0bd4b52cc10bf47d54ae5 | CRAM        |
| 1187 | PBBMCC           | 0DESUA_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/05a79155-bdbd-40f6-ab5d-c30a167e4fac   | IGM_PBBMCC-0DESUA_20221130_tumor.aln.cram.crai   | f7423d8d144750a79d67999e93c18dd5 | CRAI        |
| 1188 | PBBMCC           | 0DESU9_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/49bd66e1-3220-453d-992b-0e94f3888921   | IGM_PBBMCC-0DESU9_20221122_bam.bai               | 20f376288b518bd7a030a2e67d1c226d | BAI         |
| 1189 | PBBMCC           | 0DESU9_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/547d97c8-caf9-402b-a954-c7f065bf2b4f   | IGM_PBBMCC-0DESU9_20221122_merged_bam.bam        | 41cb1518a74b4192553440b0db853c11 | BAM         |
| 1190 | PBBMCC           | 0DESU9_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c313abb5-03c0-4c21-ba7a-4a310fe5cca6   | IGM_PBBMCC-0DESU9_20221122_bam.bam               | 513fd0fe751a33e0ffa14067d0b018b7 | BAM         |
| 1191 | PBBMCC           | 0DESU9_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8621418a-809f-4b96-a476-0a663b15f946   | IGM_PBBMCC-0DESU9_20221122_merged_bam.bai        | 597b8d120050210e4ef9928e911554c4 | BAI         |
| 1192 | PBBPJK           | 0DE7E2_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/028e7c79-8776-4756-91fd-44700942f97a   | IGM_PBBPJK-0DE7E2_20221021_normal.aln.cram.crai  | 30214737b62a6fd0a33b9e5b0a03d2ad | CRAI        |
| 1193 | PBBPJK           | 0DE7E2_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/26397d10-bf5b-46b8-a937-e42413a16bb7   | IGM_PBBPJK-0DE7E2_20221021_normal.aln.cram       | d63b5660b60090d946354f6e5070b6fc | CRAM        |
| 1194 | PBBPJK           | 0DE7FQ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/ea319d92-7c4d-44ac-98bf-ff35fb18b698   | IGM_PBBPJK-0DE7FQ_20221021_tumor.aln.cram        | 2a82bdbfb9c062b27f5d69c2abe39d16 | CRAM        |
| 1195 | PBBPJK           | 0DE7FQ_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/597b6bb4-b244-417b-a6e7-22f54aa9350d   | IGM_PBBPJK-0DE7FQ_20221021_tumor.aln.cram.crai   | 9512cabe619025c077c76bd371653310 | CRAI        |
| 1196 | PBBPJK           | 0DE7CT_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ed288c3a-f073-40ea-bca5-2ead3eda7cd2   | IGM_PBBPJK-0DE7CT_20221024_bam.bai               | adaf0318fbed94404d08a20fd11f7f07 | BAI         |
| 1197 | PBBPJK           | 0DE7CT_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/dc528f54-da02-45f3-9654-5424a8372811   | IGM_PBBPJK-0DE7CT_20221024_merged_bam.bam        | 2dfdb0a2a73f96d0b27460e3ef6e3422 | BAM         |
| 1198 | PBBPJK           | 0DE7CT_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/6b39048b-0483-4c6a-9b0b-53d81db18186   | IGM_PBBPJK-0DE7CT_20221024_bam.bam               | 352017b9a490deda7fa6697125c3bc2e | BAM         |
| 1199 | PBBPJK           | 0DE7CT_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ba67b534-1a81-4aa5-8220-506228d2119a   | IGM_PBBPJK-0DE7CT_20221024_merged_bam.bai        | 11c74d2899193202a00a2a39cdcb641c | BAI         |
| 1200 | PBBNZV           | 0DFF77_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f763204f-e032-41a9-97e2-e3f7b0918e3e   | IGM_PBBNZV-0DFF77_20221130_bam.bam               | 67be47720764b1bddfd6a8674728d0cc | BAM         |
| 1201 | PBBNZV           | 0DFF77_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/aa51c332-ba2b-45f7-a0f5-c35a8c0e1bd8   | IGM_PBBNZV-0DFF77_20221130_merged_bam.bam        | c0d41a535d8229e7467e6c7bec86ec89 | BAM         |
| 1202 | PBBNZV           | 0DFF77_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/36390a28-3136-47b8-858f-51eedba9b6d0   | IGM_PBBNZV-0DFF77_20221130_bam.bai               | a8729caa4c1daea5e68eeb71ea0bb978 | BAI         |
| 1203 | PBBNZV           | 0DFF77_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3cf3c0f0-864e-4ad7-ad05-0a0c739ec9bf   | IGM_PBBNZV-0DFF77_20221130_merged_bam.bai        | 081faa8f66393d5c77276f0006885139 | BAI         |
| 1204 | PBBNZV           | 0DFF75_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/771017bd-2a74-4a54-93c4-023afe87ebdc   | IGM_PBBNZV-0DFF75_20221124_tumor.aln.cram.crai   | 37f7bb10586a028fb6740027f2daa462 | CRAI        |
| 1205 | PBBNZV           | 0DFF75_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/67c56419-0c64-421a-ad5b-86e44383c72a   | IGM_PBBNZV-0DFF75_20221124_tumor.aln.cram        | 7de041a9142473c0e7faf3bec56283f9 | CRAM        |
| 1206 | PBBNZV           | 0DFF6Y_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/6e8d724e-b522-4e0c-92c4-b22b79ee7940   | IGM_PBBNZV-0DFF6Y_20221124_normal.aln.cram       | bce5cb62e9a2e378cd31d073ea1036ba | CRAM        |
| 1207 | PBBNZV           | 0DFF6Y_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/f3664d1f-0bd3-42b5-91b6-4d4e08b2215d   | IGM_PBBNZV-0DFF6Y_20221124_normal.aln.cram.crai  | 52cd5217a97ea19efa8e7bdca46bbe16 | CRAI        |
| 1208 | PBBPCE           | 0DF0BH_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/1f9d8858-3e05-42b2-a9fa-28aec14fcd8a   | IGM_PBBPCE-0DF0BH_20221111_normal.aln.cram.crai  | d23ddbf0ef5baabc70329ca4bdf5614b | CRAI        |
| 1209 | PBBPCE           | 0DF0BH_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/893c3d4b-5f17-47b7-8edc-84cd190e87c8   | IGM_PBBPCE-0DF0BH_20221111_normal.aln.cram       | 9745e26e18684652d8d583c50f57fee9 | CRAM        |
| 1210 | PBBPCE           | 0DF0B9_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/aa1cf8a2-2e78-4eb6-a412-9f65515219b7   | IGM_PBBPCE-0DF0B9_20221111_tumor.aln.cram.crai   | dbb44003e8270091ce95c04db23abff8 | CRAI        |
| 1211 | PBBPCE           | 0DF0B9_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/8da8640d-43bc-4a3b-ac7c-a032120a3227   | IGM_PBBPCE-0DF0B9_20221111_tumor.aln.cram        | d8473c929e33ff99901b304223b080d7 | CRAM        |
| 1212 | PBBPCE           | 0DF0B7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d8eec1b9-4df3-4b6c-95df-6222f675bf56   | IGM_PBBPCE-0DF0B7_20221125_bam.bam               | 6430960c16e59e12f3ccb54bcaa85a45 | BAM         |
| 1213 | PBBPCE           | 0DF0B7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/144eb534-3781-4430-b281-169073026a24   | IGM_PBBPCE-0DF0B7_20221125_merged_bam.bai        | 59cfa18dec5edd113dac7c26ea1b5f3d | BAI         |
| 1214 | PBBPCE           | 0DF0B7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/37a97a8d-f92c-4068-896a-6e772b93caa1   | IGM_PBBPCE-0DF0B7_20221125_merged_bam.bam        | 39dedff0d6cf9884c5aea10920f7d76f | BAM         |
| 1215 | PBBPCE           | 0DF0B7_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/5ce0ec18-c17a-48d1-8508-d44e39365f98   | IGM_PBBPCE-0DF0B7_20221125_bam.bai               | 764f8c9837e5c763c98b5af29a0d6181 | BAI         |
| 1216 | PBBSJN           | 0DFKHQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/14a81e12-094c-4b66-9dc2-88226daeaf64   | IGM_PBBSJN-0DFKHQ_20221129_bam.bai               | d3688f450b052a572816831e5583d0f8 | BAI         |
| 1217 | PBBSJN           | 0DFKHQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/61391e0c-0bb3-4593-a37f-9ea0c2059a9b   | IGM_PBBSJN-0DFKHQ_20221129_bam.bam               | 97adfa53bda9a6b0661c17d7e6f6e2aa | BAM         |
| 1218 | PBBSJN           | 0DFKHQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f3dc115c-c755-4de1-88b5-3feb4170e02d   | IGM_PBBSJN-0DFKHQ_20221129_merged_bam.bam        | eda2ad8a83b8621fb4c947d077f461f5 | BAM         |
| 1219 | PBBSJN           | 0DFKHQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/45147ef0-293a-494d-ad24-8f1d1b178cfd   | IGM_PBBSJN-0DFKHQ_20221129_merged_bam.bai        | 72edeff375da1a544f562a2adaad0bae | BAI         |
| 1220 | PBBSJN           | 0DFKHT_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/19f2526f-8359-44a9-88a5-b5e46df898de   | IGM_PBBSJN-0DFKHT_20221130_normal.aln.cram       | 8b21afcbf0c02d36457fc2bd10c249df | CRAM        |
| 1221 | PBBSJN           | 0DFKHT_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/008ffc6d-71d4-4530-900d-aee300cf5ed2   | IGM_PBBSJN-0DFKHT_20221130_normal.aln.cram.crai  | 072a0bceaf80816ee5ae53d77eb2338c | CRAI        |
| 1222 | PBBSJN           | 0DFKHR_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/2cb721de-8841-48c4-a26c-cb3b64d1ff46   | IGM_PBBSJN-0DFKHR_20221130_tumor.aln.cram        | 4a89afd695b1bb410556f04504836273 | CRAM        |
| 1223 | PBBSJN           | 0DFKHR_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/b2d5872a-4e6b-4fe5-af98-20400b26881d   | IGM_PBBSJN-0DFKHR_20221130_tumor.aln.cram.crai   | 82f3c6a64808d2424c7e1e60d2543f81 | CRAI        |
| 1224 | PBBPUG           | 0DENBT_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/d088bf29-b67d-4146-ada9-66eadda8497e   | IGM_PBBPUG-0DENBT_20221103_normal.aln.cram       | 771849e5dfa99dc42b708e5920ad7afd | CRAM        |
| 1225 | PBBPUG           | 0DENBT_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/8446cc1e-3e18-4300-87c3-2908246852d5   | IGM_PBBPUG-0DENBT_20221103_normal.aln.cram.crai  | 92d9aa1dbda8d9742e71e8b92e8aa975 | CRAI        |
| 1226 | PBBPUG           | 0DENCL_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/79e27a3d-910e-48b7-b601-e8e8d7a65b59   | IGM_PBBPUG-0DENCL_20221103_tumor.aln.cram        | f0555855e95773e728f3b31dec75d08f | CRAM        |
| 1227 | PBBPUG           | 0DENCL_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/82683757-e403-46ce-ac4e-fe587240720d   | IGM_PBBPUG-0DENCL_20221103_tumor.aln.cram.crai   | 99ede3714a8a551d10d972c6a7900dd8 | CRAI        |
| 1228 | PBBMZW           | 0DDX8R_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/ac00f545-3bbd-4543-a6bd-bfbb66e442a2   | IGM_PBBMZW-0DDX8R_20221015_tumor.aln.cram        | d32f0ff4eaf779f278e763ddd476631f | CRAM        |
| 1229 | PBBMZW           | 0DDX8R_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/4388e2a2-aeb0-483f-a002-623f0b313cc6   | IGM_PBBMZW-0DDX8R_20221015_tumor.aln.cram.crai   | 57808a63512007b50053c6453ffe525a | CRAI        |
| 1230 | PBBMZW           | 0DDX65_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/d7695de9-aa3a-4b95-a6e7-3272d8472c1e   | IGM_PBBMZW-0DDX65_20221015_normal.aln.cram       | 44af2e56fdd974db92d75009421946e4 | CRAM        |
| 1231 | PBBMZW           | 0DDX65_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/88b946bb-c801-4bf9-a7fd-b5991f07d5a6   | IGM_PBBMZW-0DDX65_20221015_normal.aln.cram.crai  | 2556886901b766590b526dce1839e334 | CRAI        |
| 1232 | PBBMZW           | 0DEK0Y_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a0cff4e3-fb2d-4f32-8c9e-3ce1ad3c3e92   | IGM_PBBMZW-0DEK0Y_20221026_bam.bai               | 76733453bdc608d0ea3177f519f863f4 | BAI         |
| 1233 | PBBMZW           | 0DEK0Y_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0bf36728-e4eb-4131-8484-9fe64f6c8569   | IGM_PBBMZW-0DEK0Y_20221026_bam.bam               | b4557553ef5c8bd3f9ac3c0f9625e963 | BAM         |
| 1234 | PBBMZW           | 0DEK0Y_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ca748ed9-dd6d-43f5-8482-0e1b67d6857c   | IGM_PBBMZW-0DEK0Y_20221026_merged_bam.bam        | 62d48fd98a36ac08e9007aefd00c1eed | BAM         |
| 1235 | PBBMZW           | 0DEK0Y_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3597ea59-728f-4e12-b8b7-e1e6a5832429   | IGM_PBBMZW-0DEK0Y_20221026_merged_bam.bai        | d9ae612c9b6ec43b5d34cd811a259231 | BAI         |
| 1236 | PBBLYY           | 0DEV28_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/1231a10e-c4fb-4aa9-96ef-3dad63dd9b70   | IGM_PBBLYY-0DEV28_20221109_normal.aln.cram       | 6eb0bb10ce1a2c02fc389b0940f3e747 | CRAM        |
| 1237 | PBBLYY           | 0DEV28_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/a13a9dbd-060d-4117-9451-85c5d36a2d75   | IGM_PBBLYY-0DEV28_20221109_normal.aln.cram.crai  | c921075df517c23159061c719be94c66 | CRAI        |
| 1238 | PBBLYY           | 0DEV49_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/ae456830-5f39-46c3-8eb8-265361339c80   | IGM_PBBLYY-0DEV49_20221109_tumor.aln.cram        | 5403b702bfc0158c9985aa0a9e2b1a20 | CRAM        |
| 1239 | PBBLYY           | 0DEV49_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/32b6350b-4c23-42a2-a5f0-f600f27f5c0c   | IGM_PBBLYY-0DEV49_20221109_tumor.aln.cram.crai   | 661e3f54885ea14b39ab39cf09f6b38b | CRAI        |
| 1240 | PBBLYY           | 0DEV3A_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e0de7139-6926-43b7-8b98-3dd1bf2dfb9d   | IGM_PBBLYY-0DEV3A_20221122_merged_bam.bai        | c0df63ca87b5f3c6412707ded2302519 | BAI         |
| 1241 | PBBLYY           | 0DEV3A_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/078ba6a4-6a5d-49c2-9ba4-5f48ab47b98b   | IGM_PBBLYY-0DEV3A_20221122_merged_bam.bam        | 90a21a7edbbbf668c5502e2806565f28 | BAM         |
| 1242 | PBBLYY           | 0DEV3A_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/35e15c8d-d5b9-4779-9609-7608c4b9b1fb   | IGM_PBBLYY-0DEV3A_20221122_bam.bai               | dce5304a58145b76f6be56b9393f4342 | BAI         |
| 1243 | PBBLYY           | 0DEV3A_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c2e3ce8b-0e5e-4653-ad8b-5798481c14b2   | IGM_PBBLYY-0DEV3A_20221122_bam.bam               | c78a5ad7bf0ccd420a3eea2ba8a975d5 | BAM         |
| 1244 | PBBNXZ           | 0DDX64_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/85ff0ace-ea03-48cb-848d-a82ebe93ce11   | IGM_PBBNXZ-0DDX64_20221014_normal.aln.cram.crai  | 64619b2a756cbda4fcf94c579c6077c3 | CRAI        |
| 1245 | PBBNXZ           | 0DDX64_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/7c7f0c30-fb7b-4931-9983-cb9dcce3ecfb   | IGM_PBBNXZ-0DDX64_20221014_normal.aln.cram       | 9e9166d4decdac1cdeaceb7dcda7317d | CRAM        |
| 1246 | PBBNXZ           | 0DDX69_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/59b7c1b0-c7b1-41e9-bcae-caf5a8a6c844   | IGM_PBBNXZ-0DDX69_20221014_tumor.aln.cram        | f46de9f5029927d011a2894c132283b6 | CRAM        |
| 1247 | PBBNXZ           | 0DDX69_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/447981cc-0c87-4865-a62e-5ac84814c4ea   | IGM_PBBNXZ-0DDX69_20221014_tumor.aln.cram.crai   | f79d3caae376691ddcae0aab7957b247 | CRAI        |
| 1248 | PBBNXZ           | 0DEJZZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b3960355-044e-40a7-8ca1-75b0f6eb73f0   | IGM_PBBNXZ-0DEJZZ_20221026_merged_bam.bai        | 6357425b24176399a9fff4b261364c51 | BAI         |
| 1249 | PBBNXZ           | 0DEJZZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/dd654b25-3c6c-4895-8690-bf1488a95f27   | IGM_PBBNXZ-0DEJZZ_20221026_bam.bai               | 4b1f797a1d3cae3e76b960ddf82e4c2d | BAI         |
| 1250 | PBBNXZ           | 0DEJZZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/03915455-d865-499c-9902-e26b7ae17428   | IGM_PBBNXZ-0DEJZZ_20221026_bam.bam               | e0c48b488b81d38f309898562002e21b | BAM         |
| 1251 | PBBNXZ           | 0DEJZZ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/addb38d4-c204-4db1-9840-d03190152be1   | IGM_PBBNXZ-0DEJZZ_20221026_merged_bam.bam        | f86eef8b3f8486b4c5ade2ebabb3d038 | BAM         |
| 1252 | PBBPWH           | 0DENBY_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b37d6a35-2252-4fe6-8a3d-d9afe93bb1f2   | IGM_PBBPWH-0DENBY_20221103_normal.aln.cram.crai  | e88c4bf30f227861fa65265ad7c71543 | CRAI        |
| 1253 | PBBPWH           | 0DENBY_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/ad24dbc0-ff7b-4fb5-90ac-37de031f7dd9   | IGM_PBBPWH-0DENBY_20221103_normal.aln.cram       | b43e09b42e4ee7e3c3cb2c464a4fe1af | CRAM        |
| 1254 | PBBPWH           | 0DENCU_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/bf20defd-58b7-45b0-a1d3-47bf5114f3f5   | IGM_PBBPWH-0DENCU_20221103_tumor.aln.cram        | f6accba59857f0fb6536592221170b39 | CRAM        |
| 1255 | PBBPWH           | 0DENCU_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/c46abb33-58f5-4876-8033-edfcc1db60e7   | IGM_PBBPWH-0DENCU_20221103_tumor.aln.cram.crai   | 6acdcf5c02dcc37d0cfcf3d4a75f392f | CRAI        |
| 1256 | PBBPWH           | 0DENCF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/dcf797ab-55b9-479b-b9f8-81479acf7bd8   | IGM_PBBPWH-0DENCF_20221112_merged_bam.bam        | a658ed3c21e35d50896eaa163347fddd | BAM         |
| 1257 | PBBPWH           | 0DENCF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a77fd67b-78ce-4ad4-b9fd-27b785fef8c5   | IGM_PBBPWH-0DENCF_20221112_bam.bai               | 0bc2f6881956ea167bb294fa695d4aca | BAI         |
| 1258 | PBBPWH           | 0DENCF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3d4d3081-c863-4905-8cb6-a1b0b6b25d1c   | IGM_PBBPWH-0DENCF_20221112_merged_bam.bai        | ea08a3c7c247fbe174310d3e9f18d713 | BAI         |
| 1259 | PBBPWH           | 0DENCF_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3e0a8fb1-b8eb-423a-b48b-befdd9d8f9c0   | IGM_PBBPWH-0DENCF_20221112_bam.bam               | 330ef96f259f494871b3848866d118a9 | BAM         |
| 1260 | PBBSYH           | 0DJE31_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b488accc-4450-4358-9792-29c9e8802679   | IGM_PBBSYH-0DJE31_20230316_bam.bam               | d5525ff36a290d354bc754e275f52092 | BAM         |
| 1261 | PBBSYH           | 0DJE31_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7cd3e6ba-b832-4807-8c31-19500ecf6371   | IGM_PBBSYH-0DJE31_20230316_merged_bam.bam        | 582b3ab8e9a680be882df7354d22a1b1 | BAM         |
| 1262 | PBBSYH           | 0DJE31_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/78830629-0c5e-4bcc-819c-410c42d9a2cc   | IGM_PBBSYH-0DJE31_20230316_merged_bam.bai        | 40635a3ec5ebb2dd3e6fb3e2e5c2ba3a | BAI         |
| 1263 | PBBSYH           | 0DJE31_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2248c9fa-5111-4c15-802d-a0201bcd951a   | IGM_PBBSYH-0DJE31_20230316_bam.bai               | a6c017064d1b4dd44e07673ea55012e1 | BAI         |
| 1264 | PBBMIY           | 0DD6SO_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/2e2a3d50-d0fb-4aa5-99dd-a9009ab52ac5   | IGM_PBBMIY-0DD6SO_20220923_bam.bai               | 68e8620c10b9585d4e5feed7826d6ce5 | BAI         |
| 1265 | PBBMIY           | 0DD6SO_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/babbb930-66e2-471e-8f9f-d9db0434c762   | IGM_PBBMIY-0DD6SO_20220923_merged_bam.bam        | 060f71279f1a511101143f92ad19e6a0 | BAM         |
| 1266 | PBBMIY           | 0DD6SO_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/aad7aa3b-7e98-4bf9-a06b-bdbf2edf8c43   | IGM_PBBMIY-0DD6SO_20220923_merged_bam.bai        | 1232d0f4fd0ed7a588cfa72f348d7239 | BAI         |
| 1267 | PBBMIY           | 0DD6SO_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8616a265-8af8-4fe0-9d07-8b7d47491936   | IGM_PBBMIY-0DD6SO_20220923_bam.bam               | c8883858636a2d93009d068877c26001 | BAM         |
| 1268 | PBBISR           | 0D9MK6_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/908b7307-949d-4268-8458-ef1499516e02   | IGM_PBBISR-0D9MK6_20220902a_normal.aln.cram      | 7c0072fc0c326deee8dde97a7145b3be | CRAM        |
| 1269 | PBBISR           | 0D9MK6_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/fdc3c9f1-640c-45a0-89ae-ae48c2e34d36   | IGM_PBBISR-0D9MK6_20220902a_normal.aln.cram.crai | f470f39fb5a779866e3af538f4f65e36 | CRAI        |
| 1270 | PBBISR           | 0D9MK1_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/570c2f50-5d47-4448-badd-fc0b5c639366   | IGM_PBBISR-0D9MK1_20220902a_tumor.aln.cram       | 2b3e60c5c17d6dbf26e05448867acaf8 | CRAM        |
| 1271 | PBBISR           | 0D9MK1_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/fab732bf-eb5f-4f68-bc1b-8dd360c08037   | IGM_PBBISR-0D9MK1_20220902a_tumor.aln.cram.crai  | 36a2a54e715437f3471eef9075d58c05 | CRAI        |
| 1272 | PBBISR           | 0D9MK0_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/be96b11b-9e2a-490a-b6e7-b861d8e4757b   | IGM_PBBISR-0D9MK0_20220919_merged_bam.bam        | 104e5546d783350c09d715cc117550a6 | BAM         |
| 1273 | PBBISR           | 0D9MK0_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e093fdd8-ebf8-4ebd-9f24-6c1deaedc22e   | IGM_PBBISR-0D9MK0_20220919_bam.bam               | 92fcf5c418ba82c58e3ead83b9e98b9d | BAM         |
| 1274 | PBBISR           | 0D9MK0_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ce69c3e3-da97-426f-ac81-d67f98031fd4   | IGM_PBBISR-0D9MK0_20220919_bam.bai               | a0d6eaa39bfbfec55d5308738d2732a6 | BAI         |
| 1275 | PBBISR           | 0D9MK0_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e069cebf-37b5-4559-8c83-b53fd4be7457   | IGM_PBBISR-0D9MK0_20220919_merged_bam.bai        | 5dbce844f22cb74b3fa29819622dd6ee | BAI         |
| 1276 | PBBJIL           | 0DA8LA_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/9928d819-ff1f-42e1-ae7d-9f46caf2f9d1   | IGM_PBBJIL-0DA8LA_20220902a_normal.aln.cram      | a559c4253da636c7467196ae748ca1d9 | CRAM        |
| 1277 | PBBJIL           | 0DA8LA_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b03b04af-e293-48f7-9499-21a144959a92   | IGM_PBBJIL-0DA8LA_20220902a_normal.aln.cram.crai | 8583988233542d1c16d72a25d91e88b8 | CRAI        |
| 1278 | PBBJIL           | 0DA8MA_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/272fe604-cf31-47dd-a7be-30e4c58f4373   | IGM_PBBJIL-0DA8MA_20220902a_tumor.aln.cram.crai  | ccc07406f0f8e870986f9873227e6c1d | CRAI        |
| 1279 | PBBJIL           | 0DA8MA_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/f27d3f3c-f15a-4900-98b4-4c89f43831c9   | IGM_PBBJIL-0DA8MA_20220902a_tumor.aln.cram       | ac31ba7c2563df3a4a24d84275c5db09 | CRAM        |
| 1280 | PBBJIL           | 0DA8MC_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/1102ca82-3ed6-41db-a959-6ad3d3ec335b   | IGM_PBBJIL-0DA8MC_20220919_merged_bam.bam        | ce928f2c45ae6490b4ef0397662a6569 | BAM         |
| 1281 | PBBJIL           | 0DA8MC_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ba8e6d4c-de44-4e29-89a6-c9401b5057e5   | IGM_PBBJIL-0DA8MC_20220919_bam.bam               | 222e8770428f8e70cfc6e6eb97701fcf | BAM         |
| 1282 | PBBJIL           | 0DA8MC_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/38aedcb6-8117-4d3d-8b0a-ac895c708d84   | IGM_PBBJIL-0DA8MC_20220919_merged_bam.bai        | 8394421c2dd66b1a781af82b5bda6c15 | BAI         |
| 1283 | PBBJIL           | 0DA8MC_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/854304da-5513-4e71-a917-a2f340c6199a   | IGM_PBBJIL-0DA8MC_20220919_bam.bai               | bfb710b7b55260827f5e79bd4b82c9ae | BAI         |
| 1284 | PBBWHU           | 0DIW6C_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/a7d8aac6-459b-4f0e-a533-b8bb8d90cb63   | IGM_PBBWHU-0DIW6C_20230304_normal.aln.cram.crai  | f17f3173d739eda68537094717e45b61 | CRAI        |
| 1285 | PBBWHU           | 0DIW6C_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/16152398-3a16-4e2d-b1e2-cd5c70c3a8f0   | IGM_PBBWHU-0DIW6C_20230304_normal.aln.cram       | ff0f2e09b63bbab67989b415a5644348 | CRAM        |
| 1286 | PBBWHU           | 0DIW88_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/1dbd8870-baa0-43a5-96fc-6d5b3e399ec5   | IGM_PBBWHU-0DIW88_20230304_tumor.aln.cram.crai   | faaad83e6c4ac88db571d31499cd8dd2 | CRAI        |
| 1287 | PBBWHU           | 0DIW88_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/272c24c6-2949-4028-8514-6a7f387cd5fd   | IGM_PBBWHU-0DIW88_20230304_tumor.aln.cram        | a942b1abfd2da7d214666d5b2ce58da5 | CRAM        |
| 1288 | PBBWHU           | 0DIW78_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/6f07a855-7675-4080-93eb-efe7a4313aa3   | IGM_PBBWHU-0DIW78_20230228_bam.bai               | 4f08ebc290d526da420d3097a8d88f70 | BAI         |
| 1289 | PBBWHU           | 0DIW78_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c561fd9e-bedb-4ae1-b21d-9706a8765185   | IGM_PBBWHU-0DIW78_20230228_merged_bam.bai        | 18455cc4f23c76bb2b38bd01ec641339 | BAI         |
| 1290 | PBBWHU           | 0DIW78_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/a2898ef8-1902-43c1-a2ea-b4d7d133cc40   | IGM_PBBWHU-0DIW78_20230228_bam.bam               | 202ce879e72276a64db97f9b520bff19 | BAM         |
| 1291 | PBBWHU           | 0DIW78_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/d7b11b38-2c0e-4753-8ebe-0237c02ce65b   | IGM_PBBWHU-0DIW78_20230228_merged_bam.bam        | 100107f7a88d4146777245321080ac45 | BAM         |
| 1292 | PBBKIL           | 0DAOBR_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/21351d75-6dac-47bb-80d1-1f36a121d743   | IGM_PBBKIL-0DAOBR_20220902a_tumor.aln.cram       | e0ac5d418773741bccb1de5aaff92f1c | CRAM        |
| 1293 | PBBKIL           | 0DAOBR_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/f0d7c939-d821-4245-bd03-370ed31d4cdc   | IGM_PBBKIL-0DAOBR_20220902a_tumor.aln.cram.crai  | 33041c0da6e321095be895e0a10fb88e | CRAI        |
| 1294 | PBBKIL           | 0DAOBQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/e8304eba-dd34-4aac-a2f6-ef359b79c3b5   | IGM_PBBKIL-0DAOBQ_20220707_merged_bam.bai        | e44fbf60c925a624ccd19cd36f89acf5 | BAI         |
| 1295 | PBBKIL           | 0DAOBQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8bc06b82-5a92-456c-b7ac-15066c448e6b   | IGM_PBBKIL-0DAOBQ_20220707_bam.bai               | 643d12cbb36fc40d035d43bdb668e548 | BAI         |
| 1296 | PBBKIL           | 0DAOBQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/30067049-41b0-4e05-b466-2d7afc51ac22   | IGM_PBBKIL-0DAOBQ_20220707_merged_bam.bam        | 65d123dcbd9c9b82668e73401960e27e | BAM         |
| 1297 | PBBKIL           | 0DAOBQ_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/138f06e0-6a25-44b4-a32a-a0ad19abfadb   | IGM_PBBKIL-0DAOBQ_20220707_bam.bam               | 8ed7d5fa5bb6df193b5f20cff34d7c99 | BAM         |
| 1298 | PBBKIL           | 0DAOBS_DNA  | Central Nervous System | Normal         | DNA           | dg.4DFC/e06a5c9c-854b-49f6-b9b2-3dee0a21e370   | IGM_PBBKIL-0DAOBS_20220902a_normal.aln.cram      | 296c7151fe8fd95d7f6649ecec527b97 | CRAM        |
| 1299 | PBBKIL           | 0DAOBS_DNA  | Central Nervous System | Normal         | DNA           | dg.4DFC/ba7edcf1-faa1-40cd-ad01-d0a6006e6867   | IGM_PBBKIL-0DAOBS_20220902a_normal.aln.cram.crai | 04cd3d5c737f1335b5561e6f03fc4206 | CRAI        |
| 1300 | PBBXHW           | 0DK0AN_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/5412274c-e9cc-4fa1-8ea4-9352457d2dd0   | IGM_PBBXHW-0DK0AN_20230331_normal.aln.cram       | 65eea78bae0568bbbe360cfb38217921 | CRAM        |
| 1301 | PBBXHW           | 0DK0AN_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b85dacfd-de3f-44a0-be23-7d3a2ff9a7c3   | IGM_PBBXHW-0DK0AN_20230331_normal.aln.cram.crai  | 8b0db584d129f28e90b3684ee45918d8 | CRAI        |
| 1302 | PBBXHW           | 0DK0AA_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/0eba4c8e-6d11-4e12-84ab-0a2a34de0d9a   | IGM_PBBXHW-0DK0AA_20230331_merged_bam.bai        | b1b9ff0159dee44389985aaef5cfacae | BAI         |
| 1303 | PBBXHW           | 0DK0AA_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/36eeb2e0-e85b-4bc2-b80f-123eb1fec966   | IGM_PBBXHW-0DK0AA_20230331_merged_bam.bam        | 6c5e87d82da39ed65086426d5ba74770 | BAM         |
| 1304 | PBBXHW           | 0DK0AA_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/1ed7b56d-04ed-436a-b482-150f71a45ab6   | IGM_PBBXHW-0DK0AA_20230331_bam.bam               | c3e25e73bfea848bccee1086e1f00fee | BAM         |
| 1305 | PBBXHW           | 0DK0AA_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/bf4b7c9d-f6af-48b4-b056-aff47bce6885   | IGM_PBBXHW-0DK0AA_20230331_bam.bai               | dae834628ba958781e5a547892083a9e | BAI         |
| 1306 | PBBXHW           | 0DK0AL_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/118959fb-3ce6-44f6-8648-a7c48618a644   | IGM_PBBXHW-0DK0AL_20230331_tumor.aln.cram        | f906205a6fef34d612c038931a4e5fc1 | CRAM        |
| 1307 | PBBXHW           | 0DK0AL_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/f5797e0c-fb98-4aba-92de-67466b325697   | IGM_PBBXHW-0DK0AL_20230331_tumor.aln.cram.crai   | 6fc5be81d59095f0e9caecf1924528c5 | CRAI        |
| 1308 | PBBUYL           | 0DI6KM_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/aa34d026-81ce-438d-ba10-de52cf625f89   | IGM_PBBUYL-0DI6KM_20230215_normal.aln.cram.crai  | d7dd7da40fe905f1f77736408e1fc228 | CRAI        |
| 1309 | PBBUYL           | 0DI6KM_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/2436778e-b40f-4427-b448-a162c826bcbc   | IGM_PBBUYL-0DI6KM_20230215_normal.aln.cram       | bfa7430f0e1c27cda6b5a4c21740859a | CRAM        |
| 1310 | PBBUYL           | 0DI6KX_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/1c106c04-0139-4d3f-bf09-a20fc60f8910   | IGM_PBBUYL-0DI6KX_20230215_tumor.aln.cram.crai   | 4da118f7799f4b7ceb2fd02292c2bf98 | CRAI        |
| 1311 | PBBUYL           | 0DI6KX_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/37ff0c7c-31e4-48ec-a6fc-abed0bc89af8   | IGM_PBBUYL-0DI6KX_20230215_tumor.aln.cram        | 4ed9dbb7cea3d3d9703e70e8b5105777 | CRAM        |
| 1312 | PBBUYL           | 0DI6KR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/930c6424-8587-4452-a835-d890fd7cc439   | IGM_PBBUYL-0DI6KR_20230216_bam.bai               | 8eed50c87439f69f35612ac5ba3e22f9 | BAI         |
| 1313 | PBBUYL           | 0DI6KR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/b202e63c-50e3-4d52-b7af-124b594781d0   | IGM_PBBUYL-0DI6KR_20230216_merged_bam.bai        | 09402e9588c5dff68186611ed4ef43d0 | BAI         |
| 1314 | PBBUYL           | 0DI6KR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/ee22dc2d-bb8b-43d0-86b4-e9b275a069aa   | IGM_PBBUYL-0DI6KR_20230216_merged_bam.bam        | 3a56e57a2c32e96f7b11c1b42025fefd | BAM         |
| 1315 | PBBUYL           | 0DI6KR_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f24b5d7b-c184-4649-8f1c-69a7c71601dc   | IGM_PBBUYL-0DI6KR_20230216_bam.bam               | 594db5109c352331ee74d1311a3682d1 | BAM         |
| 1316 | PBBTGI           | 0DGFVU_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/ace0aa11-08e8-4248-918b-b2965f5a202b   | IGM_PBBTGI-0DGFVU_20221223_tumor.aln.cram.crai   | 3d79b6a4ae5a4ed6fd9e5c5f27055348 | CRAI        |
| 1317 | PBBTGI           | 0DGFVU_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/c786fc0b-42e7-43ea-b29c-d07aa45591a0   | IGM_PBBTGI-0DGFVU_20221223_tumor.aln.cram        | 645d9b20831c6992f6b4449a42fb6aa3 | CRAM        |
| 1318 | PBBTGI           | 0DGFTL_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/4c3d24d6-1d53-4590-9f4e-0846650ed9fb   | IGM_PBBTGI-0DGFTL_20221223_normal.aln.cram.crai  | 85b73615a70d2fac8bd47ea2cfda8b27 | CRAI        |
| 1319 | PBBTGI           | 0DGFTL_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/c4584d0b-facd-4ba6-8e02-0f2cf800ea38   | IGM_PBBTGI-0DGFTL_20221223_normal.aln.cram       | c3127c38da7890ed1eeb70c64a862e29 | CRAM        |
| 1320 | PBBTGI           | 0DGFW4_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/58aeec7c-9d25-4a95-870f-5188b52ddb30   | IGM_PBBTGI-0DGFW4_20221231_bam.bam               | 55bf7c10c3114096daa2721b5b5826bb | BAM         |
| 1321 | PBBTGI           | 0DGFW4_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/1ed9d771-bde8-4fa3-bbc9-fb9f43d42dde   | IGM_PBBTGI-0DGFW4_20221231_bam.bai               | 0c7038b7a7bb4b0ee9d5795d1cef3395 | BAI         |
| 1322 | PBBTGI           | 0DGFW4_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/79892e79-33f1-4133-b661-7ae361d92f9e   | IGM_PBBTGI-0DGFW4_20221231_merged_bam.bam        | 1fc9c56adb9aac0f991413c255124ac0 | BAM         |
| 1323 | PBBTGI           | 0DGFW4_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7a2160ed-6903-4cc3-87b3-85d564bee642   | IGM_PBBTGI-0DGFW4_20221231_merged_bam.bai        | 227c47197432d663f024f1badbee7081 | BAI         |
| 1324 | PBBRUJ           | 0DK1V2_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/9509d586-4f6f-40dc-bd61-7bab1e729b20   | IGM_PBBRUJ-0DK1V2_20230405_bam.bai               | 613a6f51afae3d26226f4f0448b8a335 | BAI         |
| 1325 | PBBRUJ           | 0DK1V2_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/7c197818-59f5-4982-9422-48e8bb2603ff   | IGM_PBBRUJ-0DK1V2_20230405_merged_bam.bai        | 816e24354e38eccb35729bba3d32ec80 | BAI         |
| 1326 | PBBRUJ           | 0DK1V2_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/827dd69e-9228-4e80-9a70-35222f46441a   | IGM_PBBRUJ-0DK1V2_20230405_merged_bam.bam        | 4a0ee1babee09922389bcaad4e853683 | BAM         |
| 1327 | PBBRUJ           | 0DK1V2_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/57f4ce6d-fb3b-4ef1-ac21-4a3f6d3d0dfe   | IGM_PBBRUJ-0DK1V2_20230405_bam.bam               | c603944caf1e7ddae53d5f85309b3e52 | BAM         |
| 1328 | PBBTUH           | 0DGQBD_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/1111751a-b2b7-430f-800b-2acfc7d1b75a   | IGM_PBBTUH-0DGQBD_20230103_tumor.aln.cram.crai   | 80376553cf2dc2a68f8926c31373cb9f | CRAI        |
| 1329 | PBBTUH           | 0DGQBD_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/614213fd-b3b9-4d19-9ed8-08e58fb30bf2   | IGM_PBBTUH-0DGQBD_20230103_tumor.aln.cram        | 41ba0119fa57a5951c7b4ff4d1edf13d | CRAM        |
| 1330 | PBBTUH           | 0DGQAH_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/a98d89be-1a8f-468e-8ad8-834c3ac9ca69   | IGM_PBBTUH-0DGQAH_20230103_normal.aln.cram       | d553a676cfbcb5c17366e4149901e510 | CRAM        |
| 1331 | PBBTUH           | 0DGQAH_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b95b8910-a7da-4b38-928c-24b304f909f8   | IGM_PBBTUH-0DGQAH_20230103_normal.aln.cram.crai  | 75f4bf489ad1c023d42601a2efefc982 | CRAI        |
| 1332 | PBBTUH           | 0DGQAL_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/af917839-c242-45c8-a29a-cd4c3fbd1b50   | IGM_PBBTUH-0DGQAL_20221229_merged_bam.bai        | d53e918e9d5a102acd1af7ab085257b2 | BAI         |
| 1333 | PBBTUH           | 0DGQAL_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/69991ac9-0d5c-4015-b7d4-aee8e1d1ad5d   | IGM_PBBTUH-0DGQAL_20221229_bam.bai               | 71bc61d52c9117647da217401aa71678 | BAI         |
| 1334 | PBBTUH           | 0DGQAL_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f8f6f95e-6cf2-49ec-8f4d-4e555bad8829   | IGM_PBBTUH-0DGQAL_20221229_bam.bam               | 4622e58144a0ddfc82d1cac9c4d9dac1 | BAM         |
| 1335 | PBBTUH           | 0DGQAL_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/5124dc85-6793-4f4b-881a-891ac8236076   | IGM_PBBTUH-0DGQAL_20221229_merged_bam.bam        | 5cd7574442fac250846706d331d568a9 | BAM         |
| 1336 | PBBJPZ           | 0DAF7Q_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/d77fff8a-0993-4425-a71a-14c12f9747d9   | IGM_PBBJPZ-0DAF7Q_20220902a_tumor.aln.cram.crai  | a3abeec5137ffc2e0ebab38d056f6d73 | CRAI        |
| 1337 | PBBJPZ           | 0DAF7Q_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/91c2f165-d0d1-4a16-9483-a8592d85dced   | IGM_PBBJPZ-0DAF7Q_20220902a_tumor.aln.cram       | 20f15438546ec49e4a9be2067af2b11e | CRAM        |
| 1338 | PBBJPZ           | 0DAF7S_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/b21484cf-dfe0-4b2f-972b-c217826ea1c1   | IGM_PBBJPZ-0DAF7S_20220902a_normal.aln.cram.crai | eb3cd50f96333de851a1382ed1899ae6 | CRAI        |
| 1339 | PBBJPZ           | 0DAF7S_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/c51368b5-4b42-4fe5-8eca-66568dc37822   | IGM_PBBJPZ-0DAF7S_20220902a_normal.aln.cram      | a8fd1160b512b4b82367d0f7d594b882 | CRAM        |
| 1340 | PBBJPZ           | 0DAF7A_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/86b8857a-3c11-4a52-aa7d-a11c8e4ad85a   | IGM_PBBJPZ-0DAF7A_20220628_bam.bai               | 576787a5b0f5598bb513e8022eeeadcd | BAI         |
| 1341 | PBBJPZ           | 0DAF7A_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/760047d2-0acb-4c05-9a34-77f8e4c7451e   | IGM_PBBJPZ-0DAF7A_20220628_bam.bam               | d4392023610cfab43a7718bb97f674be | BAM         |
| 1342 | PBBJPZ           | 0DAF7A_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/3e090511-e815-4bf0-ae38-cbb228980258   | IGM_PBBJPZ-0DAF7A_20220628_merged_bam.bai        | c540261142cae03773347c6135d1ba49 | BAI         |
| 1343 | PBBJPZ           | 0DAF7A_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/dccc9bc1-9d26-4fa9-8a2f-0f89b393c379   | IGM_PBBJPZ-0DAF7A_20220628_merged_bam.bam        | 6c28c190e12f94783ee45fe7fcca63ab | BAM         |
| 1344 | PBBNZF           | 0DDZ60_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/ea238d58-cb9d-4e28-9161-e22721a15633   | IGM_PBBNZF-0DDZ60_20221018_normal.aln.cram.crai  | 0acb7856a040e3180d7876f73d44013d | CRAI        |
| 1345 | PBBNZF           | 0DDZ60_DNA  | Blood                  | Normal         | DNA           | dg.4DFC/26ab449a-660a-4af7-af5b-00a29b4c4034   | IGM_PBBNZF-0DDZ60_20221018_normal.aln.cram       | 445fe193e60b6cac27a2ed7ec7ede807 | CRAM        |
| 1346 | PBBNZF           | 0DDZ6X_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/ff77a76c-7e70-48dc-9a78-d78a1fa30cec   | IGM_PBBNZF-0DDZ6X_20221018_tumor.aln.cram.crai   | e1afc12b5ec4168a57cae73658aa423f | CRAI        |
| 1347 | PBBNZF           | 0DDZ6X_DNA  | Central Nervous System | Tumor          | DNA           | dg.4DFC/bd5b8365-dca6-492b-b74c-916156deb7ef   | IGM_PBBNZF-0DDZ6X_20221018_tumor.aln.cram        | d2fd509d45db735d4e48c4482d7c5b05 | CRAM        |
| 1348 | PBBNZF           | 0DEK01_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/8d2b381b-eae2-4da8-a1a7-9a9a7985c9d1   | IGM_PBBNZF-0DEK01_20221028_merged_bam.bam        | c1600df25c15398a9ababd1fc063f3f9 | BAM         |
| 1349 | PBBNZF           | 0DEK01_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/f9964ce3-f7aa-49f4-b173-56b8b2ec1945   | IGM_PBBNZF-0DEK01_20221028_merged_bam.bai        | c08d5c2b58e74aa695a4df3d1b3cde82 | BAI         |
| 1350 | PBBNZF           | 0DEK01_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/06d08c69-fbc2-4db6-bb95-ab743e5ab748   | IGM_PBBNZF-0DEK01_20221028_bam.bai               | f92f98a3e81d1bb0b224d08455472a8d | BAI         |
| 1351 | PBBNZF           | 0DEK01_RNA  | Central Nervous System | Tumor          | RNA           | dg.4DFC/c015713a-a1b1-4bf7-a932-3c10d5c5d1f6   | IGM_PBBNZF-0DEK01_20221028_bam.bam               | 38c6f438eb1dd91046380da00c100785 | BAM         |

We've now gotten to a point where we can use this information to build a file that we can take to one of the NCI Cloud Resources and start doing analytical work on the files.  CRDC uses a simple comma-separated file to transfer basic information about files, and it has the following headers
  -  drs_uri    This is the DRS URI that is used to access a file.  The CDS file_id field (File ID above) is most of the DRS URI, it just needs to be prepended by drs://nci-crdc.datacommons.io/
  -  name        This is the file name
  -  Participant ID
  -  Md5sum
  -  User Comment   Anything can go here, this is an open field

Note that the columns must appear in the exact order and format as shown above

In [88]:
rna_df = genomic_df[genomic_df['Sample Type'] == 'RNA']

In [89]:
rna_df

,Participant ID,Sample ID,Sample Anatomic Site,Tumor Status,Sample Type,File ID,File Name,MD5SUM,File Type
2,PBCEET,0DPU2P_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/9290b0d0-95e4-4ee4-9443-377ebd64f09d,IGM_PBCEET-0DPU2P_20230906_merged_bam.bai,89e62149f208e54c34997524c138ef89,BAI
3,PBCEET,0DPU2P_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/329df1db-a36d-4e93-a448-b668f0ae5c72,IGM_PBCEET-0DPU2P_20230906_merged_bam.bam,fa70d90daee09bee463a50d8eb01dfa8,BAM
4,PBCEET,0DPU2P_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/28168dba-4108-40ac-8586-743ee6f74916,IGM_PBCEET-0DPU2P_20230906_bam.bam,f4a244aa13e28148feda3bf27c012060,BAM
5,PBCEET,0DPU2P_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/31a2459a-fd3a-4e2a-94ec-c39a0317a94e,IGM_PBCEET-0DPU2P_20230906_bam.bai,598631bca417bf7c9e3da9806e95bea5,BAI
10,PBCEGU,0DQ18G_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/02dbd10f-91f2-4464-84d1-d0ede87526c9,IGM_PBCEGU-0DQ18G_20230918_merged_bam.bam,73fd6cfbc8c1f97bcfa3054b6a8fb631,BAM
...,...,...,...,...,...,...,...,...,...
1343,PBBJPZ,0DAF7A_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/dccc9bc1-9d26-4fa9-8a2f-0f89b393c379,IGM_PBBJPZ-0DAF7A_20220628_merged_bam.bam,6c28c190e12f94783ee45fe7fcca63ab,BAM
1348,PBBNZF,0DEK01_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/8d2b381b-eae2-4da8-a1a7-9a9a7985c9d1,IGM_PBBNZF-0DEK01_20221028_merged_bam.bam,c1600df25c15398a9ababd1fc063f3f9,BAM
1349,PBBNZF,0DEK01_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/f9964ce3-f7aa-49f4-b173-56b8b2ec1945,IGM_PBBNZF-0DEK01_20221028_merged_bam.bai,c08d5c2b58e74aa695a4df3d1b3cde82,BAI
1350,PBBNZF,0DEK01_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/06d08c69-fbc2-4db6-bb95-ab743e5ab748,IGM_PBBNZF-0DEK01_20221028_bam.bai,f92f98a3e81d1bb0b224d08455472a8d,BAI


In [95]:
rna_df.rename({"File ID":"drs_uri", "File Name":"name", "MD5SUM":"Md5sum", "Tumor Status":"User Comment"})

,Participant ID,Sample ID,Sample Anatomic Site,User Comment,Sample Type,drs_uri,name,Md5sum,File Type
2,PBCEET,0DPU2P_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/9290b0d0-95e4-4ee4-9443-377ebd64f09d,IGM_PBCEET-0DPU2P_20230906_merged_bam.bai,89e62149f208e54c34997524c138ef89,BAI
3,PBCEET,0DPU2P_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/329df1db-a36d-4e93-a448-b668f0ae5c72,IGM_PBCEET-0DPU2P_20230906_merged_bam.bam,fa70d90daee09bee463a50d8eb01dfa8,BAM
4,PBCEET,0DPU2P_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/28168dba-4108-40ac-8586-743ee6f74916,IGM_PBCEET-0DPU2P_20230906_bam.bam,f4a244aa13e28148feda3bf27c012060,BAM
5,PBCEET,0DPU2P_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/31a2459a-fd3a-4e2a-94ec-c39a0317a94e,IGM_PBCEET-0DPU2P_20230906_bam.bai,598631bca417bf7c9e3da9806e95bea5,BAI
10,PBCEGU,0DQ18G_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/02dbd10f-91f2-4464-84d1-d0ede87526c9,IGM_PBCEGU-0DQ18G_20230918_merged_bam.bam,73fd6cfbc8c1f97bcfa3054b6a8fb631,BAM
...,...,...,...,...,...,...,...,...,...
1343,PBBJPZ,0DAF7A_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/dccc9bc1-9d26-4fa9-8a2f-0f89b393c379,IGM_PBBJPZ-0DAF7A_20220628_merged_bam.bam,6c28c190e12f94783ee45fe7fcca63ab,BAM
1348,PBBNZF,0DEK01_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/8d2b381b-eae2-4da8-a1a7-9a9a7985c9d1,IGM_PBBNZF-0DEK01_20221028_merged_bam.bam,c1600df25c15398a9ababd1fc063f3f9,BAM
1349,PBBNZF,0DEK01_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/f9964ce3-f7aa-49f4-b173-56b8b2ec1945,IGM_PBBNZF-0DEK01_20221028_merged_bam.bai,c08d5c2b58e74aa695a4df3d1b3cde82,BAI
1350,PBBNZF,0DEK01_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/06d08c69-fbc2-4db6-bb95-ab743e5ab748,IGM_PBBNZF-0DEK01_20221028_bam.bai,f92f98a3e81d1bb0b224d08455472a8d,BAI


In [96]:
rna_df.drop(columns=['Sample ID', 'Sample Anatomic Site','Sample Type', 'File Type'])

,Participant ID,User Comment,drs_uri,name,Md5sum
2,PBCEET,Tumor,dg.4DFC/9290b0d0-95e4-4ee4-9443-377ebd64f09d,IGM_PBCEET-0DPU2P_20230906_merged_bam.bai,89e62149f208e54c34997524c138ef89
3,PBCEET,Tumor,dg.4DFC/329df1db-a36d-4e93-a448-b668f0ae5c72,IGM_PBCEET-0DPU2P_20230906_merged_bam.bam,fa70d90daee09bee463a50d8eb01dfa8
4,PBCEET,Tumor,dg.4DFC/28168dba-4108-40ac-8586-743ee6f74916,IGM_PBCEET-0DPU2P_20230906_bam.bam,f4a244aa13e28148feda3bf27c012060
5,PBCEET,Tumor,dg.4DFC/31a2459a-fd3a-4e2a-94ec-c39a0317a94e,IGM_PBCEET-0DPU2P_20230906_bam.bai,598631bca417bf7c9e3da9806e95bea5
10,PBCEGU,Tumor,dg.4DFC/02dbd10f-91f2-4464-84d1-d0ede87526c9,IGM_PBCEGU-0DQ18G_20230918_merged_bam.bam,73fd6cfbc8c1f97bcfa3054b6a8fb631
...,...,...,...,...,...
1343,PBBJPZ,Tumor,dg.4DFC/dccc9bc1-9d26-4fa9-8a2f-0f89b393c379,IGM_PBBJPZ-0DAF7A_20220628_merged_bam.bam,6c28c190e12f94783ee45fe7fcca63ab
1348,PBBNZF,Tumor,dg.4DFC/8d2b381b-eae2-4da8-a1a7-9a9a7985c9d1,IGM_PBBNZF-0DEK01_20221028_merged_bam.bam,c1600df25c15398a9ababd1fc063f3f9
1349,PBBNZF,Tumor,dg.4DFC/f9964ce3-f7aa-49f4-b173-56b8b2ec1945,IGM_PBBNZF-0DEK01_20221028_merged_bam.bai,c08d5c2b58e74aa695a4df3d1b3cde82
1350,PBBNZF,Tumor,dg.4DFC/06d08c69-fbc2-4db6-bb95-ab743e5ab748,IGM_PBBNZF-0DEK01_20221028_bam.bai,f92f98a3e81d1bb0b224d08455472a8d


In [98]:
rna_df.loc[:,['drs_uri', 'name', 'Participant ID', 'Md5sum', 'User Comment']]

,drs_uri,name,Participant ID,Md5sum,User Comment
2,dg.4DFC/9290b0d0-95e4-4ee4-9443-377ebd64f09d,IGM_PBCEET-0DPU2P_20230906_merged_bam.bai,PBCEET,89e62149f208e54c34997524c138ef89,Tumor
3,dg.4DFC/329df1db-a36d-4e93-a448-b668f0ae5c72,IGM_PBCEET-0DPU2P_20230906_merged_bam.bam,PBCEET,fa70d90daee09bee463a50d8eb01dfa8,Tumor
4,dg.4DFC/28168dba-4108-40ac-8586-743ee6f74916,IGM_PBCEET-0DPU2P_20230906_bam.bam,PBCEET,f4a244aa13e28148feda3bf27c012060,Tumor
5,dg.4DFC/31a2459a-fd3a-4e2a-94ec-c39a0317a94e,IGM_PBCEET-0DPU2P_20230906_bam.bai,PBCEET,598631bca417bf7c9e3da9806e95bea5,Tumor
10,dg.4DFC/02dbd10f-91f2-4464-84d1-d0ede87526c9,IGM_PBCEGU-0DQ18G_20230918_merged_bam.bam,PBCEGU,73fd6cfbc8c1f97bcfa3054b6a8fb631,Tumor
...,...,...,...,...,...
1343,dg.4DFC/dccc9bc1-9d26-4fa9-8a2f-0f89b393c379,IGM_PBBJPZ-0DAF7A_20220628_merged_bam.bam,PBBJPZ,6c28c190e12f94783ee45fe7fcca63ab,Tumor
1348,dg.4DFC/8d2b381b-eae2-4da8-a1a7-9a9a7985c9d1,IGM_PBBNZF-0DEK01_20221028_merged_bam.bam,PBBNZF,c1600df25c15398a9ababd1fc063f3f9,Tumor
1349,dg.4DFC/f9964ce3-f7aa-49f4-b173-56b8b2ec1945,IGM_PBBNZF-0DEK01_20221028_merged_bam.bai,PBBNZF,c08d5c2b58e74aa695a4df3d1b3cde82,Tumor
1350,dg.4DFC/06d08c69-fbc2-4db6-bb95-ab743e5ab748,IGM_PBBNZF-0DEK01_20221028_bam.bai,PBBNZF,f92f98a3e81d1bb0b224d08455472a8d,Tumor


In [94]:
filename = "example_CDS_manifest.csv"
rna_df.to_csv(filename, sep=',', index=False)

,Participant ID,Sample ID,Sample Anatomic Site,User Comment,Sample Type,drs_uri,name,Md5sum,File Type
2,PBCEET,0DPU2P_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/9290b0d0-95e4-4ee4-9443-377ebd64f09d,IGM_PBCEET-0DPU2P_20230906_merged_bam.bai,89e62149f208e54c34997524c138ef89,BAI
3,PBCEET,0DPU2P_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/329df1db-a36d-4e93-a448-b668f0ae5c72,IGM_PBCEET-0DPU2P_20230906_merged_bam.bam,fa70d90daee09bee463a50d8eb01dfa8,BAM
4,PBCEET,0DPU2P_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/28168dba-4108-40ac-8586-743ee6f74916,IGM_PBCEET-0DPU2P_20230906_bam.bam,f4a244aa13e28148feda3bf27c012060,BAM
5,PBCEET,0DPU2P_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/31a2459a-fd3a-4e2a-94ec-c39a0317a94e,IGM_PBCEET-0DPU2P_20230906_bam.bai,598631bca417bf7c9e3da9806e95bea5,BAI
10,PBCEGU,0DQ18G_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/02dbd10f-91f2-4464-84d1-d0ede87526c9,IGM_PBCEGU-0DQ18G_20230918_merged_bam.bam,73fd6cfbc8c1f97bcfa3054b6a8fb631,BAM
...,...,...,...,...,...,...,...,...,...
1343,PBBJPZ,0DAF7A_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/dccc9bc1-9d26-4fa9-8a2f-0f89b393c379,IGM_PBBJPZ-0DAF7A_20220628_merged_bam.bam,6c28c190e12f94783ee45fe7fcca63ab,BAM
1348,PBBNZF,0DEK01_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/8d2b381b-eae2-4da8-a1a7-9a9a7985c9d1,IGM_PBBNZF-0DEK01_20221028_merged_bam.bam,c1600df25c15398a9ababd1fc063f3f9,BAM
1349,PBBNZF,0DEK01_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/f9964ce3-f7aa-49f4-b173-56b8b2ec1945,IGM_PBBNZF-0DEK01_20221028_merged_bam.bai,c08d5c2b58e74aa695a4df3d1b3cde82,BAI
1350,PBBNZF,0DEK01_RNA,Central Nervous System,Tumor,RNA,dg.4DFC/06d08c69-fbc2-4db6-bb95-ab743e5ab748,IGM_PBBNZF-0DEK01_20221028_bam.bai,f92f98a3e81d1bb0b224d08455472a8d,BAI
